In [1]:
using Distributed
# addprocs(4)

@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Test
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
end

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1278


# Univariate IHT ($k$ known)

In [4]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 0.0

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_b, correct_position = simulate_random_response(xla, k, d, l, Zu=z*intercept);

In [28]:
@time result = fit_iht(Y, xla, z, d=d(), l=l, k=k)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse linear regression
Link functin = IdentityLink()
Sparsity parameter (k) = 10
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 100
Converging when tol < 0.0001:

[-0.047392402118988805, -0.04592318602192057, -0.08559368984449688, 0.040334152795521876, 0.044611541554794555, 0.035045443510595284, 0.05291791131094008, 0.02164491382537681, -0.014100467586535255, -0.06197809597472496, 0.13419329863350796, -0.12630073461903984, -0.034556360754857726, -0.05211347631632636, -0.02882980602526739, -0.016301199411976865, 0.08223263236

LoadError: [91mUndefVarError: fdsa not defined[39m

In [6]:
# compare estimated vs true beta values
[result.beta[correct_position] true_b[correct_position]]

10×2 Array{Float64,2}:
 -0.438857  -0.402269
  0.747399   0.758756
  0.691381   0.729135
 -1.42472   -1.47163
 -0.194788  -0.172668
 -0.860604  -0.847906
  0.338021   0.296183
  0.0       -0.0034339
  0.0        0.125965
 -1.20796   -1.24972

# If $k$ unknown, try clustering by absolute value

In [14]:
"""
    project_by_clustering!(x::AbstractVector)

Projects `x` to sparsity by clustering. We run k-means with 2 clusters for 10
iterations, and then project the cluster with the smaller mean.
"""
function project_by_clustering!(x::AbstractVector{T}) where T <: Real
    center1, center2 = zero(T), one(T)
    members1, members2 = Int[], Int[]

    # run 10 iterations of k-means
    for iter in 1:10
        # assign xᵢ to the nearest cluster
        empty!(members1) # refresh cluster members
        empty!(members2) # refresh cluster members
        for i in eachindex(x)
            xi = abs(x[i])
            abs2(xi - center1) < abs2(xi - center2) ? push!(members1, i) : push!(members2, i)
        end

        # update cluster centers
        center1, center2 = zero(T), zero(T)
        for i in members1
            center1 += abs(x[i])
        end
        for i in members2
            center2 += abs(x[i])
        end
        center1 /= length(members1)
        center2 /= length(members2)
    end
    center1 ≥ 0 || error("center 1 is negative! Shouldn't happen!")
    center2 ≥ 0 || error("center 2 is negative! Shouldn't happen!")

    # project cluster with smaller mean
    p = center1 < center2 ? members1 : members2
    for i in p
        x[i] = 0
    end
    
    return center1, center2, members1, members2
end

project_by_clustering!

### Lets experiment on IHT. 

In [30]:
# x is first iteration of beta
x = [-0.047392402118988805, -0.04592318602192057, -0.08559368984449688, 0.040334152795521876, 0.044611541554794555, 0.035045443510595284, 0.05291791131094008, 0.02164491382537681, -0.014100467586535255, -0.06197809597472496, 0.13419329863350796, -0.12630073461903984, -0.034556360754857726, -0.05211347631632636, -0.02882980602526739, -0.016301199411976865, 0.08223263236673081, -0.04951701718032453, 0.09050363184140413, -0.10689058056912425, 0.017999917557051322, 0.07154420299103223, 0.10044021042074291, 0.08229191503508859, -0.053031635707712715, 0.04023595523184342, -0.00418996856717304, -0.007118392457848233, 0.07398937679057982, 0.002684558950316436, -0.11643696561628171, 0.030587080125844063, -0.08132396701870936, -0.04423651688171543, 0.025529072123843456, -0.01947355526973054, -0.15832051476537035, -0.08058411292042801, 0.017500991771158013, 0.0034877737014768523, -0.025015212612882982, -0.10715606263279016, -0.06750779271406977, 0.11452397128602658, 0.0397854616475773, 0.05541550264796351, 0.07353313953296634, -0.03962619683658096, -0.012872469633466352, -0.12913862551592528, -0.07012508394354805, 0.10234651414653369, 0.017094759887380833, -0.0405258190256052, -0.015049552664075825, 0.20667971530992582, 0.08790514673516218, -0.09669285083353053, 0.07852110123120015, -0.010889440768644688, -0.017587614544791935, 0.17346091350425616, -0.03754923850255462, 0.02587634196254281, -0.04175630607649144, -0.07122151992088309, -0.06616786620350656, -0.10748712435074571, -0.018093913331641374, -0.008645925825714587, -0.0961974036732744, 0.03473076961416827, 0.005697370155093382, -0.07635908473436039, -0.09372012300673081, -0.008257275205270734, -0.010985015436126408, -0.16077606432225824, 0.09397254316405407, 0.04152411314288596, -0.006118747474363929, -0.11429372164480693, 0.0163512144895958, 0.012421459341758813, 0.047895455758153925, -0.051790205952648674, 0.05036294744667507, -0.012101772549363041, 0.006120265246093082, -0.0240339182501244, -0.02687490699671869, 0.07167153261326471, 0.004288424826087926, -0.07069234367773981, -0.035912497355104524, 0.09412115594647452, 0.016948377719369554, 0.0903267262199846, 0.05548847616242226, -0.017857361929968434, 0.020933213966682712, 0.13824541667278017, 0.010001525905132301, 0.1108446069762732, -0.00887957057015215, 0.030971082933180336, -0.15391049658930492, -0.05816310852667112, -0.009366300957374445, -0.031229025244281932, -0.048648123502845225, 0.06556656825752727, 0.04239277067974048, -0.03364574946008213, -0.06754141880249462, -0.055424066634150844, -0.023092493596791414, 0.10049451556288169, -0.039144700336096336, 0.04108620829643375, -0.07684531770714993, 0.12563350692934946, 0.0685334036719804, -0.02178131249736176, 0.07786032129881836, -0.0752194132407086, 0.012497693062816331, 0.023017618724522227, 0.08047771919968295, 0.006439531612498616, 0.06633134969060803, -0.031759927898934844, -0.01911637289662555, 0.07698232721787159, -0.018132705617781675, -0.009778481357055798, -0.020093472921355424, 0.018128523939677666, 0.050777123195791404, -0.010394266334675889, -0.012700105303073262, 0.03522594226176106, 0.08718406513378185, -0.01207766584858934, 0.020146076870802172, 0.04852021025016367, -0.020022660015961986, -0.14162266927008632, -0.012412771299286399, 0.04158271541908852, -0.08297640454796922, -0.025128645826121076, -0.0788893591752109, 0.0649249224077416, 0.04180083537313633, 0.081708925925677, -0.004754334364500749, -0.16605418681930106, -0.08188187714122447, -0.16825024855975118, 0.01115795154332205, 0.02545595198909276, 0.04503412542390495, 0.00812205585414866, -0.017322948934333525, 0.016948507913176994, -0.007418324543258458, 0.06600906227638653, -0.017454465235630365, 0.031533895870942796, 0.0010859532386804903, 0.018803746611213746, -0.04144066196202513, -0.06658306615627241, 0.051317428117121984, -0.029421550225190527, 0.0007080393143187216, -0.06158225016376433, -0.008100714348841414, -0.03539744181608938, 0.02788709048358717, -0.03797966945705893, -0.034081470613023364, -0.0752598643224904, 0.00037091521941881724, -0.016061977671752665, -0.18482381727753122, 0.11265132341032982, 0.04113029934787454, 0.029896861418303364, -0.05606854956285021, -0.024790132191615515, 0.10504906916438228, -0.09934056607088766, 0.010784956316854895, 0.029754432083464424, 0.07068630230141014, 0.06291011801228923, -0.025985678300101137, -0.020752390918803983, -0.12469055608319164, 0.008092153015368359, 0.0011681934341080176, -0.05419705255420622, 0.09780738234438591, -0.05912350443278905, -0.04565420399144772, 0.1181435524557754, -0.03554257877327971, -0.09588140788260258, 0.04753480228469886, 0.0260284860089877, 0.14500180136953594, -0.06875010888148399, -0.10131061474665778, 0.012301222704091288, 0.14136535769194417, 0.07860062196040947, 0.0013935722860143702, 0.03904866084156507, -0.14709064779555336, 0.10664983692739176, 0.006074092945720149, -0.005924327912682028, -0.029223400836564217, -0.16791903933414856, -0.08334512397976074, -0.06777814744755813, 0.13425911716856026, 0.0405998354940877, -0.04800004781073761, -0.01228159950111381, -0.06636515735397792, -0.06431546631018878, -0.008986382677699377, -0.13414035294941345, 0.050083420006561206, 0.004893204482428127, 0.00422759709994082, 0.03088059025590911, 0.0878656673800815, 0.022958473923569655, 0.017172135687376813, 0.03436173174178561, -0.03930475672625946, 0.0747836339322078, -0.02964127488602227, -0.06965853587347057, 0.031635573042175874, -0.007703598872177382, 0.06907987653982939, -0.0019103397884716602, 0.10202796618259659, 0.009471869573409251, -0.03003563167314528, -0.11385404551596152, 0.12305219518331804, -0.006678973973135123, -0.13581434698492775, 0.08051319845599607, -0.0879828726822655, 0.0706893214592037, 0.13796420909331686, -0.05613702720747653, 0.022013298344066963, 0.07204291931066506, 0.005336436618405424, 0.03260377437455557, -0.16293180400267804, 0.0740716563659126, -0.11434818235940142, 0.07250330718855151, -0.06543723171574845, -0.01801586362608182, 0.0024874119240268057, 0.12279379105428326, 0.02399148194508808, 0.10675507100152987, 0.06677532498077858, 0.025489038519279077, -0.010428505109131412, -0.17815617498220288, 0.08338244145245373, 0.07291854023830986, -0.038728494206120186, 0.07320167016866133, -0.038456419805755565, 0.07291372512281574, 6.983920504087262e-5, 0.0020979658028431485, 0.005217073573439524, 0.008564880958856243, 0.021027572599290494, -0.04903398636235771, 0.13133592223270077, 0.018758499291830537, -0.05464961938451373, 0.09345827607782564, 0.06730836769306975, 0.17270754136848074, -0.006506482955860643, 0.09715968582562845, -0.008379540952991003, 0.025293868926613082, -0.020621271920899128, -0.10954896914586898, 0.05022333894864619, 0.008413060154331426, 0.06361420025158074, 0.04359772695716172, -0.056262367537877195, 0.07098928857138027, 0.05436858338304411, 0.10593055978590454, 0.18277155268675552, -0.05454105083440472, 0.08287710983180092, 0.02781208478643912, 0.02365763376959547, 0.026568617647586973, 0.024396185456849982, 0.13172839240721193, 0.014846691446584442, -0.023868251928520465, 0.050361452890948406, -0.025897330746102527, -0.07780216509968176, -0.07724949067616854, 0.0681329000529236, -0.08028354083118742, -0.1839803783795196, 0.07122596666964247, -0.09009558188607343, -0.04712619693789463, -0.02829430037128776, 0.040916513956924826, 0.16490899730118483, 0.024708219004527143, 0.09050558048227292, -0.027191748448696117, -0.10493069148405376, -0.05898578997054205, -0.0019395416852886907, 0.07033434911553686, 0.0708558540554652, 0.05689943422942733, 0.03533868229042925, -0.1832041540493564, 0.02331309155836552, -0.13885949039415757, 0.04525758868921054, 0.11492389478325186, 0.017223926289304033, 0.03193989729584933, -0.09987374759549911, -0.04473990539861892, -0.05886715242325514, -0.08387721010867132, -0.15186884223824448, -0.05274294355404084, -0.0005283030215088106, 0.06297952466406098, 0.14534160957626033, 0.003383123882712455, 0.007133390266702636, 0.08205358605042742, 0.19407999161224107, -0.03620953428711868, -0.0831037225471706, -0.09637126142762102, -0.052379536361341704, 0.0021781169782997397, 0.04800702986673291, 0.05759597806722811, 0.07437284227781531, -0.014021583713681682, 0.046961681907052674, -0.03140158007388959, -0.04966422637719664, 0.039613000927801775, -0.049811556280948674, 0.04435005963961211, -0.030964916623223985, 0.014227283125541553, 0.06671689183704566, -0.0005699366310332098, 0.11255339883448143, -0.216587613853378, 0.06246738552617926, -0.04525402392455021, 0.14781650701211588, -0.03699216821420912, 0.021905323650934422, 0.07966888263014787, 0.03932742998593019, 0.04249719682045537, 0.04526882466724004, -0.07593330239628122, 0.06727723675006915, 0.03617045390516286, 0.013312095233049312, 0.00212119006469606, 0.0031823030482630485, 0.04970853079627866, -0.005586489638207559, -0.045219521017170805, 0.007432014921755379, 0.011591992523005306, 0.03009254020260787, 0.06323681400849032, 0.07118979804848337, -0.05186718376430112, 0.09030871565865697, 0.05594207250019394, 0.035793421559260796, 0.011554450356529003, 0.09686111959972295, -0.0554198731466896, -0.08952284739864538, 0.056273473957316765, -0.06232139020110431, -0.11585872847396414, -0.0832800328728965, -0.024926921363961685, -0.037532567467776626, -0.12153728960827745, -0.06058521050049637, 0.08124238085465427, -0.13168983829073935, 0.009605872666881433, 0.0038924654153027665, 0.0024523082090306837, 0.051506048306802506, -0.09200539555752202, 0.028119533281789195, -0.009742953067166035, -0.09395600106012311, -0.024636538760309117, 0.03816675448013687, -0.014613049607912452, 0.017142750035491466, 0.007485463332652464, 0.16132679256785096, 0.1259764187496765, -0.08597956904450223, -0.043190482731847835, -0.018302681650267064, -0.03728928867190185, 0.042565690843571, -0.05288047934330461, -0.1448376614568741, 0.06201188907324896, 0.07651492467640582, -0.02706018626777364, 0.029033990839821364, -0.02488642531638961, -0.05448145220171739, -0.030476363169209056, 0.10328074822656193, -0.05424099376650901, -0.1606448192664241, 0.055184834456931155, 0.11816476905318303, 0.05675127300542165, -0.09517522436184225, -0.14709323465082394, 0.08369573030048962, 0.16237364830353554, -0.031230327367502576, 0.012815567047685023, -0.042897122612478654, 0.04420940947961232, 0.048981924878563024, -0.17754032770084122, 0.03950230469075679, 0.057785545138490796, 0.045652216359151905, -0.07528685860958238, -0.016809962403327338, -0.05326619593465959, 0.023547382317776956, 0.048743983267705966, 0.053532471626948026, 0.06625284649929927, -0.048860325262006676, 0.04283774587345031, -0.07433881142778902, 0.0017931690321148393, -0.10267258163172424, -0.05251526757064165, 0.038379937608201575, 0.0550620060491877, 0.0022473577252081853, -0.03203898569563307, -0.01697277041083784, -0.04172637308331535, -0.03752492758149428, 0.10138602807252625, -0.05064251702172614, -0.006400878797322528, 0.07153663899410506, 0.03272228699236267, 0.039752496503435174, 0.03248599055606006, 0.019014586459644177, 0.0508825786895117, -0.08345263346398138, -0.007643822143488625, -0.01165141554143334, -0.04205782463690207, -0.1070053926243333, 0.05217278250283162, -0.09382667253499873, -0.07587543169034075, -0.08239369029428745, -0.04876591444335103, 0.05525913187352219, -0.16857525428535414, 0.07762245839764864, 0.07551464363130145, 0.06464741220117602, 0.011005950782266578, -0.04295122217014324, 0.03066641556724054, 0.1057720788302619, -0.013370508386291495, -0.025902181956693197, -0.1784519677019804, 0.0038686926545618873, 0.038747470535108665, 0.03069239176419413, 0.05428143896611603, 0.09246920503234754, -0.06556229197165198, 0.03836714780802329, 0.0827181346812369, -0.059509909257053795, -0.009790238143587015, 0.0807027282169083, 0.19484844070302387, -0.06475978731380175, 0.04215500799609651, 0.10962143476000519, -0.10795996778926403, 0.07368610610238015, -0.06295197067547025, -0.040409120475275874, 0.0641648423354087, 0.009529461942088589, 0.08938147979027546, -0.011637956022729042, 0.008844177506602451, -0.07559272695280814, -0.03675741737420948, -0.04478927121341628, -0.10527867297719709, 0.009219436462319106, 0.07847271486961206, -0.03756753007023095, 0.03929811212598747, -0.05169281532890742, 0.10551000055559025, 0.03390222835797354, -0.04964550055544441, -0.07723582187829607, -0.05918172259413068, -0.029602376430095837, -0.0351509928854749, -0.03287336296686709, -0.045799422995264835, -0.051611901372625564, 0.17540466608623628, 0.037189718722249396, -0.014036787057751058, 0.051953224157969984, 0.016376099139123194, 0.14542479824021812, 0.05816705016087469, 0.07690140721657102, -0.03948369818751275, 0.06971782571330534, -0.05157526968985595, 0.06130174735989841, 0.12285965708307307, -0.016377396797018447, 0.07562511651044274, -0.09041471992181503, -0.053952710958584574, -0.0040294957592992234, 0.004826793833625348, -0.0021298153659602757, 0.06159240428366144, 0.15932350941612797, 0.09385898517101054, 0.016047616986243605, -0.061008679057770125, -0.02799950548824155, 0.03626979206552889, -0.036761918270170287, 0.06155149574223471, 0.074782694845978, 0.036966979855797874, 0.06828939492027358, -0.09100201232646456, -0.14543350770890653, 0.08915124706432527, -0.02299830363287891, -0.03426315398249774, 0.19671997268221572, -0.013632375773279944, -0.021162633528863346, 0.03391108206236938, -0.026708770922598962, 0.12175473623642272, -0.033953445810560604, -0.09872670363521077, -0.057631889380454424, 0.08179641154734635, 0.029487358517164605, -0.046304685611440695, 0.0014551924619264466, -0.05113274996845874, -0.028961622958447713, 0.06458469542665547, -0.015404532157629937, 0.06336795046893524, -0.10340616923719359, 0.026713547984895867, 0.07653149533051738, -0.017928379594746264, 0.038732528128132174, -0.0247907988394833, -0.007762411322523706, 0.026673075818980298, -0.018365691989646375, 0.004845582934065868, 0.09141863471145578, -0.06529727676352529, -0.09596641198224992, 0.003954132509584371, 0.07647274680004801, 0.022307459164556828, 0.06951182301138431, 0.07363634253896743, 0.04842549514247748, 0.0943933807279586, 0.08506798385875025, 0.012226653395907693, 0.07468171932111228, -0.04803932357535335, 0.03396223048309702, 0.06122138162847761, -0.034998578344294165, -0.031509872082147485, 0.046725361602131546, 0.10194688688125073, 0.013048895795701057, -0.023325231268000953, -0.016629041674727695, 0.016561452780814827, 0.06788600766645195, 0.08596720591374275, 0.010902384045329847, 0.02547200044979724, -0.03660629113620445, -0.1305682242743222, 0.13017456961745724, 0.018668014835651083, -0.07354526083247455, -0.08117473396858164, -0.09095380434538086, -0.12598905942500685, 0.05086942613025385, -0.03947852759799036, -0.0344645535768168, 0.07814910707019991, -0.07708899807263746, -0.04107337372513974, -0.048315228217364045, 0.11632630339032311, 0.05698897305245399, 0.014962337420714029, -0.02706357766616233, -0.04861285080467182, -0.04042381567295393, 0.018673739071502077, -0.017103223152586274, -0.007651679615861097, 0.028846857111565018, -0.021098538054211153, -0.1079370398098949, -0.02242194358454397, 0.04516135958243629, 0.03545351983252232, 0.026474672675781602, -0.08577340096377856, -0.015272342608098023, -0.025058977957510935, -0.03739381884823885, 0.04322545813551026, 0.047442243286638576, -0.14765296901843294, -0.015159265480292753, 0.07560220993302474, -0.0115563722921721, -0.0047368537425584445, -0.0009272334135748942, -0.013858182602329903, 0.09955277384412409, 0.010326374237207283, 0.07179799307483899, -0.06561155518262904, 0.15314335865549603, 0.06343425202756127, -0.03569601885151702, 0.032642776511541495, 0.029286208313095734, -0.024488350822990095, 0.10858473068919967, -0.06536802219382566, 0.07821975993587207, 0.05700055073332036, 0.012900632608467869, -0.027070550307488665, -0.0022138205098396328, -0.020839380126486787, 0.035307073384106796, 0.0396239160428178, -0.08523009109862294, 0.06169994363987507, -0.031029884350013446, 0.00035868315731391807, -0.03717965784448329, -0.04820315254148382, 0.04336744718778435, -0.08331806825152278, 0.020533520336042596, 0.04610517873859371, -0.09701757617074697, -0.03343994117219712, -0.040349721034670716, -0.019933984315609127, 0.06925745260269159, 0.06347471725628122, 0.056353091425170664, 0.030525799997968495, -0.009838815563052557, -0.15163677229312553, 0.009973695689974795, 0.07519226282724793, 0.015598805067257216, -0.011225266576041082, 0.05355082567528623, 0.017276572808642022, 0.11152078689822016, 0.02035279243107947, -0.011302133548746844, 0.026584519336011657, -0.11284663009962574, 0.11020937302598033, -0.14929995475214028, 0.13958607281770072, 0.027760333451242538, -0.0870747586812583, -0.08207907419864097, 0.1142889911243218, -0.008178289977316715, -0.02854836972659148, -0.021501135272623723, -0.011556833882098242, -0.03934326011175519, -0.001446848811128924, -0.05967051258841452, -0.01585918515711775, 0.048231181659076854, -0.014896042887108668, -0.07691873471188428, 0.04732436328313009, 0.03591466138042303, 0.010584999357691393, 0.09074742615378167, -0.39436009802924665, -0.01078316564420533, 0.10804509739545196, 0.12876215470841618, 0.04280243566335762, 0.09621424814894969, 0.08518576740551638, 0.02595752139157407, 0.017264190034840362, -0.07707321395702907, -0.12187671807926692, 0.07238576397138548, -0.07732188297144701, -0.005608870154120739, 0.002584557533455891, 0.08826761086134502, -0.06681097366615414, 0.03092005601807024, -0.05525986520177401, -0.019077021279688213, 0.11000091206055035, 0.10325010842314995, 0.022364768656283986, 0.08266280498251145, -0.0652000719335474, -0.004793832393027418, 0.0004340473924108809, 0.06083441186002697, -0.057652133221616525, 0.02081367253336614, 0.06304153513976685, 0.016332796160191603, 0.04377306966514933, 0.040903743591469775, 0.0026800585216350324, -0.039722952045186624, 0.05808378400130198, 0.1277797023777133, -0.0007897175263719489, -0.05190642876492088, -0.1333564182617684, -0.055932518982844834, -0.07097245752963895, 0.06979104830981905, -9.352466270833234e-5, 0.031041758742369013, -0.057470337342030796, 0.020501561617992162, -0.10422450830033961, -0.11295893527175953, -0.044041020435931694, -0.105640082157362, 0.09832889911006644, -0.07881461727865646, -0.06428969774032053, -0.17072563879494862, 0.11894874131063113, 0.03244684521775369, 0.11124625696291444, 0.06962008902085412, 0.0876589368169981, -0.022652706914170328, -0.06383222934624527, -0.0987541554201234, 0.08404396655046559, 0.003434099113053964, -0.02017590384582786, 0.07216079862410689, 0.0262938058182744, -0.022355513409686167, -0.040236719503799166, -0.09085733405885499, -0.1464670297222051, -0.14635252043021812, 0.12299388482737718, -0.01622395950452232, 0.09148157604105725, 0.15673512805447312, 0.03151470924875022, -0.019168595239815598, 0.0046193426025155915, -0.02870269087055617, -0.049743977745140465, 0.09599613853032318, 0.005540697955470953, 0.05817581639992047, 0.08440818298486817, -0.00697847086455935, 0.007677292911152922, -0.00987351284736691, -0.14042740983778376, 0.01828941691731132, -0.0007769231894356822, -0.1910191819603028, -0.08199645327932112, -0.039731963071735335, -0.10735511149217059, 0.04860660794937068, 0.04078589553055198, -0.012658069550025621, -0.041087222443499424, 0.07225629274425309, 0.07751621739809487, 0.06895686669937691, -0.040679578373937895, 0.03406834494396789, 0.17305165548524837, -0.010606907677149651, 0.010609636521189967, 0.015750408498300353, 0.010360075981449255, 0.04878168625804975, 0.03118053651648293, 0.18432689437522065, 0.0038320764663206134, -0.029094400418687015, 0.04290035745072394, 0.0868977996681231, 0.09142996577364221, 0.6872933031873395, 0.004842278339926296, -0.054306003032112256, 0.08475531234316488, 0.1613106093799429, -0.06996632000835899, 0.01085742198735136, 0.031842390711549866, -0.13953961663582287, 0.019053691879056155, -0.06480474904075872, -0.0749538059607586, -0.10741151183835118, 0.03486697165633435, 0.006770476370584447, -0.16021725813284285, -0.003027836882972773, -0.04163383047054897, -0.033500701983078134, 0.03851841738899847, -0.081812152348814, -0.057363443475084576, 0.04504541962183682, -0.026635978475912047, -0.09171895552418638, -0.03446297929384598, -0.05447392907671902, -0.05556421976084817, 0.007360068857050565, -0.047641955898356, 0.001280782084768255, -0.0739125792248384, 0.014200801224820777, -0.00761568169496214, 0.034689070228645945, -0.03926012588300198, -0.0017253034938463994, 0.0617135984729317, -0.03088356208060649, 0.0645675044840875, 0.011163055823807859, -0.007970597861503222, 0.10070850287020029, 0.025667273611578407, -0.02365112482578553, -0.047688527969136925, -0.020737701465296726, -0.09994880828878798, -0.03128842082691844, 0.07185659945705201, -0.0828426515783121, -0.07082866705794932, 0.04201032977554369, 0.009120211550118357, 0.013106352550198618, -0.050168019100833836, 0.16045346482461925, -0.09522690719566304, 0.009258247989742138, 0.009790890051595932, 0.0464347027620691, -0.06708070226101703, 0.0524068249854139, -0.08550022553276256, -0.06597432921379168, 0.0075967624913265705, -0.09585852634948686, 0.09133527215170342, 0.17921872330941271, 0.0013311659883053304, 0.11893722337952331, -0.015735542895461477, 0.08681727924255805, -0.01140249569063329, 0.02343519783632117, -0.030245154899998617, 0.01586173354718875, 0.08030144985284854, 0.019929417667003797, -0.019875023866268954, -0.0038735792327000617, -0.05157560712332802, 0.021067415443146643, -0.16397781081533275, -0.0049957916815705815, -0.0012379342746584047, 0.007003182641191448, 0.040480809921046704, 0.11036997782573617, 0.003348648458788751, 0.14157706998992006, 0.026564205032689834, 0.07755002715665195, 0.06853395475234035, -0.03397464023026044, -0.06776302844154948, -0.013421792609919539, -0.07854178784874753, -0.011715022315792018, -0.11119412027828438, -0.0842674392370125, -0.04422739155200419, 0.024844295322402555, -0.0851790964018653, 0.0012048467765794044, 0.03437537168689408, 0.00042950791794464904, -0.001134231768073746, 5.7365075817521366e-5, 0.08555973585278845, 0.04024516018927343, 0.07138402344098191, 0.07319655303423987, -0.058461224975205124, 0.11927991236594829, 0.09587129149485578, -0.10558367904300997, -0.07832885490314322, 0.03803408895524183, -0.013930946762051493, 0.0954330826568741, 0.01620345685251289, -0.1450108100629958, -0.04151220120752802, -0.011405562318531744, 0.0017300654334460178, 0.06562590757133444, -0.00012294055923463802, -0.0709239119031986, -0.05597496243468134, 0.13615058088998447, -0.037627234482702485, -0.0489996154328782, -0.0031875885454813423, 0.07207464480788393, 0.0949651612200298, 0.06775983304641096, -0.03329703019196721, 0.006956224454235714, -0.05838254674064764, -0.02409848731549533, 0.001746843082820001, 0.168686025371908, -0.031048555427056358, 0.042062683859078985, -0.018949195426342602, 0.01572380838975686, -0.10181832249248962, -0.02728901548351202, -0.03535234554108049, -0.0320093144523899, -0.01910403739505247, 0.061684226759965474, -0.027711063626544987, 0.07948318283944467, -0.1317914083622829, 0.10362320059936492, 0.010535700304166038, 0.1554836263674753, -0.0034944723014758436, -0.036736797125455946, -0.023326163403814805, -0.04464664360040059, -0.044822774833313726, -0.034204532939575476, -0.1432718896725507, 0.020405712726792338, -0.04009804088323133, 0.06631102958802694, -0.040442487247624476, -0.03480052143362431, -0.007679039971006487, 0.0562545832602872, 0.036464557682941795, 0.0704021975052554, 0.033203237692892525, 0.07365837525043321, -0.0018939483274787732, -0.06668184941148979, -0.015378647250434075, 0.07434106955537059, 0.024520832896708446, 0.035575412105188164, 0.119343491271729, -0.007380368390651411, -0.053401822560633395, -0.04883142612837282, 0.09399948356823958, 0.06437103568058358, -0.05649007326678557, 0.024140190651357713, -0.057305801939225263, 0.03639124721834021, -0.13752435256441664, 0.03731403724767965, -0.03650423202461316, -0.016377522454360208, 0.09380661240109227, 0.013187410840807441, -0.021456869503903096, -0.028068732324435963, -0.015546345644839832, 0.060840020630262046, -0.08611407704922909, 0.04465407750451483, 0.01604960267378027, 0.07496858968753779, -0.08030842811259416, 0.00889243829060234, -0.11198778090277436, 0.009231956466266827, 0.08188458335879385, 0.04258209221218467, -0.033679702970106515, 0.11382816851544715, 0.03523521375112433, -0.04201147634461148, 0.0755267395984148, 0.0351165107726305, -0.09533497303112921, 0.05338692271783052, 0.07795531745230441, 0.1430068985719659, 0.016399136634891252, 0.07852197590387804, 0.051863706238700215, 0.04689209324783901, 0.0752476941523914, 0.025522590071168035, 0.05741846125459512, -0.0759210955509005, -0.0561478380269986, -0.06718053464050203, -0.029236085411825093, -0.05316037901694575, 0.029114196647078882, -0.0032451769559049726, -0.037509689159726495, 0.0038251023679714636, -0.12741573551415544, 0.16356682216571894, -0.061246849434123674, -0.1458993015671419, 0.023332835028400525, -0.0029809171656243276, -0.02540869691436091, 0.008437871228539093, -0.05004755153087423, 0.012051578731411923, -0.06718083945999007, 0.10987415239170303, 0.11550237346706416, -0.014243103410153567, 0.011514033649892872, -0.021173067965147292, -0.011407985309370022, -0.06339371834786031, 0.07117350516802473, 0.034521440674067115, 0.12748898725920485, -0.023292508419341253, -0.1178486769142673, 0.08055491396162533, 0.0004880084833772147, -0.028660583815801714, 0.04317522245811036, -0.055919578234410246, -0.0137991145791727, 0.07303552824720799, -0.030137596937126743, -0.17228278632507088, 0.05395959698163153, 0.03487197357186222, -0.02044970545253128, 0.015426630654257919, 0.04941039324032651, -0.018525583513287834, 0.00695214357740952, -0.012040151542831836, -0.042309414955191284, -0.04117654697007166, -0.026844583235491458, -0.005196114418350832, 0.030441451292935944, -0.04614463789557546, -0.007059724062814491, 0.02780651357220697, 0.0317569153364708, 0.1282915792851984, 0.038252077929831214, -0.062368113251697464, -0.020645630685235505, -0.06451166473956658, 0.0591068317146831, 0.054837455013065084, -0.054568509522430864, -0.04095994163690266, -0.03522675253402551, -0.0769922772094645, -0.06265923529984327, 0.04441288050689699, -0.061576670812654874, 0.004589458322460591, 0.5782451914761512, 0.05074968131548936, 0.006522874320257308, -0.07218700750979284, -0.01604089418178856, -0.05278756077306721, 0.09994223118333513, -0.032303004483498994, -0.038421127384525654, 0.007425796122750034, -0.007274565442070487, -0.07123332462794807, -0.06071959073085013, 0.008321036418975409, -0.015408257603486776, 0.11489118117639853, -0.07274245932971993, -0.09008245095867329, 0.08919008784137546, -0.037935720342810744, -0.015516697888971016, 0.044616573358577126, 0.0008512907474608578, 0.0023722583602525694, 0.11493608644691715, -0.06817415423600227, 0.041022405309828466, 0.00012921710257402107, 0.12437091847718525, -0.05273697928550079, 0.013244625480576084, -0.018744962862859083, 0.10395661003710525, 0.04839827653819466, -0.009520328660550643, -0.1179627359555285, -0.00450527835458501, -0.05808460837210963, 0.016366894222085714, -0.014412490082795746, 0.004024635839361551, -0.03094714720813943, -0.03481121901271162, 0.09729317429646173, -0.06404423261472895, 0.08185086817027641, -0.12041264266595349, -0.0047584441512433205, -0.028384075735215973, 0.01247468763019766, -0.03290507612847021, 0.05901069424302323, -0.016039769736861435, -0.012002506706982285, -0.05328381023487736, 0.03151991525220094, -0.11596458003951428, -0.04274913122699806, -0.005055364188890148, 0.04070290478933622, -0.004319320795172477, 0.014027633637580405, -0.004156503129767902, 0.08990501337183382, 0.07639739796982242, -0.10176906063151624, 0.0021182449462064106, 0.0006979406597297687, 0.09227266369425832, 0.031955188248027774, -0.056082199778825996, -0.09129064044149655, -0.03980846080977637, -0.0039602379682859845, -0.03758188542392401, -0.10059619222281828, 0.11108971447400874, -0.09094089616366414, -0.021624837957035747, -0.08831853474959037, 0.007052709057052648, -0.05491830038806579, -0.14157923946341713, -0.03006631735952504, 0.06887604154650429, -0.07203655847196277, 0.02995728437197682, 0.03661035524246738, 0.02541818849778081, 0.16878733324685782, 0.0821770837072003, -0.09547593062912543, 0.08601101545809255, 0.005747238296573285, 0.02539183894570242, 0.09811483298579958, -0.07107104086307922, 0.0901069087188754, -0.06502645870457789, 0.0074531442029529904, -0.03716949275197071, 0.041827639885586285, -1.2628795275335738, 0.10691997847002765, 0.0011876610087860223, -0.062195402097931014, -0.13572413598117297, -0.07293611190897661, 0.1020365352099441, -0.07843271376410667, 0.003972722653272829, 0.07453973402590863, 0.021182059257048483, 0.005348013307684307, -0.07292111553061849, -0.0852564740479801, 0.04891797911627684, -0.013926974421303875, 0.036875867608557066, 0.04604146829701453, -0.008199003364897426, 0.08077312497102808, 0.10144369355157765, -0.002537150271134099, 0.08859279845050101, -0.0752321764973187, 0.06170572799025565, 0.11123303447931115, 0.010873142538307552, -0.024772065642643174, -0.07098207603334959, -0.0432696124228319, -0.06126905608863255, -0.06741342439143559, 0.08802494473242924, 0.0685952122112255, -0.00338100309573338, -0.04402745290055375, -0.09703522401036334, 0.03515041677119202, -0.01863220549343736, 0.01236754057128304, 0.06043626403488603, 0.038848533661686106, -0.037590891405361246, 0.017305587505401655, -0.014312196470332437, 0.04994353741023326, 0.045335340030416414, 0.0890057813445232, 0.045716729020695096, -0.14440284745248066, -0.03441456313935703, 0.05727278211991422, 0.05618981374060074, -0.06800091351059893, -0.08588954930393598, -0.11560729864325986, 0.010937973353517694, 0.01910755087983452, 0.05228811094110853, -0.001156536858090226, 0.014739027284420074, -0.04471253520273491, 0.12633914715546316, -0.04998083213513056, -0.048454616147423454, 0.019012262425364634, -0.010613239605247427, -0.04344354976493233, -0.0008167053622753042, 0.039712023247588296, -0.056134046909019714, 0.042198870057137544, 0.018753722619932408, -0.06421268281281475, 0.061702443674748614, 0.08157621145309638, -0.08184442653577391, 0.19257946379985003, -0.14738203323329865, -0.08846325524943009, 0.10682975810983625, 0.04915944036141198, -0.021509412678462632, -0.07121116936302978, 0.048130721629884285, -0.02484629382379323, -0.015826741514612212, 0.20043417052329085, -0.08752376469999212, -0.03416426223033361, -0.02261715897079238, -0.012497972776739244, 0.05029054610918137, -0.06255794164895738, 0.18403517456698912, 0.06674225824630099, -0.1128121419243463, 0.14498311036028008, 0.010738639089112987, 0.0884129997566855, -0.010662935815047795, 0.0579603636106332, -0.010607376086218723, -0.1270108784263861, 0.005055873211031356, 0.09816895887155794, 0.03144975153807535, -0.09387090050711364, -0.11914919804469037, 0.049041203070816886, -0.05231189921793793, 0.015285628227488317, -0.03698280569306834, -0.08501369864588676, -0.05461083104887299, 0.10296729702320917, -0.011381149921765511, 0.03572253069090935, -0.052987806895298424, -0.05686733700803957, 0.11406031045483474, 0.12300666344605657, 0.015017462683835858, -0.04505659827465734, 0.0327707299025135, -0.08055390420473302, -0.09125514018887669, 0.02200503975656256, 0.076047073755473, -0.0017326621116478995, -0.04559621439299959, 0.07554274759580268, -0.04176273271772072, -0.010658877937942134, 0.011463296598074308, -0.08428044833301354, 0.03192380614903419, -0.11442242494083028, 0.06069553052010757, 0.07851750885020119, 0.0608393717729264, -0.10310843053573975, -0.05489875369415872, -0.06023014280319514, -0.06678804345292065, 0.08598570952228801, -0.017751237488807813, 0.006927359453936525, -0.07115542252861656, -0.02447921904453504, -0.1198835778866467, -0.07726939381960615, 0.036458883167916664, 0.027050715565647074, 0.022833604810386623, -0.07417235554491222, 0.08498572379076196, 0.0060560322781283565, -0.0038658656297924387, 0.10145898439873678, 0.11107459346913734, 0.036254590388090875, -0.04938955297427657, 0.053880951346739085, 0.012214810005433997, -0.06050355351122132, -0.12471856869484947, -0.05678044596437264, -0.002899345943207623, -0.05882521381519511, 0.1479197182105442, 0.05730306083303066, 0.02040533752742214, -0.06541844313680246, 0.06347059716962529, -0.09481866468054873, 0.0353796599058229, -0.14417077774357046, 0.07475861435006317, 0.0188560374958497, 0.04756994911222069, -0.005837635308238147, -0.051154700905096384, -0.08784829835245293, -0.038304686845446065, -0.09299809759268653, -0.024321331484798234, 0.006861958033330006, 0.10569705068827322, 0.11268782928888349, -0.008479155499023991, -0.05582068216133249, 0.011625756272038767, 0.051082709221847454, 0.1489854723465724, -0.023691871631281076, -0.10413331351914676, 0.09209611296418789, -0.07963526992012172, 0.006711537390217191, -0.0428507524125597, 0.0917984141971569, 0.08526158451212047, 0.04593007761901649, 0.08937174002845831, 0.0745326450714806, -0.05078407919410667, 0.030700010130594334, -0.04545464549847699, -0.11116856490022035, -0.05038491169739313, -0.010455371652658947, 0.09076270318505808, -0.05597648678198276, 0.12254508190512223, 0.02865444849133325, -0.09631936049874205, 0.044007391119719824, 0.020021253421612675, -0.002561021459889139, 0.13489156064277194, 0.004871974826927632, 0.05738298426548174, 0.024197562836160796, -0.03789671426041052, -0.0462329225551141, 0.07805268835087176, -0.1349048032581283, 0.08364308849595224, -0.03506733635832053, -0.09462235251646056, -0.029575647421189462, -0.07499454008336068, 0.02195755144838062, 0.04067329022692313, -0.050327463930509894, -0.03929777281575295, 0.06083757706266938, -0.0054393136002878, 0.07299864676207964, -0.023685493343798183, -0.06737823180589425, -0.028161782108143116, -0.1352066986987829, -0.01722310477489151, 0.02437738369125076, 0.08832882285207744, -0.08307120634966042, -0.045034829340446565, 0.06632248787541767, 0.12639440584956793, -0.005590877483168497, 0.10114142235233035, -0.01056130883456374, 0.05047120024431047, -0.03604658799903618, 0.11419560428363444, -0.034502421663286126, 0.07377523479259672, 0.08878189696619108, -0.14223854007134276, 0.0018543628275557614, -0.01680399293772523, -0.1875202053559137, -0.03166609594622546, 0.007632041004722737, 0.024678660936672852, -0.04916163856629933, -0.14579122346487808, 0.0026383415712581937, -0.015016903726641944, 0.035104365806803275, -0.07910874938335334, -0.06092977504954172, -0.08095940619033215, -0.11594622437922449, -0.034797828656534344, 0.05728012688008317, -0.11053406326352413, -0.03979585732473681, 0.1462320929102828, -0.06851743357397448, 0.0910075659318723, -0.0634348152466024, -0.0702980185742177, 0.05763952783681843, -0.10725108105846214, -0.013863689268062083, 0.0037351932035744316, 0.07180056195169421, -0.07158838841061582, 0.16019897538464975, -0.028461470059236497, 0.03446384879816517, -0.0018147582792453776, -0.024138472486833998, 0.042707617584324946, -0.11762155602241592, -0.06300587655616428, 0.007754301370593472, -0.007274544420477511, 0.0009298015639106864, 0.025533594771591617, 0.08374648661319302, -0.0011616618779357844, 0.019389195795771953, -0.21743109175382197, -0.0037067617380071934, -0.042694408951078684, -0.09370461512131212, 0.03433870042998473, -0.03770907787501883, -0.04524568723216581, -0.007946937213460207, 0.08052293053283352, -0.026434167247147724, -0.0240021383200569, -0.018184807581068625, 0.015658672974230046, -0.07161036914959752, -0.056146265941784644, -0.0703651082558992, 0.05613248216874609, -0.0006307551612048742, -0.04884394542470418, 0.011796387362071765, -0.14905618843642074, -0.01655974055168729, 0.021286504836011005, 0.04241022290927556, 0.0648844763859604, 0.029848869459653215, -0.11811349722248383, -0.0038995297849319776, -0.0035937112466554863, 0.0762845678999545, -0.13101313946431548, 0.01638165398844873, -0.09103516267004721, 0.09010069808073222, -0.08129924014476933, -0.01256671590169275, 0.035892433373071186, -0.01209474692197892, 0.019556581531525664, 0.04817782699854561, 0.044843765330643284, 0.06171596292844309, 0.005394818661128278, -0.16397511665697104, -0.11040059875015563, -0.0760484171291277, 0.07258745318021156, -0.050762083830143814, 0.014460874100761528, -0.0324165695910407, 0.04262712312105401, 0.10857613948285633, 0.0372504711037808, 0.0679114798451596, -0.021824687890388935, 0.013578304964449965, -0.04229038984882663, 0.03999618854252043, -0.008339482688425036, -0.09649139970032576, 0.053620214058430864, -0.004845880787923074, 0.04357672779434452, -0.057751958255982956, -0.03499372669386758, -0.001337091568580452, -0.0319541304423611, -0.01283238408909347, -0.04306021805657448, -0.0271791651421683, -0.0033494056852048817, 0.008689957256806186, 0.04112728082818645, -0.10745533557323426, 0.1201110116335477, 0.029787442564392183, 0.05998869423483208, 0.0903572176891356, 6.71136418603428e-5, -0.057644379472875464, -0.11319652538862537, -0.06564868047399362, -0.11268491523479605, -0.04556048717594972, -0.049761729038866866, 0.008080481984641936, 0.11131950535839544, -0.047815127555722044, -0.037352308458912326, 0.03588697285617381, 0.03898919674329708, 0.0262549553507193, -0.06361494422855446, 0.07921772040957802, -0.11293164835767107, 0.03734340932380729, -0.06110400846170404, 0.025669308784154774, -0.023778492797260322, 0.042914421049253644, -0.04225678468600499, 0.05664371251311743, -0.003866937222466306, 0.08097999759429746, 0.099715680447194, -0.10185545380651308, 0.07290051153633301, -0.09739627974776296, 0.006743843731549067, -0.04993961826493161, 0.07997017582384414, -0.033657607032610844, -0.028366066855421902, -0.051794555639015576, 0.09224635680512401, 0.05273771137680157, -0.07471156522943666, -0.03570889631677991, 0.029833820001515958, 0.028935667614889166, 0.0526855532306853, 0.08341042889440302, -0.08975139551024063, -0.020751202434401397, -0.051511482459531216, -0.09431560433576194, -0.13635497502748042, -0.04651259101327636, -0.07175902138910532, -0.019421614538487595, -0.1584929821592346, -0.07954259237006583, 0.00043191591618490225, -0.13343532222407964, 0.05712426277838711, 0.04980855734225377, -0.08569212417467552, -0.00966081872081716, 0.04984820001672511, 0.1013192158510715, 0.06795638940414495, -0.06186879826905376, 0.017193590803096872, 0.053517383800006195, 0.07448451298352517, 0.07153413401472927, 0.03472343313021459, 0.02613471063726315, -0.0025739051843159054, -0.03467685379506835, -0.004620325757867774, -0.039100810948147, -0.07480846834301058, -0.08599773528506348, -0.07640231747883743, -0.08343865640633832, 0.07101020025489095, 0.05113163770019781, -0.07365646022025929, 0.11734868397458036, 0.028837771619666008, 0.03179804824582735, 0.13425505943807073, -0.010312763092962942, -0.07653532656366267, -0.18959260318847085, -0.060583797418001475, 0.06904026720192201, 0.09116692268386953, -0.0773844382801469, -0.03606601324276126, 0.09740466880751109, 0.001975326827397883, 0.02702339870953478, -0.0172394121009821, 0.13204812352276799, -0.030261722895974265, -0.022411374517737715, -0.04178129097190268, -0.05339240470011741, -0.11511181082681407, -0.0011763383898488527, 0.07362916916405243, 0.12668493592042113, 0.1200851647899932, 0.0684949837833021, -0.016817119244366196, 0.07017609086062097, -0.071441747862039, 0.08504478811667614, 0.09854153622621946, -0.08757175855205203, 0.13437819370271673, 0.08338167664849142, 0.013782398247439896, 0.07906864502443113, -0.034371761762275604, -0.04833784885032353, -0.13840823241929529, -0.02430099723690224, 0.10825193723338836, 0.11083386504455886, 0.0776308140221914, -0.05728290620590117, 0.017926050018821187, -0.04717624110958021, -0.16376518335018359, 0.035533977677495304, -0.0576679363209731, -0.029102868454536586, -0.00699709532776595, -0.01344381030442291, 0.012089938764237973, -0.14566053275997526, 0.09690717518171231, 0.013977980359201512, -0.12254170061448103, -0.04293520769023038, 0.030459598070118332, 0.030683792668537112, 0.018353422056857194, 0.059883918385373866, -0.026481402020018, 0.06452844273840586, -0.003966255514298487, 0.04089280070505699, -0.095687098801258, 0.054458183969445406, 0.15189680720756252, -0.08842563424783928, -0.0754504603683462, 0.07596931422190326, 0.07757632990323568, -0.007888318711928704, -0.03818443631211716, 0.017859846483343177, 0.11368304146047997, -0.013814032697166634, -0.09825037603759583, -0.05077307885127354, 0.10176239304423448, -0.12306628144566299, -0.09349964036098372, -0.11545658842766719, 0.06749164047911087, -0.11453063012101862, -0.011573296345783015, 0.0075216401409406875, -0.020110297692595747, 0.16447237484146318, 0.11347941241890054, -0.02159008014882457, -0.024036201554156634, 0.004335489441776048, -0.05261182886868766, -0.02823499424537976, 0.025836098849566873, 0.02253239235164203, 0.06517191249997252, 0.11679625092170517, -0.07920347238028815, -0.019922462006850578, 0.031261407703259914, -0.03878002977074608, 0.04959856659141294, -0.035059524268351655, -0.15823815973605335, -0.023012086430963193, -0.08103519784408833, -0.059779744448420234, 0.16386068210158738, 0.025067522512566193, 0.02109255045660188, -0.020304278283068054, -0.09656707677713956, -0.02916714023699249, 0.010592107125158986, -0.10644340827454232, -0.024781294660819905, 0.0877293359591105, -0.13185657111843294, -0.01860765367765373, 0.04744788113196109, 0.0373059846013868, 0.06706792311410097, 0.009123411506520556, -0.004584091121915134, 0.07574931745454479, -0.06433276899138266, -0.06415652425043591, -0.056758158928043416, 0.034823537448283204, 0.06873000986138976, -0.02527259756779557, -0.0034804202701968472, -0.029300881286942214, -0.0024833426307326123, -0.09058079896350381, 0.015635843303569773, -0.007199510766301563, -0.0028246392651368364, -0.16489911381459363, 0.06834797979648995, 0.04614175008356072, -0.1095074221450618, -0.12212977201611377, -0.058354023658282225, 0.053460921330095085, -0.05750633083657338, 0.08226169094812354, -0.028913883006882996, 0.04475992954452304, -0.0053212488783548504, 0.1341824740827653, 0.027685682942444508, 0.12169473984126941, -0.08190932426926428, -0.03997223103048132, 0.013147920708979466, -0.06148226968802492, 0.005492057141090143, 0.04552156515356309, -0.0524409207964768, 0.12350437269653881, 0.03883059673317203, 0.062364562793601246, 0.08280792329318405, -0.15564733878644493, 0.08717169769578144, 0.019696649595561604, -0.047746299339744065, 0.012935703367263256, -0.119152109125589, 0.0012941817737974197, -0.001422461051432564, -0.04843753162226751, 0.010863157982356033, -0.08450445414341139, 0.03611853196121422, -0.0638680345602468, 0.0338848337556067, 0.12845658111948027, -0.13770932189722307, -0.056325142685122884, 0.09330746429100605, 0.03369741619850631, 0.06317031977508876, 0.03478758765102411, 0.10745054018203132, -0.10400579514195703, 0.06734779640616095, 0.038243928153928165, -0.09622282750758206, -0.011234833069356586, 0.06264855684797149, 0.026320197877768032, 0.08622303909198811, -0.08493010526032739, 0.12186270112538121, 0.0002776546229637605, -0.02227109745270982, 0.008655421788000956, 0.08015593067010449, -0.007544813813016261, 0.012958874294079683, -0.08134975654390565, 0.0080725427920987, 0.09939826579106624, -0.004914300447221601, -0.009095130312355608, 0.019306766051405817, 0.09628349754353256, -0.006258844797035771, -0.09386538870624608, -0.0058694903431464164, 0.08724724999294559, -0.08800394111508977, -0.11831305612284236, 0.03622132700133491, -0.03667799932338093, -0.058254840089722905, 0.20008702416881355, -0.008549419019816295, -0.012040364896037837, 0.00228876318530571, -0.0038278310085803117, 0.12578483402568566, 0.0440836518997943, 0.019851900640740872, 0.0246391025652699, 0.05582914489264418, -0.01802834099869204, 0.05878283249459389, -0.08606950404022701, -0.06079483264156015, -0.029398920371395736, -0.02427247184960865, 0.012162241654940193, -0.01980842340636295, -0.025379100176336752, -0.0644334278467657, -0.02283540839227034, -0.09825434644402203, 0.03886756614139347, -0.033621611992936895, -0.024648188188406535, 0.011266021620893898, -0.08212985098906236, 0.010353428462103218, 0.08297405332901461, -0.07812384123328352, 0.1129921707340277, 0.0032227444179164884, 0.03253810988700412, 0.03321256401291442, -0.07975201209724546, 0.12095887445604082, 0.08882105697984742, 0.03345254407035508, 0.02428077934448356, -0.0015158534997429286, -0.0063840208211499265, -0.04587986194381412, -0.12129837804460146, 0.08533058402930573, -0.06018505152269957, -0.08742400395437488, -0.010340040917740207, 0.1400812354224747, -0.03440951835961251, 0.13181718707258572, 0.038547548529388846, 0.06938733264133756, -0.029455854383234423, 0.10681551371834476, -0.009287353724597236, 0.10375419644078883, 0.02229396531941546, 0.2174135469178723, 0.08096410651295054, 0.07398707010553021, 0.014462533076209441, -0.030061337505265427, -0.13468347384832643, -0.006201898628479883, 0.023537302311522575, -0.15956240806531924, -0.09788739127375526, -0.0012613035096947017, -0.06983491688652922, 0.07486862323759096, -0.01159337537554539, -0.019463394903210567, 0.15652334821673847, 0.10124114381901374, 0.011172914295399997, 0.007928997015105056, 0.04294126049465001, -0.092558626207163, -0.0949054748688799, 0.01920579189950451, 0.03049511928277367, 0.0032042899040107173, 0.0666773229950154, 0.012128846740001085, 0.08104498950951454, -0.029071644636482947, -0.0751363635379826, 0.034278101327121634, -0.018840364167631737, 0.12322016852262867, 0.022545298186166428, -0.04398469762415664, 0.06838344371968745, 0.018474884643633102, -0.052932100006132445, -0.08014720376577564, 0.04648468117693921, 0.14658072880127254, -0.05768647272610029, 0.01962413059708274, -0.062134505130156205, 0.04632466374864799, 0.02605848839037443, 0.027684910724711534, 0.028979344190198156, 0.0634936124875693, -0.0071052060193809675, -0.11673475910509294, 0.11592086614829462, 0.01744629337998849, 0.18277609423618318, -0.04558954712783737, -0.11909435519127932, -0.11342737860570527, -0.036490578961524925, 0.01810908200054636, -0.06875300343595744, 0.055486406212682936, -0.04709144010661885, 0.048871757556730205, 0.07666501425186409, 0.02146945638919673, -0.08362432436917912, 0.09764042980217322, -0.0820336535357461, -0.050666875608951216, 0.058428419512931026, -0.046078335489038676, -0.008144375792869965, -0.0798504755873986, -0.01866988371811269, -0.01403050162733682, 0.07879322626992624, -0.054844917534789074, -0.25763368430520184, 0.016170340165011083, 0.06001167681566575, 0.03173264362457074, 0.04292471657003886, 0.03110142134255024, -0.01111505436832763, -0.08006911086736881, -0.12564432339297527, -0.043555195766744555, -0.03961704016703406, 0.10124750465565853, -0.007449415855101633, 0.017777749781646673, -0.11037500663455055, -0.06355129492782831, 0.02374489414289546, -0.009808845963833324, -0.08373658881578272, -0.07646368058160931, 0.032137860562515705, -0.021937289432623273, -0.0756795069982941, -0.0073230126577267275, -0.03558527845108123, 0.005611394549741895, -0.015047717670567604, -0.05212983331890139, 0.07191857286296496, 0.018835695069629593, -0.028964493328684067, 0.17215412761242982, 0.0878346852419232, -0.1068020137457324, 0.09774189849180297, -0.0008999618800053237, 0.11116373985589666, -0.08230520992940898, 0.019237423372568476, 0.08968585268972021, -0.026968115120148652, -0.02069418767269885, -0.05694638477476294, 0.023247687983199407, -0.11609379681574601, 0.02055168735526689, 0.006798210212964038, -0.030674413822672494, 0.011478186690481697, 0.04830213047861422, 0.0030188842156545176, -0.1212292314633911, 0.11239506130375085, 0.036392893586623336, -0.05385743973499246, -0.1269841124140892, -0.0807771931207169, -0.05807477207632032, -0.07700654776882912, -0.062351791333043305, 0.07551668148221645, 0.05779146382058392, 0.057098767373437255, 0.015413239955713151, -0.05185429937062284, -0.05607329172846349, -0.06594528485828607, -0.09730857011395465, -0.014562536731086177, 0.1155915860796708, -0.20347265601980516, -0.02472431034700538, 0.06308932410378681, -0.07872549629293826, 0.0444134451517717, 0.017172562445891344, 0.060524460708020054, -0.10587731240543541, 0.13552539232837896, 0.11112112493093032, 0.010680142713736405, -0.022586234585257006, 0.025921918452020992, -0.027243631285960152, -0.1382508298676765, -0.04207120955719651, 0.05457875377964554, 0.017368407640361166, -0.02524011517414411, -0.08754369876783068, 0.1052184231136879, 0.021199688997424224, -0.0691219352453773, -0.06811976950435557, -0.07294489847160986, -0.04780766025799675, 0.08165704014069006, -0.008541252518342228, 0.013572759477408812, -0.060012971026057765, 0.07247259021036642, -0.010454426311478728, -0.051746195002350164, -0.08255459087299208, 0.05882913113661393, -0.008667059615920056, -0.05943621186415214, 0.19507864850818202, 0.08349727088991117, 0.10475406532823058, -0.2124120366734479, -0.010764043851797014, -0.011931818275225593, -0.013055005479934597, -0.07211532082614576, 0.12558932535513326, 0.07220880681270647, 0.08670020920415664, 0.1604146641277386, -0.023608724935795508, 0.10163860165056848, 0.012913368177162932, -0.037904450486854924, -0.05085001582609177, -0.11029770265018114, -0.035394600552124275, -0.057554293196356396, -0.1072727874117951, 0.0020314644910258985, 0.07086510264687078, 0.2007627225818675, 0.08167120320493339, 0.024766409898612736, -0.04071961752435167, 0.010179703414457364, -0.07139615044043018, -0.011197300215437684, -0.03416197261878828, -0.11732603145518573, 0.012674231861869993, -0.022943839584480322, 0.0368183183924148, -0.04403319175620474, 0.0037753713850734176, 0.0325346963419962, 0.08841230328291462, 0.10709676352924656, -0.021474475124466028, 0.04689983980737918, -0.19936058926000305, -0.03550112390878375, -0.05079271132897146, 0.052218488918933825, 0.06480411506360825, -0.03691811538855869, -0.0239703059449557, 0.03310780844938887, -0.14745664272528014, -0.1355056705136265, -0.005523399218088228, 0.0890402092306807, 0.07442409389087826, -0.12510460805141424, -0.01463416219111487, 9.30407846897037e-5, -0.03096622864116708, 0.13297080630157712, 0.08418279010726745, -0.174353325817209, -0.08859714961902425, 0.002370197151175296, -0.073914584735034, -0.026254670755473906, 0.07028732050919761, -0.01442707841060148, -0.0663894585030122, 0.009393987162626455, 0.052194616127163244, -0.027584428961753237, 0.05545472838886917, -0.046512543726384706, -0.04601917192434914, 0.029712001182638323, -0.0724674530336838, -0.1260968893724919, -0.04062968767281208, -0.08691587612167363, 0.09511790808512542, 0.07975302546177584, -0.02892879312348182, -0.008449797353034376, -0.014382969752729862, -0.09611781517012899, -0.0822935285915567, 0.05203326056047588, -0.060639669515169796, 0.060985576208072705, 0.12437492672352454, 0.012383181802001658, -0.059016824141399055, -0.04330490212385979, 0.03393044552752214, 0.06747192727560633, 0.06957321192843796, -0.009224256703830792, -0.0131260801338754, -0.10090834179025213, 0.18163763794083723, -0.03425213412895095, -0.1516733946020985, 0.10647871346300626, 0.06027188644766467, -0.009050333544166237, -0.09084852529364497, -0.27952083370381875, -0.1601691091714731, -0.05085197166033732, 0.051615731794404936, -0.08610620039982447, -0.06902404563998304, 0.018989163545482506, 0.025973724365787487, -0.09952640168104789, -0.015958186671848777, -0.01757681255221826, -0.04906258589624922, 0.19825385207720306, -0.048410705117836156, -0.0009014856530350233, 0.0071312535764187545, 0.01383970305315892, -0.020849530249763062, -0.18415021039121318, -0.09333016576283946, -0.06412873946599004, 0.021411794238620697, 0.03761083961507257, 0.11215451984017863, 0.15656426180617522, 0.024017628979010677, 0.00956963036340257, -0.04157163995356646, 0.052279513046446846, -0.08368254228823059, -0.13325512244177004, 0.05070092631888843, -0.0015191969180130007, -0.10185286102853178, 0.042567746617776646, -0.12993901263292623, -0.1157842940282548, -0.11583182055858743, 0.08337353587391491, 0.07776988425467593, 0.03004073862846979, -0.1490314936482281, -0.015739827223096435, 0.026573736620624244, -0.014421399829045556, 0.03572083659236517, 0.06077757138608599, 0.07137571738620921, 0.11687402797679551, -0.011447982394412465, 0.03634643611723344, 0.09675454958968539, -0.036595508453935464, 0.050005610738146605, 0.01301863049922249, -0.056297098896029456, -0.11330292675004916, -0.005444352685938036, -0.06512083825983495, 0.07352276145821099, -0.009906894814944391, 0.018306606705790908, 0.09876610463306962, -0.05703446495808636, 0.03488283473530766, 0.0680277206236739, -0.019160092792284672, -0.009921545851054205, -0.11192005780165001, 0.04682933843483125, 0.06674675853505757, 0.0789312822722987, -0.051341343947246085, -0.07406954461342743, 0.15204510913351463, 0.022185427906328225, 0.022217959945249624, -0.035746153981091024, 0.010272060960737489, -0.04106620602993345, 0.005938831108927313, -0.10694138901843155, -0.019443014105902722, -0.010242124689013884, -0.008531245806181226, -0.09633556139459432, 0.005677982373033153, 0.05578170475922843, 0.10771363527900389, -0.0041157319213737005, 0.0847390576897772, 0.06445968205283042, 0.0770914071111223, -0.11771427198046812, -0.0188855974050235, 0.14782390388277797, 0.03751614099506354, 0.03695525368391378, -0.018301144904718115, 0.08966772570862991, 0.035985675494278536, 0.04195504204577126, -0.03672793988222823, 0.06763954153714181, -0.08516277684509163, 0.09359107913327172, -0.04861442698459858, -0.09190466603996367, 0.0360885223717076, -0.06286420154371847, -0.04146983435512797, -0.022412071443492002, -0.06541657416036294, 0.05242636381276816, 0.011506477004671858, -0.07046439627875332, -0.042506305480434756, 0.14325672843543621, 0.03530208604095453, 0.00029615202758910276, 0.014721989547881453, -0.04129912264704181, -0.07682488544257143, 0.04039434960421385, -0.09532395948887316, -0.007540833646669204, -0.06630096509447912, -0.10219490352619026, 0.12370084297918951, 0.024491243830626477, 0.04828526249000837, 0.03707515513357394, 0.04471891107031994, -0.04866132783299336, -0.010348926096231262, 0.008729923750838376, 0.027904795493947917, -0.030878582065587145, 0.010195206701696765, -0.03983834434370447, -0.07029640471476326, -0.01820181825255604, 0.03354516685115422, -0.03969485758012185, -0.06223340646193718, 0.11874354851286505, 0.06357784796411863, -0.03730826726148158, -0.044469588093661375, 0.04749331250328898, -0.06685831114586098, 0.03891683584003262, 0.1010131146985244, -0.07188027449138705, -0.004989436738026254, -0.04473422070205096, -0.01975184166456527, -0.08280915739556348, 0.12127825349804662, -0.09364137241280979, 0.013352543462908014, 0.09870134091118112, 0.0004964312967213545, 0.03622574223129224, 0.07899028442098487, -0.1063966869282136, 0.022534982281436508, -0.015546232830902314, 0.011599680041582065, 0.030082297381696793, -0.01352142891999514, -0.022360889580783076, 0.07730911916561498, 0.08389270733927212, 0.012734294422739184, -0.02973167382431356, 0.050392092904718015, -0.08559456992077655, 0.051356846872443254, -0.18875139356933973, 0.07031818137123276, -0.03669052131627889, 0.001853210085308492, 0.047001566685525985, -0.05750069505701795, -0.013950887689040856, -0.051445822416581036, -0.07410382638628397, 0.03222016802573176, 0.06699244096948642, -0.03701956821526912, 0.0722913377827546, -0.038138800487415636, 0.004357846271670927, 0.029589991198461812, 0.05144195590976969, 0.040755708097907295, -0.031115326771178422, -0.022276993865606307, -0.060676857256035104, -0.04172973085598613, 0.07430806228557842, -0.06886158173911071, -0.08774895939278618, -0.08519416367257666, 0.016955243015326013, -0.053912218541561176, -0.06184292667774625, 0.1367453573181624, 0.05333505697116646, -0.05636763416510206, 0.006675326181010202, -0.058200235285432306, 0.052964751933308706, 0.0074168525174570605, -0.036083088936868964, 0.022252497229907405, -0.04228149015590271, 0.005768549640850466, 0.03977936744706251, -0.051780999579650294, 0.031731256081317064, -0.047156475030710715, 0.04671308099625519, -0.11230391144854586, 0.05768715278987101, -0.00035696941927612397, -0.026421393615836467, -0.00575929886988203, -0.1928282474455797, -0.034066529758610756, 0.009838005566922853, -0.02653010966854425, 0.06524099033470677, 0.07968904731820572, 0.05405915293237573, -0.016189569691664213, 0.07373567258303011, 0.036885289124925905, 0.030868014319466584, -0.16959983008856977, -0.013617421750907455, -0.010013016637369759, -0.006389647940578685, 0.04561666238603235, -0.01394697872530223, -0.12625286171953148, 0.032765840951041014, 0.049199177723293346, -0.04628147045667367, 0.05380606671987069, -0.17428494104013764, 0.06430965748755146, 0.06610189286129192, 0.03630159690368379, -0.09034692562260974, -0.025600969542813233, -0.019217740008425473, -0.1497018637592045, 0.06066991488760958, -0.02172753426508732, -0.02246425176893306, -0.021461132890648788, 0.0414816496239683, -0.0871658990718262, -0.13490000060799634, 0.09706294727620053, 0.0034876704634863582, -0.01927638531712796, -0.19140685893480422, -0.04375391132692816, -0.05866396189159469, 0.030107999702777496, 0.0005702883079655983, 0.04703779472693338, -0.1029054550549961, 0.022640546670276006, 0.1572520940633784, 0.05410508693221009, 0.00663694640927632, -0.044625868948415685, -0.008472047992894957, -0.0040507836256108215, 0.03405520665400375, 0.007147981161031323, -0.022952982670042758, 0.038268751283894474, 0.00642174881621312, -0.025870670877906995, 0.02219956665127291, 0.06934506973931549, 0.07881601927108613, -0.04671496138501915, -0.010011742503831184, 0.04856980712504448, -0.07270413533579453, -0.07346422960430513, -0.1020626395920312, -0.021982303360379814, 0.12574703575628568, -0.07905991190244185, -0.044425543421475824, 0.040406457153767276, 0.013355159961299441, -0.030842984840920403, -0.059837958088421316, -0.004723553751491193, -0.018605372032717407, 0.039756417119628504, 0.006981761479058903, 0.046615796117201215, 0.1275151395653593, 0.02244753965743022, 0.011462004886900062, -0.025262639927861198, -0.05093648117253532, -0.04942854008063604, -0.012679195605456915, -0.08419693675532644, 0.053628884607465836, -0.08021289379796279, 0.02864557886576216, 0.062181554605286134, 0.05863146649406436, -0.10875123876974074, -0.12474127104048595, 0.05160386150242563, 0.000996915117257132, -0.03963258394694748, 0.1467866716040919, 0.019527911896966316, -0.03714107589620235, 0.024150175517927742, 0.059916307785841776, -0.03702631461089888, 0.03611100768280589, 0.012454676504016458, 0.0037110721969498836, 0.0307152014615267, 0.06876611252896347, 0.0830362023355131, 0.0938508703750013, -0.01781830728921366, 0.0343297305109889, 0.18350896713562614, -0.01021678529949686, 0.10552520937448322, 0.0399524842548476, -0.014967042576846586, 0.025772753667478616, 0.05601611449160579, -0.11895641371792055, 0.012886916235596197, -0.06886435492400811, 0.03202014094704231, 0.025075122674916458, -0.050593495686675975, -0.023981494066910268, -0.07043669305505225, 0.11579883534931806, 0.007413903176368826, -0.09594265923523711, -0.007429425121096622, -0.13798806772636504, -0.08325447287620945, -0.15585877401007614, 0.03045964453257491, 0.061798071538097, 0.018581271953885196, 0.18424598451704552, -0.1834255948232837, -0.08465452285998373, 0.0033014987917418183, 0.07546340512744099, 0.06845476924681651, -0.06755631388125218, -0.07411282226557438, -0.061615478976489, -0.08279199046478224, 0.04381051051900354, -0.06330045836313253, -0.01463031930607864, 0.057789978953361604, -0.03985587746065317, 0.0031965596880482003, -0.06663771385205028, -0.10990451638443838, 0.009656909025079758, 0.016743997289577695, 0.07965791889077595, -0.04126906702394886, 0.09360185833664748, -0.01953738061295751, -0.14221533975178097, -0.10076988041977052, -0.11445448792061624, -0.02284209373203128, -0.15291193564841968, 0.014252462792546855, -0.04026778478302473, -0.07480066753728036, 0.030132747943912727, 0.08711321161818057, -0.039574620504336924, -0.06647373887536782, -0.16923076709315463, -0.09950020328034632, -0.16105868406893986, 0.022694056131080418, -0.04482383004833841, 0.05160070299474573, 0.04921200073520443, -0.005246332729709402, -0.13328096489330235, -0.05305666953672373, 0.08979780474636263, -0.031229394756365346, -0.1090496772665887, 0.11550996669242902, -0.022405978896285293, -0.07187611255827883, 0.07370122380962107, 0.0755121324448594, 0.10391757050421044, 0.10553171973484794, -0.13566043538084102, 0.1208068336558371, 0.012301257793751788, -0.03110206093780121, 0.0956355781262342, -0.03182633518563438, -0.057379137966164985, -0.11513978974781044, -0.0048084166917766605, -0.09297205934437805, -0.022848713534704482, 0.016467003567189616, 0.046056167984530154, -0.0688174490988433, -0.15275743107976147, 0.012255119475430948, -0.15306401715192744, 0.025894943402652835, -0.05452964033363616, -0.06537481297539027, -0.010365928542284587, -0.048336159118598544, -0.04659296257582066, -0.0426287970126757, 0.02205754168407978, -0.02103603706540568, -0.0007371023571372758, 0.013878978494603227, -0.051343597037643275, 0.04695619667598372, -0.07828658062489056, -0.06305648559887699, -0.10305765523565878, 0.1337570383410879, -0.14808752329649882, 0.00814135237184468, 0.004118085511478192, 0.10300438757564562, -0.020027086074743933, 0.15397339296181606, -0.10575790510517666, -0.023768052977637705, 0.07621056555874782, -0.02503772605294433, 0.08027973035064023, -0.0007456559093948995, -0.01292145406982648, 0.012198352626759413, 0.08666928959693851, -0.033954964371922, 0.03165287575968921, -0.08106795370256331, -0.015366253291135761, 0.028843730138140573, -0.015137185611678698, 0.11035827836771547, 0.007347555477223497, -0.0023299392090232146, 0.034452033139674286, -0.006265698178542339, -0.032998017673727606, 0.035427482183555715, 0.033447448925870905, 0.0029806926567503287, 0.06278033954667152, 0.003787345430166879, -0.08202965402647913, 0.08678350041811612, 0.0976131273420362, 0.01366571105628689, 0.10927566084009785, 0.11916142385360283, 0.11363589683753834, 0.01426615558735093, 0.06724363945614176, 0.046840132796584444, -0.056102582801028644, -0.021166581708871073, 0.06846032386622303, 0.01643232877196521, 0.05573640880949047, 0.08207347130618564, 0.022828895440495954, -0.11957159529809984, 0.12389278723747071, 0.04184790704658607, -0.0632173323192347, 0.025691937584958472, -0.08720777209201738, 0.07023912154462551, -0.035382524321331564, -0.05877652880431818, 0.0320058325340123, -0.003928699734781366, 0.11989126859878972, -0.02994062505896525, 0.02663314174681581, 0.0010916683615476556, 0.10226723690722586, -0.04848633400718841, 0.04001345336197237, -0.03782694362167106, -0.06522297516935324, -0.029908976222390624, -0.018829419470297096, 0.02838861670084046, 0.1378001667019901, 0.03635119583802964, -0.0005982443710694629, 0.12755312357044238, -0.10972437663188428, 0.08867529359086167, 0.1509150443844534, -0.03813181010839707, 0.08781421109247288, -0.08129489332035926, -0.11193148633327156, -0.10374318349555053, 0.019041029330998466, 0.021992690364437295, 0.013545103653899961, -0.08219347866957713, 0.17320352189112048, 0.1541647526864281, -0.046771311435416636, 0.10613865592167923, -0.03723251335537556, -0.050537628396867755, -0.12260350287723024, 0.010612908630252913, -0.02630754680711285, 0.0836247214956608, 0.009320634806060987, 0.042302514269980496, -0.08125544587810216, 0.07896699416862343, -0.07363992545841964, 0.007163871052508332, 0.08935851447411156, 0.05490225278329184, -0.11158703117749133, -0.07269661781910516, -0.04955733914214942, -0.07470552584639076, -0.0491459676775532, -0.0536234142905093, 0.033354332866831427, -0.07589754667779774, -0.0379666543523851, 0.07245905527789359, -0.019017734275763053, 0.044191777824758464, 0.08339696519522802, 0.09049249037316877, -0.03040200342592087, -0.11159356197144525, 0.049364780742879165, -0.10085136679766345, -0.012666626937091452, 0.025953122605231534, -0.06414993319613205, 0.046828566804939645, 0.2009551188230983, -0.03941611728210708, -0.018097967342328377, -0.01525256957287573, 0.018664562314894752, 0.03020848020676605, 0.072806158624825, -0.0316425024770516, -0.03211741817727081, -0.08179023537439725, -0.16940110058531652, 0.06511037312029305, -0.017170422489773053, 0.03440963814088052, -0.029408975078215237, 0.07357176165400382, 0.0037596984454291832, -0.0232996432187286, 0.08299931995792108, -0.028204931403459466, -0.05128631623061288, -0.04878276191107512, 0.11993366826580322, -0.08406147791817226, 0.05250148315302963, 0.08460715776170168, -0.02920218823624815, -0.0433000222050897, 0.08276550717799962, -0.07364014706236476, 0.051366702816885684, 0.11806352944671868, 0.0844326003101078, -0.023924991850507192, 0.08824434217180861, -0.153330001620417, -0.08406888912477749, -0.07475503657588295, -0.04012503531462315, 0.026551974875308915, -0.024838515570755386, 0.012764986132973666, -0.021520140286286012, 0.07696357050042663, -0.023972045475814992, -0.13937589931320254, 0.04458876591847994, -0.10608560407905158, -0.0530428847068281, 0.08390495426617962, 0.03368033135327866, 0.05885158938737076, -0.023637605318179166, -0.05062411063879958, -0.05854669711992512, -0.12426849692058226, 0.011757271186129618, -0.012486008771914374, -0.03680289994888279, -0.0104276960746117, -0.14716306209972108, -0.03893836420892227, 0.017188320893175714, 0.02151282451350376, -0.021461982493439346, 0.04373918878493569, -0.06771006060350176, -0.24909519576190606, 0.034401861720630526, 0.14285360537069067, -0.012978070164415921, -0.02772481013787012, 0.11757789513560511, 0.07945866386758113, -0.01486125757480183, -0.16382461405946466, 0.02878923324900378, -0.046150236143200296, 0.029999555515442818, 0.029924617366633315, -0.004686241772866772, -0.11486131693439035, 0.030648292744328287, -0.19104433542875074, 0.06779522478835845, 0.05198800663004031, 0.004573724611982122, -0.03900543337957168, 0.022728405442868877, -0.04777322828121454, -0.12719413449001282, 0.1205392260658636, -0.028904306710538785, 0.03745562892027533, -0.08445691463586556, 0.00797013900639797, -0.13640870072870323, -0.0325917950213192, 0.00922892674268272, -0.15046933588125058, 0.08861727678971584, -0.03817678234551849, -0.09704880449424028, 0.011371508189150833, -0.028223268229537926, -0.14256717433719968, 0.016981259262554976, -0.011504360110168147, 0.05404000346710291, -0.05463506179904299, -0.25648292761935537, 0.07733356843393074, 0.011983106594758366, 0.024259123245034646, 0.0010227288365629505, -0.012130719019445986, 0.05145095033363473, 0.05389587861866154, -0.06738953502238876, 0.0028979856684149038, -0.025054459635822555, 0.19034510268629548, -0.07025825180647788, -0.03390762901102176, -0.018242636655340674, 0.11609423371127278, -0.1293959038747017, 0.02742652089531537, -0.028230167181543128, -0.07322723787351011, 0.02116017143167639, -0.09803335586270132, -0.009373509950943578, 0.02034197225201771, 0.04931719729160948, -0.02768852206234708, -0.028541898380082937, -0.026572946749527978, 0.17283819042544843, 0.12108743355605912, 0.035091917546910184, 0.0746733236201957, -0.03802651671872349, 0.06769578347571142, -0.03875247656654645, 0.016742500971488997, 0.04697483413682311, -0.05863075291630206, 0.043991720369332285, 0.07152245535895928, 0.16390896881935385, 0.0002492040656025018, 0.03228374974399391, -0.037434984089295124, 0.053224965827633995, -4.6484515407130594e-5, -0.04904180583049729, 0.03231622284546025, 0.030579277504984785, -0.048190725088387155, -0.016683631329419878, 0.09233193908805822, 0.12102869782204315, -0.030989688137620874, -0.01257524119823909, -0.11192771862255409, 0.0646548129971917, -0.09996447433648387, -0.0704258042112059, 0.02317734025587458, 0.053958484205621424, -0.04137624298420399, -0.0664557210041968, -0.008340093795101713, 0.027989087790462462, 0.04833374569841384, 0.014072129221839508, -0.03116952381183686, 0.017215045541157523, -0.003696535459078893, -0.030878505923379027, 0.00381800645412588, 0.04242894966309515, -0.05462536977011635, 0.003819483451612404, 0.04786349856977703, 0.07268904901508752, 0.06481095410726655, -0.09788364725336023, -0.04323269000173527, 0.08167968259640375, -0.01215399389227572, 0.045288329322534675, 0.032991924922811784, 0.10478349767079001, -0.047328992742185, -0.04155135035306505, 0.05211156948706052, 0.1339101538093947, -0.016498693763682824, 0.12099185504556914, -0.022486282761275043, 0.05448123901667883, 0.11060130489031635, -0.0025557855292946932, 0.09367020276423191, -0.09312588404324952, 0.05692718471618532, 0.02922619741938998, -0.1401797131789668, -0.031213987478086246, 0.03227974229299075, 0.016125327852240797, -0.07211670711659282, 0.036933072324412196, -0.037714746017726404, 0.0158178529987642, -0.05672038989412214, -0.06148651621865472, -0.0715759525254203, -0.004412733362287682, -0.10159770942082377, -0.039472669609504595, -0.1228811685167449, -0.014712316948816001, -0.07663730670819478, -0.14605807420210273, 0.04430137805808138, 0.02437578067591506, -0.0731328114169805, 0.05606665548217084, 0.10755059795382382, -0.8074338526492819, 0.06284290327448547, -0.09041795428542994, 0.13552096679126235, -0.01763914032554657, 0.05228168077280278, 0.011570559146714123, 0.1560401758446945, 0.024102454245928207, 0.07905957438712168, 0.03439397287699637, 0.14358465080293248, -0.05633397639943316, 0.12047516893194714, -0.03550642950488215, -0.10300076547150704, -0.04038000451949791, -0.08516437467198906, -0.14199210058308342, -0.04191132079810581, -0.0843711746896166, 0.0034849216834596684, -0.2062520954871824, 0.009489133298062967, 0.0022639687674184355, 0.22240899453602925, -0.0647348571031605, 0.012355336787461854, 0.08479411679484049, -0.06655064504418776, -0.10229048817742893, -0.13496509014147479, 0.010372987045171738, -0.07340952091883018, 0.041010293970353995, 0.02650419074317204, -0.14779029745887032, 0.006610126617030395, 0.11591248675232085, -0.07249938184540933, -0.08028791265515925, -0.0757575008567332, -0.12367873533699421, 0.06680449021251976, 0.12476851093246594, 0.018991962817192128, -0.05734348754354686, -0.002650436619984253, -0.06490417319301868, 0.05291559744223698, -0.04010239064191248, 0.07051067818759392, 0.12565110354776707, 0.02120519694962655, 0.1034612514340244, -0.08363574971502673, -0.02012040731443847, 0.005543134762417496, -0.07778160918878814, -0.02755668782618632, 0.037495340588350815, -0.08347073587130761, -0.10304687233414775, -0.04684149584138048, 0.04930848260710604, -0.06444728893480967, 0.08177376530515393, 0.025029363883810173, 0.01375027581102515, 0.0684722204826201, -0.054928812986005326, 0.017872546139618153, 0.07743978167448494, -0.015557079286170995, 0.01086738350022068, -0.036144423275473785, -0.09541809073508738, -0.13673909086862335, -0.044867576037464996, -0.02432597192828167, 0.09833804253377354, 0.084824331317584, -0.038795543444971596, 0.0553162129640329, -0.0366878782624976, 0.0006176761962051594, -0.07773896361951077, 0.007180209733625549, 0.09761249571427898, -0.012380232059893043, 0.07221220444095326, 0.06960611499817836, -0.06602474529108754, 0.010851152627421135, 0.10716450699694316, -0.07406838985232429, 0.03234014244064139, 0.148717784852325, -0.038178488196394036, -0.03549312728744197, 0.049011419816575544, 0.04652349135631532, 0.07847987664101443, -0.08032376734488755, 0.045087162232426266, 0.008437743563819693, 0.046683604762259974, -0.09016735747250534, 0.04286706209716025, -0.06752028856555216, -0.022814842228395017, 0.07118839860495589, 0.04855878171225266, -0.06195241718485639, -0.07158354942917465, 0.11041822021669997, 0.05775112414093102, 0.05380459847338442, 0.0326156352597558, -0.13642340731760724, -0.048228820988831446, 0.16410901224994967, -0.033963341306910734, -0.01489497690699798, 0.012575430709412182, 0.05140410254080849, 0.06861342104919803, 0.09045742064165109, 0.025163981675129874, 0.1891413550762581, 0.004040322265882002, -0.037260576933665, -0.00296037780848126, 0.03556209975477919, -0.04859035533485637, 0.03281599913477114, 0.1639296601082501, 0.08940566471102332, -0.14152872124962115, -0.047739647034879124, 0.07939147392350643, 0.0007376774153967609, 0.024294627630530245, 0.017866988870375626, 0.17310320275049415, -0.1250122442508717, 0.02406045025630197, 0.0828426893201539, -0.10700566511682934, 0.011015406323010976, -0.09513948222350743, 0.08105693354548556, -0.09780090581336719, -0.0515801961822417, 0.028788836025214395, 0.007430129164422803, -0.0028057757715841077, -0.006594394253472656, -0.07627388245204361, -0.029177326946990313, -0.011951188985268231, 0.08777011890439627, 0.039012153117007856, -0.036534302215128454, 0.15378305325974337, -0.04200356720600532, 0.16766759931980152, -0.03694826967816104, -0.08198712185510644, -0.07093694479313654, 0.07869605737303577, -0.014199070251526369, 0.02486254408153162, -0.04252323776379674, -0.02576939422351612, 0.016859047828019214, -0.0017033260633174954, -0.04781972164594149, 0.04888696566419736, -0.017886861045985053, 0.10937534137429088, 0.025788280146600295, 0.0076661527355792024, 0.10079727641804888, -0.06363181314055076, -0.004318422189111202, 0.13383071815775607, 0.1457234329987726, -0.07452055739682721, -0.2736119020308348, 0.06843654149756562, -0.01157003957108815, 0.048474749692078, 0.09868047166738515, -0.04242118595353493, -0.12859139232064778, -0.023437454838768055, -0.01198647481548145, -0.01134224299522492, 0.07450866677007464, 0.011431446856214469, 0.007658554241096647, 0.10231279358671201, 0.07138355148631596, -0.027777744473211838, 0.0895729217084769, 0.12496653206936939, 0.002688460829384358, -0.02449226713699477, -0.032532936521152515, -0.022569098972478763, -0.040581598448095335, 0.01053317450592823, 0.15307070050696386, 0.11062216916739691, -0.04245274521239246, -0.011692579793842717, -0.07652078106561319, -0.037160649224590614, 0.0021848382906684625, 0.005536655558162162, 0.101638527577094, -0.0752455187120226, -0.004019316374055389, -0.060068549914553356, 0.09634230874144298, 0.04096298954371309, -0.16858731076244532, 0.07611655739217141, -0.04932028481085503, 0.01593437589979996, -0.0019289636497952787, -0.03365787587146528, -0.10898145556242433, -0.03075515660578916, 0.03922119228758285, 0.029788591987214622, -0.1274335584083568, -0.04528618691035778, 0.08058493608496237, -0.019052128909241017, 0.07324946857074345, 0.04958365239976516, -0.1157839670184621, 0.06099327741538876, -0.07096862857866267, -0.005248748233599715, -0.1317370563602426, -0.11098861513232318, -0.07068726140913048, -0.0068574832146091735, 0.007495656211277378, -0.1355679694587489, 0.0013220851992000909, 0.01122525843822805, 0.15468445698959776, -0.06644576194121452, -0.023456609249212213, 0.1086988001082097, 0.005913825163788137, -0.02678155325082045, 0.051793673280418236, -0.0009433312068168132, -0.044466226887477846, 0.06019251261460947, 0.15970112136265655, -0.035275593503410536, 0.13709524376965407, 0.019466545395265857, -0.06318343073129681, 0.053378687471957946, -0.11392912952708961, 0.06178584469986663, -0.11140684070222903, -0.07345386420361802, -0.0527580595975425, 0.08082654099501678, -0.05872555616658942, 0.0805817561931931, 0.030163234678368754, 0.026211028063821817, -0.08121840202606119, -0.08775542759156012, -0.008080596809148662, -0.004137154554151145, 0.10325398399536526, -0.04442238096249505, 0.0874227003979039, -0.10512082414145235, -0.013923932756372354, -0.06536264164565357, 0.016907648783531137, -0.023638652314685597, 0.06647276988891852, -0.01446409641771498, 0.05600004940071324, 0.04070756966852384, -0.04403843101618565, -0.1254590548859633, -0.09704622219081464, 0.14979524155539545, -0.12016864712508543, 0.042364616626109866, -0.02801642968408351, -0.013718398502375792, 0.05605350961395396, -0.010842428195148503, 0.15919472381966485, -0.037481150114579816, 0.09042882028725133, -0.07666172497766627, -0.05504692052873514, 0.04651661513665353, 0.05288461346014086, 0.055582427223261885, 0.032968966453907025, 0.08950774460904859, -0.059308011619633186, -0.14666645802170197, -0.04026077552688389, 0.0269561497770004, 0.02090791100982046, -0.1460590776772047, -0.012159385836996826, -0.0325616685488603, 0.039995624398357986, -0.12727981707965697, 0.04622796931007648, -0.0682959869313577, 0.010908506798066642, -0.03192979258778952, 0.1578866429964789, -0.009770464046892373, 0.14582922482210717, 0.01847758802016849, -0.0060182705325491455, 0.04248102759945468, -0.11792237139328347, -0.06185799318393867, 0.05647481493585713, 0.10001650085733702, -0.15964363417376035, 0.03768621783516091, 0.04021921756935752, 0.00979591520437595, 0.06946105997653533, 0.09570549175935925, 0.09901628176248434, -0.1603866804997706, 0.08044020288018888, 0.06131416863223207, -0.056308767800946596, -0.006424769120212366, -0.057272379049481334, -0.08893077791968684, 0.06491558264965964, 0.015968258378822747, -0.029904286815653686, -0.11190400306845637, -0.016487510073282287, 0.030330409275210315, -0.026718636151748038, -0.046654128058777486, -0.012579761652413687, -0.040589182709530235, 0.03840241448802205, 0.015431364225743668, 0.035617608498632804, -0.027008756647158752, 0.08367339761353305, 0.02165450798305617, -0.12979019180514623, -0.0026316316917478523, -0.02600329295569009, -0.014250752959021947, -0.07003489553154406, 0.08222572538139099, 0.008320372076092855, -0.07354945521326729, -0.05702777604970299, 0.1053220147571909, 0.05813690185764415, -0.043397132292125405, 0.028732279116867907, 0.0023803452064419996, -0.035482804862614695, -0.1607801755418591, -0.014435712751222723, 0.04270453744633707, 0.07663134758252317, 0.07569214754327123, 0.09847142126838841, -0.006759787174391028, 0.04760500152240954, -0.09801270450011168, 0.11137161901932949, 0.055379853164064706, -0.06738811551324375, -0.041007991719446364, 0.05985760983220261, -0.1285310536495655, -0.013503379859326032, 0.044037851015905655, -0.02902561016982152, 0.04311137500203268, 0.038554455527436145, 0.001953008742846476, -0.1113455793474936, 0.057228676545340626, -0.040478318383227, 0.05658319283088123, -0.008662879434200672, 0.08656904212748678, 0.04183165235289337, -0.052941673609988825, 0.07737340619124507, -0.07056601127372285, 0.08793187462163542, 0.06912915175574534, 0.11545221975524321, -0.049673720975350374, -0.0029126166039951584, 0.012648929298947124, -0.028718470130103577, -0.05097411887065906, -0.09232881958659372, 0.004517879691815405, 0.026835382677421384, -0.04377265848206172, -0.13190579146680145, 0.11924556621403778, 0.03182519378475451, -0.0155391413922501, -0.021047053592033314, -0.08212640984659408, -0.06537295789417408, 0.08462489344191874, 0.09909993576562436, 0.10014735481318843, 0.06572612073642328, -0.005428445457162352, 0.013651268239485167, -0.10548228447006572, -0.15109215847277496, 0.04448034323368719, -0.03722854274102112, 0.07953434699959301, -0.03480393447750525, -0.007024562233823972, 0.062140163216823606, -0.07496710134308823, 0.006713618801154712, -0.09000825678341867, 0.026652790892916868, -0.07723974198422946, -0.0026483400471773283, 0.03993298572398042, 0.08599427374340954, 0.08466585269380525, 0.018507117751187026, 0.027643343839539772, 0.1124302039639114, 0.003966292479298444, -0.014843454896470598, -0.10806072464156685, 0.10447189697476626, -0.07419924352010517, -0.016358563731169708, 0.011734408822838013, 0.1592825365018192, 0.02179008374776605, 0.02950135193350243, 0.0019261724018824619, -0.044692663909306124, -0.039001186913086204, -0.002373846684199293, 0.0045623997696200725, -0.02898017163793863, -0.12291981778530676, -0.07212630062696279, 0.20407162057427608, -0.07749991269055283, 0.1091731586046144, 0.03762099010200688, 0.05170127473965694, 0.05917314171989405, 0.015093261852492757, -0.01483901638787794, 0.03054363726220479, -0.022698806494548262, -0.011375283523201352, 0.07834573686794563, 0.012809273381323781, -0.06970099479416758, 0.10420356361322562, -0.009493054679423941, 0.0239441207060854, 0.02703022149802464, -0.0312623873340511, -0.06217844540302338, 0.008126762226415361, -0.04473433744065628, -0.06267629284140343, 0.12149293350767994, 0.05974120849104899, 0.020457563730163796, 0.07374222591186787, 0.08269324364792988, 0.07775296676732599, -0.06145293642913779, -0.11017084060452427, -0.05481312908211938, 0.028337485021059195, 0.08786881760079213, -0.09169128530474915, -0.09578981409497027, -0.08448892790070511, 0.010222144447850919, -0.08631300254245815, 0.036909729620406256, -0.037197427988966844, -0.0875165229976093, 0.1131154841440829, -0.09419902647251018, -0.03244930908545583, -0.04973739041906889, -0.013776164056704449, -0.037561140920786405, 0.1380210372873701, 0.07866221992717107, 0.05653289354193581, -0.03461384919354757, -0.16406193231489333, -0.09052844755432689, 0.06247361275114679, -0.008430569124486373, -0.06513044212380552, 0.09338187189437491, 0.05007432295415301, -0.04035718909912683, 0.10156317851742849, -0.01923515580349155, -0.011203287438471229, 0.026878316991091458, -0.166502800486958, -0.17892316740815783, 0.026575407081659622, -0.14120357265178385, 0.007675598180157089, 0.07719268239044398, 0.039122977182979114, 0.06848392937372592, -0.018684680700035736, -0.14143639295976473, 0.009694519342180205, 0.1429398148631032, 0.008879260893835521, 0.0682051153216061, 0.0018435686148928433, -0.00801687832324126, 0.016486343894261724, 0.04080226029473091, -0.0010097985914258383, -0.03135068807481581, 0.07428419930271629, -0.12667605038438498, 0.11049818552687768, -0.09799555219732509, -0.027383276766407203, -0.04287542308546807, -0.014872573532399822, 0.057980811356335316, -0.07021378231208973, 0.12092244782356237, -0.0271492060700984, -0.018163186765890214, 0.009618491824558715, -0.0317589675098074, -0.007729556860652004, 0.04010239263504045, 0.01438038415523914, -0.02919976160604187, 0.06688095365897943, 0.07034591788361043, -0.009667359729195017, -0.05570944836029676, 0.05462780624844264, -0.05761153765228969, 0.0174955678786811, -0.06169299444616704, 0.12436635025873309, 0.013739587392606858, -0.14010147384357155, 0.0754152961295647, -0.1277299741307021, 0.13033392633847596, -0.019460747987540786, 0.045216035195392835, -0.07543626594777618, 0.035009045241140145, -0.04929250846624165, 0.013250784138836952, -0.057084584698759606, -0.05116271630380421, 0.028298712248844154, -0.04366865561726373, 0.10283365017794231, 0.050766414129886916, -0.028551764593577757, 0.05488271796682423, -0.10566910095584034, 0.0806253688397516, -0.03837605225788332, -0.02770064183173071, 0.0176487663765898, -0.07065732623838468, 0.15218501448476388, -0.07152077917559414, 0.014855433667261674, 0.06554334277863304, -0.0009297439351752578, 0.021837845461741417, -0.1351892412328014, -0.05145716553218817, 0.026242023172409614, 0.007554638985768873, 0.014814364957878905, -0.01791457068719805, -0.023130897533065357, 0.024595150324037224, 0.1489914429380168, 0.03863782176512517, 0.08793296132869126, -0.01151187728707139, -0.01706513677147511, 0.08917025092992584, 0.09306672927192366, 0.06829708162247836, 0.0754438332705448, 0.009528802570067297, -0.005848115413402741, -0.13909470125903464, 0.027398075739391285, -0.1294147899582756, 0.07400674619286336, -0.1170431534683255, 0.08675545229853922, 0.08538113197113988, 0.14146794922750505, 0.038754662146703436, -0.04442352493648453, 0.10833255108611302, -0.07039989455891378, -0.09367540753511346, 0.08179092154888822, 0.12163870392993562, -0.11011462482313403, 0.06645596652569138, -0.0032299871841699307, 0.16837423841615257, -0.021873314779386546, 0.09479330911629417, 0.08380119889699235, 0.041022425417556234, -0.006003516977963778, -0.05443418642413775, 0.015297960444864308, -0.11804705161794393, -0.1653066252729022, -0.05177304680314945, 0.12109323398677696, -0.017179395920191912, 0.013977363805691903, -0.037637521466301005, 0.06855264864174111, -0.07558437382859588, 0.0083624023761197, 0.041748726085072047, 0.015867755745123457, 0.12729529566896977, 0.07066198446428308, 0.08759953833461319, 0.04234226050229008, -0.019577162541753027, 0.05628995775692315, 0.04669250696859028, -0.08441419270537179, 0.04777343650858623, 0.04853445215683414, 0.0709784437813735, -0.1167396426326134, 0.05570646292479572, -0.006317990424651726, 0.020848562252506534, -0.03995830231988087, 0.1306197315713916, 0.06146706110729879, 0.02309647564545268, 0.06362705061821523, -0.06036062862807652, -0.1382932263852976, 0.07340794025065563, 0.008536476578318065, 0.023591080756891425, 0.05879646716904238, 0.125678191250553, 0.14580827072573294, -0.054160799329900824, 0.17309415108370127, -0.03853590304064623, -0.109302962549662, 0.0338381657459709, 0.03882844292542471, 0.014790365802337816, -0.04355420301635001, -0.0041952721264016755, -0.010629060973933274, 0.0592644838512511, -0.0020871523441711227, 0.03532856698950317, 0.13406033496579392, -0.013831241950582206, -0.033112587212033695, -0.016250634093867525, -0.06543000537962902, 0.06689747807551076, 0.02967389927660229, -0.005362568531229431, 0.019997703702142225, -0.05985284945812498, 0.005709827473022617, 0.0674766110138146, -0.09945087025315487, 0.09131964811307457, 0.05388169183943624, 0.04797380540848429, -0.016916550220309686, -0.1409566494267297, 0.09938858829559652, -0.09556774428679818, -0.088031079110737, 0.06836680888719325, -0.08000685598214123, 0.023154492767974846, 0.054764077439882, 0.017629562597191676, -0.007972033299121929, 0.010906525682396457, -0.11394981541389784, -0.045056726398319966, 0.12630374814084366, 0.012154870964506398, -0.11105792821866359, -0.1752752009685362, -0.02812259629136452, 0.07167626287401613, -0.006183831435538749, 0.04135835296027124, 0.07633472698646603, -0.15001329837803207, 0.08139013607700189, -0.04686829892095773, -0.021162705469625634, -0.1558824334901753, 0.04295009889259229, -0.05803856939584769, 0.012277173845730028, 0.07600774811212403, -0.08344302241946164, -0.013015498686952441, 0.006048960141856796, -0.03944703241528333, 0.0049220131963056786, 0.03300417232213792, 0.05613445745457626, 0.05986401776640576, -0.041933578031563994, -0.012294411325609093, 0.025232640009743963, 0.038757605793512485, -0.007999408330790471, -0.006377531816162618, 0.04723129387531914, 0.1251695286080121, 0.08090397751256213, -0.04272335907406966, 0.07167232693360788, -0.0007280778311349429, -0.055009453918164135, 0.026307725248634867, -0.014740811278851221, -0.01699073486665714, -0.038356846390606304, 0.00820015534173125, -0.0327943573581859, 0.03346522801445551, -0.07836686499378295, -0.03334501178377395, -0.15345934250768867, 0.03856827143975854, -0.07509023489128808, -0.05068926293185955, -0.05767652118639301, -0.10985225081691145, -0.05664232766960444, 0.004017516577680933, 0.054633066828871917, 0.056523036803180195, -0.026462477782013077, -0.0790804374798933, -0.024219849426825833, -0.02713759231464, 0.06837507493346619, -0.03548011873406227, 0.029286765605077755, 0.05716883401674725, -0.1221058132734506, -0.06064671234049179, -0.10854820970120277, -0.08066035449692532, 0.0017516357839999148, 0.030720730825992158, 0.0744789281848633, -0.018741187538381224, -0.061468817265501054, 0.19109848412750127, -0.03971788230168986, -0.03329181904686163, -0.03999340656332858, -0.02607089441486897, 0.051352080100536046, -0.07341677386304121, 0.05438204450019568, -0.07101842109515626, -0.0458998869328881, 0.052355776492587665, 0.020303882337578537, -0.002299375365016906, 0.022204287289086626, 0.036039452989458876, 0.07094228276808041, -0.006737991163089721, 0.04804466382484276, 0.019929059800986908, 0.06334440263886221, -0.009762489480380646, -0.025620171495368754, 0.02249185085704395, 0.1246863220709808, -0.007051483933764383, 0.0226516529474123, -0.06506422818205294, -0.1334190943520851, 0.07580697071376517, 0.06925578042845977, -0.08818661535713386, 0.061658347378709365, -0.016347126125159022, -0.03841562308889242, -0.02820215581770207, 0.05158036149981441, 0.012730563540384071, -0.07733416865656155, -0.12450956877405425, 0.042702287267076214, -0.01485658732793269, -0.077723444773404, -0.12177906662944085, -0.08888737228983751, 0.038512545042797275, -0.05218529619380671, 0.11225478734360331, -0.07034130678140531, 0.050496706788022164, 0.050422357291963774, 0.030752182140887856, -0.17980869374110026, 0.05001838205886978, -0.03828478839091782, 0.1321801384008909, -0.05275907952821685, 0.00019081287161392774, 0.07128851714477133, 0.018968404335849065, -0.005371277914255978, -0.1314499562883616, -0.06725597038796363, 0.004946174549201303, -0.05257630866066847, 0.014222155832527797, 0.14457343443774223, 0.03751046137897198, -0.0075878630837496125, -0.003912877845530094, 0.10738727124124814, 0.0009200466229705442, 0.05372023195393819, 0.065540930143611, 0.007094593537764861, -0.019773004132086165, 0.025959471102771504, 0.146092149315818, 0.037096394606371715, -0.060531508901919204, 0.11251689766642667, 0.017761352354315266, -0.15401985887074796, 0.132758785129866, -0.00990256421409894, -0.09215333397729542, 0.08945561684562246, -0.01727678164590225, -0.010253045244345133, 0.10204174536325641, 0.006232947152166964, 0.03683976369077481, 0.017469354108897727, -0.10120103323166692, -0.06107932325863355, 0.12866240564457745, 0.0009975925055624895, 0.006474439321724325, -0.09042976483960988, 0.10688285667970589, 0.027405364534308446, 0.028210554899998772, 0.024117665305785636, -0.06737605185607928, -0.09470990958735098, 0.026074426219597553, -0.05930577294589458, 0.14441828205364135, -0.062163491406138145, -0.036194914992467776, 0.1394891539468912, 0.09386761975743818, 0.05898005637254671, -0.055295887801287684, -0.02283658263144687, 0.017631950147578963, -0.0679929899517676, 0.05659424082911855, 0.031140081276933212, -0.20034691199739954, -0.09649129463920486, 0.05292121228559575, 0.019032655620721692, -0.08979239127321025, -0.07386256537106992, 0.0365949878787086, -0.03464552021694605, 0.027097661379391357, 0.03191383557428494, 0.02346898351226243, -0.014145365564542444, 0.04224789825288516, 0.002884202823322937, -0.0377757585121551, 0.09810359502238691, -0.07693711235717987, 0.06952709551739844, -0.006676424167854755, 0.024289400512779977, -0.1405976385066415, -0.1488661539553916, -0.005860415234970367, 0.07038304388711204, 0.11224868347350533, 0.003960868992955079, -0.08450274795392103, 0.03328264861815732, 0.045331820192294286, -0.07947491233497639, -0.021558021477931472, -0.1499080149556525, -0.19871617267776137, -0.015595661613404805, -0.07846163201876148, -0.09442478262158949, -0.08133707407127674, -0.0018029796201230249, -0.0024850604585875064, 0.18476112391237348, 0.03360319761132704, -0.04595155771081494, 0.0023198420153285495, -0.09040339500031357, 0.004313846198652382, -0.016021329015772814, 0.10114618015321986, -0.1071555980078328, -0.13050797567113653, -0.04578339287407242, -0.010797889220357048, 0.023764618161725846, 0.1432960440533866, 0.11386719339523105, -0.1538198904629491, 0.08323196517709931, -0.10671570823691137, -0.060748445541018145, -0.014594502975954698, -0.016786103449316893, -0.009286869501755352, -0.04247269008014823, -0.0507700197593673, 0.11433643632604411, -0.06497533513273097, -0.022029036037196505, -0.06486502789647658, -0.012550221152445643, 0.049927003182002876, 0.026399498594483282, 0.0640036059027962, -0.12399982219937265, 0.0051725896090707825, 0.042164626895304355, 0.06204505420025328, -0.05314382383179575, -0.0005565444034311293, -0.009021079444152808, 0.02964746200813121, -0.04643142227206285, 0.033798107449410524, 0.07726252462113159, -0.032289261611757124, 0.0722131107719049, -0.13133693004887356, 0.17215444176102868, 0.0828420040954773, -0.04276416732523863, -0.027700027768946485, -0.019990510579592686, 0.015167117646555216, 0.0809321474958841, -0.03657396518018735, 0.08139250773137133, -0.023954956087501453, -0.10154226748662097, -0.07605306150049322, 0.04973016645620021, 0.3137125397740253, -0.016432685531778186, -0.10120877029626922, -0.019895720730725028, -0.01638664128578747, -0.005551711170865482, -0.1127091033172815, 0.10685616507524755, 0.09533276140501473, 0.011803620950731921, 0.08352831763465116, -0.07316121321079146, 0.011275048235069073, 0.0021245378184398807, 0.11983501830845, -0.035796801169225304, 0.13028620158471613, -0.08939297274293877, 0.014282072818096314, -0.012569767812292923, 0.03610230875229234, 0.05921201860708487, 0.05627281727404931, -0.026229314025850253, 0.0947396838176685, 0.045198892876498864, 0.05832571798409183, 0.04216849784867816, -0.03244638320289723, -0.06310414816582792, 0.05556959172824801, 0.045024504676987, 0.010256192138612103, 0.1038117550533593, 0.08152757575966822, -0.04280314215780002, 0.13099478082802618, 0.0340965852308194, 0.11638620703648193, -0.08455742798472914, -0.029281444269721096, -0.06876252124051964, 0.07748114730729386, -0.14809142896072552, -0.012091970974808198, 0.08785774084471751, 0.04714966043118677, 0.011540757964254179, 0.02398423508798982, 0.05309855825191512, -0.029842159806234492, 0.05136372624581162, -0.0612637446959817, 0.008537975614169269, -0.029513672934577653, 0.0069688084506659915, 0.014340663134432658, 0.11334595648126405, -0.010147761954627075, 0.04389203027926486, -0.11686199194563679, -0.08643109759385154, -0.03599429407804551, 0.16981341293823102, -0.02550650260413917, -0.03193547538163406, 0.07589241456981859, 0.09118387390392496, 0.06668709175016971, 0.06763280242105624, -0.0357418343472248, -0.02029580028879106, 0.1134534390952576, -0.003726266196141208, -0.10315010148863482, 0.04290203172268245, -0.14755375215961114, 0.009773808092348017, -0.08164109704029447, -0.023074605766813798, 0.11506071431466582, 0.10210284131244898, -0.0014258780261340337, 0.03794312328105768, 0.034973832480664865, -0.016948606625245072, -0.055233005599730545, -0.001505105534531426, -0.08942224720121827, 0.008751215963489066, -0.001763252286745298, 0.04340490650450611, -0.009499458860935827, 0.002757470156190282, -0.09354896087597968, -0.034208029009009745, 0.08788102486172962, -0.1576809308432016, -0.11607556476186208, -0.051075123114575766, -0.03185409582342356, 0.14360954272919815, -0.0036712907792050306, -0.13671119273042123, -0.20561399210490966, -0.0017527984759737065, 0.0059161121447671395, -0.06387946780436413, 0.10476028785354459, -0.01335571909776858, -0.018119441268904097, -0.017463993458665675, -0.01162425284138388, 0.0384302265718773, 0.02356917686317582, 0.1350039917786688, 0.03374747443901379, 0.015704383797670723, 0.14089592232855105, 0.03240557726623568, -0.0017558626408090726, 0.008675828401934867, 0.062142559521446636, -0.042514218096886665, -0.014089474573953003, -0.002999131976906837, 0.09015612773389353, -0.08453760742709804, 0.05162576290748793, 0.04608582324151012, -0.09227449290231537, 0.1207541646591998, -0.02461054248180379, -0.16478042694755884, 0.10354734236959913, -0.015012612514250028, 0.0196087888741921, -0.004535074065400789, -0.012021755389361745, 0.038578676351037476, 0.16900473394602203, -0.03838923808368978, 0.02668263940900935, -0.012788381260599243, 0.06968003507481288, 0.0478980819136634, -0.005300377945751259, -0.04851724989740804, 0.03128874243750894, -0.13310919076550332, -0.019391434765040003, -0.10778801642183444, 0.032523240239860876, -0.015516153705452874, 0.014686006284579573, 0.06376822994991639, 0.015510240347839429, 0.08071210600955991, -0.03808018724757819, -0.0519773631239146, -0.029910493715908534, 0.04662850182283509, -0.08729803775562488, -0.07695017216535799, 0.1418471121735495, 0.0434000273744659, -0.035935476213458904, 0.1261516226914669, 0.031336631517397406, -0.08383227271718702, -0.07788686742440612, 0.06316466933592066, -0.04962236251035579, 0.04322373656870308, -0.0027922934403033055, -0.023964458006915643, -0.11171588418724746, 0.00798592614667466, 0.06873489191053228, -0.1009317473395533, 0.015408493354883335, 0.050539489292221, -0.07122236182991444, -0.06240439250919277, -0.06361939680596813, 0.03585522740739057, 0.09018625058924315, -0.007061916138430712, 0.057925225615616874, 0.032575499309403254, -0.04293796798492096, -0.05868429839966805, 0.08412537386444624, 0.14003661964598937, -0.016757314022399553, -0.06988439446251932, -0.019844469122432297, 0.08922603469421247, 0.06772662974386458, 0.05059553191884615, -0.05481015609534181, -0.15048888728005136, -0.031797755563436024, 0.13015483447091614, 0.03267350127078998, -0.07021417046652051, 0.013300068994617937, -0.0021668545472521114, 0.0018614059106534717, 0.029856075501563332, -0.0358015246674199, -0.06504689481401939, 0.00039362688960159747, 0.0006238262717421509, 0.016769057814234715, 0.02417380299360421, 0.08691380245979269, -0.020771295638652928, -0.04308569812011061, -0.04155625794863392, -0.0715193939020684, -0.08457659354415763, -0.055702716874007505, 0.022064843676666744, -0.059774168252813766, 0.05361650524546136, 0.19250856396346577, -0.05260004169330986, 0.05783421913731906, -0.008660223265980775, 0.09325029630358389, 0.08480720210467904, 0.054383750392176154, 0.05599116948105916, -0.00878926866166786, -0.028142832309877675, 0.07361149281324897, 0.023901023851176206, 0.02681933775987738, 0.16225200562494158, -0.0035658054732384675, 0.09394329689285483, 0.08002874478072218, -0.12855677138779903, -0.05935160822952562, 0.1746573090593096, 0.01840504835151359, 0.08756600694821065, 0.005756776897916682, -0.06412482863163127, -0.0462106711146616, 0.059199908882430284, 0.10747324479557252, 0.06474147696745569, 0.0514490592491481, -0.03200828277343739, 0.08520793542495127, 0.07590839496375536, 0.09585428564194168, -0.0630787763959234, -0.025238605970917136, 0.11607386953627864, 0.008578738021303234, 0.050863597689409054, 0.1541645783151306, -0.05394001875994484, -0.016248232355809175, 0.029462036731172828, -0.13672176325092358, -0.011155393309199589, -0.036102375895264285, 0.010331753109114427, 0.02132679607892642, -0.09651076018172187, 0.05714155832486067, 0.1352937723227736, -0.07067943968851337, 0.03291428345590926, 0.034710711882482474, 0.010213229666344118, -0.009948451436369533, -0.10408891458322589, -0.16044956801958013, -0.07260272248724711, 0.10272863769742778, 0.04700540758547496, 0.029844815615145313, 0.022482903154168215, -0.0977654456861674, 0.014367994997035146, 0.12593990842396352, 0.12669758809338555, -0.07613535361468833, -0.0358471015906881, -0.13348266982508342, 0.05382275342562053, -0.004787140334994559, 0.009675648391190157, 0.06836928520671205, 0.09859300551643302, 0.052699331587013655, 0.03053855226917497, 0.05672603858151212, 0.054262441242873855, -0.13705178601702983, 0.05293193757704883, 0.12014706354691176, -0.005642627366695943, 0.1334006927152855, 0.04970141010189489, 0.029423648096299777, 0.012877868207703758, -0.05408665459586906, -0.06959401100038798, 0.014393043912379343, -0.021172231717602342, -0.015436470300167962, 0.10919960500689065, 0.03849141765910692, 0.02849407588575945, 0.08570491478698576, -0.044189801734332866, 0.14240226046224488, 0.02745400520911811, -0.03287953499892538, 0.045594952503826724, -0.1950783545215015, -0.0738282154950794, 0.030181422702334176, 0.07794368872868382, -0.15884525794164245, -0.057994851923042406, -0.1280269657319421, 0.007767535171839693, -0.02072746627346384, 0.03012135380119375, 0.020332746922213235, -0.012438584509626508, 0.013424579710048479, -0.12989993412634437, -0.004442294761181943, -0.015068911379822933, -0.07003575920046472, 0.02050023680319496, -0.010376448780667926, 0.08849773841502499, -0.004201235786123606, -0.05103977012206677, -0.005118867577103827, 0.06554038699898254, 0.02115722553440078, 0.0663505995295633, 0.060532577032129664, -0.040781962621252756, 0.022010410409429408, -0.09343390130299369, 0.08168587950728055, 0.01900454301217447, -0.022635522114653868, -0.0029985945103541503, 0.03656395064223053, -0.016948734975089676, -0.052034235265016764, -0.053718835277590554, -0.12638321363569685, 0.02320759509265024, -0.04331716219362931, -0.008571061423298781, 0.14034386983483652, 0.010528933505703154, -0.025436144133956128, -0.10410940892989112, -0.06254158982283538, 0.03639748671499553, 0.012989773964658155, -0.04162769922517798, -0.11500124680893231, 0.08141260218236131, 0.02808912041358046, -0.04023913428530506, 0.170557509263668, -0.05067813374469764, -0.053243225333168165, 0.047072204813939374, 0.05406082093646534, 0.12217371988965699, 0.009274939405504053, -0.057067709707528044, -0.01808870806731593, 0.07139774901266403, -0.057723004935698964, 0.0290931833215996, -0.12502825638742815, -0.015227069117640362, -0.06712538151676287, -0.1149365294979606, -0.03844363884651263, -0.0617686001949996, 0.06347318142236016, 0.053400524612617135, -0.10121136229734201, 0.08001366931511394, -0.05772771881344423, 0.061411165773743455, 0.057585153121227285, -0.05571398411718724, -0.11482038060349424, 0.06570029742169631, -0.15105352117242304, 0.15286013442412283, -0.05111822048123998, -0.01805418791924619, 0.022133383594832936, 0.02185999919678615, 0.020551689097254692, -0.020376054612997282, -0.05359173010564623, 0.179091517476246, 0.09965992245639846, -0.13864961106227885, -0.03813577035267711, 0.05247574061772462, 0.009446272167494447, -0.033683453087289844, -0.05347991406991412, 0.04201444284706064, 0.09760493271761086, 0.0337854277927719, -0.10558264202836543, 0.037822336178238825, -0.006570469470781475, 0.05340188856576696, 0.06116023602322573, -0.12017302507093768, -0.07657525171685478, -0.020676578197857294, 0.03340550709942009, 0.060266146705296134, -0.15568958439452749, -0.05428131674553728, 0.01610173412564288, -0.04017669235572678, 0.012737246300147418, 0.1684437761306771, 0.06532846766238035, 0.03170488090781455, 0.03729216587502152, 0.010486241991191936, 0.030599408234194693, -0.16326327473028696, -0.057595934022133546, -0.03023879280137464, -0.028169781660579894, -0.01733962241930739, -0.004064026183135619, -0.05381845854280607, 0.09192924745700455, 0.09101982675160172, 0.04421271644103773, -0.09136030595155085, -0.10597731840008155, -0.006500380532431356, 0.00625838789955861, -0.007081795827671431, 0.07770077152236912, -0.010704101760467332, 0.14232555849507356, -0.09970021447325317, -0.03980375919700815, -0.0524498081169623, 0.025687827465309822, -0.07215357753261946, 0.03646985405643217, 0.014266304488684666, -0.019814888907441354, 0.011511910503693566, 0.015331885396650268, 0.03949081770068014, -0.08380378440181759, -0.09725755219138195, 0.002225240064463803, 0.1150221867781528, -0.05809628268416754, 0.028837662990490235, 0.08184397495953899, -0.005038015841406547, 0.08526475397122507, -0.08787566671768103, 0.11537638009917765, -0.07195148151795852, -0.09218744966402408, 0.07643887042481659, 0.0849299513701543, 0.01858061830701706, -0.048646599623218635, -0.10588846858073747, -0.013240000553566477, 0.06923760101956797, -0.06363825049716379, 0.03967376159273019, -0.049761696231990706, 0.04146585977533881, 0.03255910949045883, 0.03943776211509255, 0.15535776298019102, 0.04628034641872113, 0.0825777967530395, 0.021121528657323484, 0.07458452528072654, -0.023340408082696273, -0.06265548126165446, -0.10253586055538645, -0.006632852102920589, -0.09050048689162364, -0.030341934660641525, 0.08260005248769843, -0.18807310396035384, 0.0176947849222187, -0.05959646628439245, 0.00226918141382235, -0.05818331836697543, 0.068059974138331, 0.1407572291194326, 0.047869957619683164, 0.07278689615359464, 0.05554001522584161, 0.10337093718860414, -0.009286145528310424, 0.1190000633883702, 0.029043627457364943, 0.014037930933472224, 0.09753068501285601, -0.014304114150762574, 0.08608060494576654, -0.05499963371427664, -0.06545827260355001, 0.02695329662870303, 0.15636643512441206, -0.0766912922185674, 0.1307756802717716, -0.1003728688049174, -0.11672722614323564, -0.02535949961758299, -0.047461351982153184, -0.13626583732248904, 0.018815281804758853, -0.02902706208050279, 0.038770026666308195, -0.01669925674966013, -0.06217140737223372, -0.05482869085681526, 0.12129113539569357, -0.19844844102544312, -0.1140229148998499, -0.07722823998478434, 0.032154753842671455, 0.002252208441649404, -0.09447531189884893, 0.07718336982051477, 0.13075937788385275, -0.00872664356820297, 0.14563672401816957, -0.03741337632212895, -0.05352774802922155, 0.08060139673521405, 0.015231188423182572, -0.13695209775777756, -0.06721695954206625, -0.02220602932156065, 0.011009854443240067, -0.03313451858926762, 0.05785116623060405, -0.06214742040031236, -0.02305327652554383, -0.17457073293790346, -0.06368970636458983, -0.015458613764118345, 0.05652993108865319, 0.1313461125858737, -0.010976996311763278, -0.05609177383956208, 0.19551313503841852, -0.03953077915785696, 0.17871511872421136, -0.05073539310962558, -0.06971185702144003, 0.04695386730318198, 0.07489812060524904, -0.009352843260275846, 0.05144570802388462, -0.05579610159527832, -0.09006409050621571, -0.026111317064625335, -0.02707071604946748, -0.034254336356942645, -0.042756285119569346, -0.021424177721927944, -0.12006889026298749, -0.10564668212133209, -0.044285943612973944, 0.033890814695433064, -0.101562692291185, -0.09441582696873994, 0.03580721689573457, 0.07268019311319586, -0.051916095065026195, 0.11574272655365508, -0.050713394794423314, 0.1015918863481204, 0.023337146892628374, -0.056961180291157, 0.09446841501535276, 0.011739284742350711, 0.03420875763922958, 0.04422054227119666, -0.02151813049996812, 0.08401659287518727, 0.053766699222430256, 0.09244510244519502, -0.07704953718105179, -0.02264050401604519, 0.018961752171143806, -0.003504862528667085, -0.028253451715421908, -0.009881927669678983, -0.0053000121435770355, 0.06198121746246179, -0.03703444347552445, -0.019461339572951195, -0.12065341215123312, -0.13566370010444223, -0.02898027751834169, -0.052177663711914714, -0.09168586817980383, -0.006245418440264874, -0.007884435447013227, -0.047034428234768104, 0.06039971530176411, 0.0028809508267142467, 0.12481844017476208, -0.001991696009645551, 0.015739500147925564, -0.016247809922478526, -0.0013977011027286818, -0.026381339984410654, 0.004842784056030846, -0.05136672756121428, -0.16941995207921223, -0.024649139265726736, 0.0872011207272107, 0.07545795684391966, 0.011870501633085217, 0.05762846385533665, -0.030097162515369896, -0.04533246755728349, 0.04804263892392126, 0.08981654150062832, -0.133718697844847, -0.07902026451423728, 0.032079811475071406, 0.06407657238395714, 0.16337117657419678, 0.00021614984374401078, -0.08953215997017103, 0.05310514072347866, -0.06343230426274443, 0.02617705545116974, -0.15841276104385504, -0.02462330801511507, 0.07404028920664871, 0.111189917949952, -0.08261428056289265, -0.033119719944974306, -0.0376679404757085, -0.04766180956840815, -0.029540844793457213, 0.032972809504340936, 0.029108444204098013, -0.05776167306260357, 0.12152346149637716, 0.04519896703061802, 0.028188513112387018, -0.011019397771627446, -0.10355846897993407, 0.007610202549163471, -0.08087310840908193, -0.024457384472334736, -0.10236593166421624, 0.10662643087695871, -0.058829139552778874, -0.04987733307342476, 0.03373961081348901, 0.028117716864888623, -0.07908536613386781, 0.0569716359319195, 0.12198643531662536, -0.05615236693475207, -0.0943301852469337, -0.06391088201330698, -0.15395966012285295, 0.012349910949618374, -0.07016327688136173, 0.11759598257941316, 0.11777111932946503, 0.031284696669850995, 0.007867292684539713, -0.010352716566076985, -0.17572898682656327, -0.04495259502309216, 0.1131560685076036, -0.07613000386222489, -0.057151953335048675, -0.05132106915952997, -0.14410994408689543, -0.2311030167276174, 0.06263254157601246, -0.05490338043845441, 0.0886369242101446, -0.00786399246635304, 0.09005360953953985, -0.05237373629108096, 0.01884874402892092, -0.016078392788193854, 0.01806695479781936, 0.06432397426925786, 0.05652391662718306, 0.055892698475909455, -0.03514331166841765, -0.002258971783101429, -0.0627694765762757, 0.013446271360003677, 0.0637471248239225, -0.06007687157162868, -0.10221955962188908, -0.1382826869326875, 0.020319719457801175, -0.03903680417153125, -0.07115703134822522, 0.08148172950505288, -0.017284893011011573, -0.03541957491325477, 0.06351818351634238, -0.05567887840845287, -0.1592917260890902, 0.1470913104103302, -0.09664319097889648, 0.036371156551257694, -0.05882150090788364, 0.007186803331489349, 0.13646142208514592, 0.03352322161926947, 0.00046955435538042627, -0.121158330976484, -0.02771617527441147, 0.0674598898625714, -0.07079337637228815, -0.09935339461494024, -0.05461107491747262, 0.04270391221542005, 0.17886193393182662, -0.07111726442705864, -0.04095743136845796, -0.16556218075444595, -0.0031143329241136797, -0.05700643394786111, 0.0051656128100981535, 0.030299179224540295, -0.021317610180963176, 0.06809875105989974, -0.08155344557905851, 0.00634197747474092, 0.11795004468911942, 0.045623179253653404, -0.0020148628565910976, 0.08335107488000883, 0.0553204049232231, -0.04020010830572288, 0.004218374000730788, 0.027058103564025972, 0.1324812492963192, -0.04733772693225011, -0.014829245902122326, 0.008004703889263434, -0.11141321810489249, 0.06968115267211285, 0.04342486728826371, 0.08048329700814486, -0.03640529986530674, 0.05619088159806059, 0.05528973467332748, -0.0017858497928731276, -0.005930301325642809, 0.03228771043072836, 0.024773657145255628, -0.061977882659971506, 0.020664484845711865, 0.03750460936847298, 0.09319459369204589, -0.010023288384984381, 0.0775863913160519, -0.06022863259011773, 0.01980244936582412, -0.08190474766465276, 0.04579907985890059, 0.07653221491027484, -0.11709627741758018, -0.00528893553888008, -0.015772251128239708, 0.01057783850241733, 0.09247423924351025, 0.07201179700596236, 0.03642046673666373, 0.009142217563785929, -0.07519127735672987, 0.018221360727700878, 0.08903341035100994, -0.05267058514454186, 0.08938324259737451, 0.03548317763954616, 0.08281000810587315, -0.009737614390935047, 0.18196809286092858, -0.03281027232054212, -0.19745211506312454, 0.18775660881113918, 0.035947053851928645, 0.09112261207562876, -0.0507122810993788, -0.010145646567795843, -0.004551999219281656, 0.12117012293725167, 0.1006704024595, -0.05322857173187071, 0.034050708167262835, 0.03501628356393359, -0.006180083049318759, 0.033421399427612804, -0.07007144800581996, -0.06956341697070433, -0.035678305609096544, 0.06668061172668217, -0.02268232651825191, -0.0467836749635472, 0.06571996136570424, 0.024008835220982835, -0.039710482350242306, -0.028175656154038522, -0.034321386187656205, -0.003707576701954948, -0.044479558700154964, 0.15079570245083096, 0.09483559817515715, 0.10087780521209633, 0.05522506537996283, 0.011804791684413738, -0.06525872253401714, -0.049970497491410595, -0.12054267569695865, -0.07882273205317376, -0.006366135438665186, 0.06117023896492711, -0.07067866621516972, 0.0873197246246549, -0.12156400216742358, 0.017019399019728248, -0.026717411717556785, 0.023578375277029995, -0.07176721467032544, 0.004614758352406869, 0.06897493136336591, 0.007060327920571478, 0.006366061443462581, -0.040187266585087915, 0.18279863200656532, 0.016682890039149363, 0.06773437623884708, 0.15559669465027579, -0.004252401174773379, -0.07283808966978363, 0.17704767815095657, 0.019947124590634666, 0.07101310661306914, 0.10038799543424248, 0.07629860364192192, 0.03264386381365389, 0.06590262740837363, 0.0020192713741437576, -0.0724618559154273, 0.09519685630004673, 0.002250052237411197, 0.03376491202806924, -0.06825157432161255, 0.08651759495954472, 0.012463294316338068, -0.13559828716132646, -0.0034008356604365874, 0.1455816326865118, 0.11250139344593672, -0.0832943747019592, 0.003848261313816289, -0.0023591976937065206, -0.09121499350819923, -0.10819111515772846, -0.09121815534375466, -0.11001265246157069, -0.06732441939431316, -0.0704336278318719, -0.09095872817927446, -0.03647353534016845, -0.0876898472176756, -0.035668490187110935, 0.014397515701599925, -0.02090021963992494, 0.03791741748249846, 0.09736699081329857, -0.07683294856507235, -0.04367466616771327, -0.04688917389924325, 0.024366725299869524, 0.0666245327469695, -0.07537133885324802, -0.039947871761335824, 0.09626685953852337, -0.006636672438695991, 0.05995992371590232, -0.0019211728900874628, -0.04823816523261591, 0.029403203915583907, 0.025362833687878557, 0.08481629657419, -0.019969792960269607, -0.014812576452692369, -0.0013265957931443745, -0.14151942872448667, -0.011236119648796455, 0.03284917540725086, -0.00967903093821261, -0.04704893521744854, -0.023616074253768948, 0.09429167227305416, -0.06122896976892391, -0.0078021205357654885, 0.021033364076764552, 0.005048559204720363, -0.04315409399945453, 0.0555517211036135, -0.023795566052159923, 0.11678047603106076, -0.0442459318776351, -0.04302735234606622, 0.0707863378410257, -0.019065853298150325, -0.18372902770278354, 0.033687742155802065, -0.027154499568218915, 0.0007387553128647449, -0.017984559379865574, 0.0240632252766582, -0.06966033904116181, -0.03474209926492521, 0.024029926467848843, 0.0533086573995816, -0.03723149261980338, 0.01867572585543509, 0.04172633896950327, 0.007434433518076395, 0.06751348108185248, -0.0646946971544863, -0.0031049885541026205, -0.1184127668184702, -0.04506938624368051, -0.03564883671272303, 0.18316241568810884, 0.14970488732531503, -0.14111576429212816, -0.08027695653423951, 0.11981715307237137, 0.0949030357584235, 0.04961342864617352, -0.05573169730087289, 0.021175168717689437, -0.09304207880210331, 0.04451771522624885, -0.0304161598211091, -0.05163172989459532, 0.057265096873518384, 0.06949633554783775, -0.10260017985410863, 0.013756255227730389, 0.07185696092906187, 0.02777372616675107, 0.07062656189739973, 0.027511528886023835, -0.10336262208971188, -0.03353624833784762, -0.06878627882948095, 0.05557080488900113, 0.027606210095696863, -0.06263417121793256, 0.1432352141262759, 0.03166054605051413, -0.014643846756996915, 0.0775642637927041, -0.010645976754291317, -0.048913749882295174, 0.05227276385332573, -0.09724681009773589, -0.005557677848838839, 0.04349580787409755, -0.024124813523604995, 0.026219177125726207, 0.014188951650345212, -0.04238209741973697, -0.1420235178474608, 0.11140250375428451, 0.06690630931682282, -0.003160022232784464, -0.11150822306177871, 0.03787011550823839, 0.06638915295583389, -0.04785775515134283, -0.01782663359595404, -0.06021573654540272, 0.055069227895928004, 0.03259035562395099, -0.07999788725089885, -0.0561205571902796, -0.0024131359275673305, 0.03225628361696268, -0.011678431551774147, -0.04549964807704878, 0.08214386884421318, 0.013667945491496164, -0.02723744041083133, 0.12030930981329521, -0.048089771836685356, -0.0600529283643316, 0.0651322598067541, -0.13926160057245238, -0.06386681299515502, 0.048791738324941075, 0.032933850333870134, -0.05199354236229783, -0.016117176227130985, 0.002918467196356674, 0.08522273181797184, 0.04807581118534684, 0.02229264900663164, 0.0805161307078354, -0.03055056144259179, -0.10060977936072556, 0.12328531808832843, 0.001699482043250104, 0.013198013729944594, -0.08364847172113578, 0.04790195938033076, -0.07567397433178065, -0.025017879741440725, 0.021738326094049045, 0.09417889538738008, 0.06339355136325034, -0.06148496723503126, 0.048530789500723814, -0.12037501413511283, 0.028084676035023647, -0.1600428540089036, -0.07874708256476058, -0.12259133425728129, 0.058785443832619265, -0.010003268970796363, -0.040497486174603366, 0.018497141888442408, 0.061098245252504874, 0.09097797088857831, -0.08025405727003983, -0.029935253860222128, -0.00897969678904995, -0.02285493121850008, 0.0811706127162001, -0.04055373512295828, -0.12130654259028222, 0.006825748181206863, 0.0033573292189087124, -0.034911868141742, 0.0667932218100931, -0.01996656736458077, 0.1341823606536765, -0.014963388105446916, -0.034659533189223386, 0.059876889582185414, -0.09225052396346665, 0.00366317549636381, -0.03525628551833915, -0.15396984631483446, 0.05821332342515185, -0.04978225329024724, -0.15289407911213682, -0.038817978877839825, -0.006386697431724823, 0.017347643485825952, -0.08148383287141257, -0.02280681666782561, -0.02555520183595045, 0.026637786237990606, 0.08317233644061299, -0.011417006061331874, -0.08751849461861634, -0.0162554770098179, 0.0031575245462071018, 0.026200706835848433, -0.004788692117056915, 0.08141418833417835, 0.015946576409683555, 0.02162698550724469, -0.006841655436402597, -0.0260781940268263, 0.03353312509772555, 0.004651374681330872, -0.19017159764190814, -0.16428063883921806, 0.047785466010580466, 0.04807110193875825, 0.009865316770676062, -0.03782048078235679, 0.01796020660179631, -0.07117314724372291, -0.027620313969445538, 0.04876403907446106, 0.01131243022653091, 0.11323224460416935, 0.003730527995287835, 0.009593871894290543, 0.023772963415444374, 0.14493754274049503, 0.014113665031691784, -0.04387917936442489, -0.10651173454209852, 0.0525629517347498, 0.09596971865641216, 0.05524047080071737, -0.06536269070746378, -0.012524109011820836, 0.04198245214500938, 0.08411230629355215, 0.0754299031548493, 0.1681121358713868, -0.004285588138877897, 0.00333119418810094, 0.03628576196945828, 0.09398789197510754, 0.006946792178105911, 0.12952205141644116, 0.047240956319388766, -0.06822753575303309, -0.05809080361427447, 0.0921259150433456, 0.035295243089462934, -0.050334247956723824, -0.007148767522153218, -0.067236906734025, 0.009323210100978212, -0.06349263821310167, -0.01686039602277913, -0.07126917862298578, 0.08776307938689253, -0.0653168812596703, 0.0545688882297585, -0.009975207977933652, 0.05122731983492067, 0.026143302894153442, 0.14172199096545632, 0.10783597216339101, 0.021018764527867793, -0.035547503516769906, 0.06998973027691563, -0.029947005242690988, 0.09824204720965957, 0.003418962752626727, 0.018237174056141885, 0.0494259252708731, -0.08938453107988459, -0.03188744948404373, 0.04300710913550492, 0.011438293380253909, -0.03380426427117999, -0.006470114509008856, -0.09742219093304111, 0.08634139375395303, 0.02970158086144227, 0.17812786873788902, 0.020992039205167255, 0.02932604215713935, 0.06879262825134957, 0.044115947179175725, -0.038790078832481804, 0.03487315559945035, -0.09772609558357247, 0.05067314849047536, -0.014854030891485225, -0.15160338424160544, 0.07857367417759582, -0.11969917063576249, 0.04975254994263643, -0.005423684202452425, 0.03793415632797743, 0.008805000114415928, -0.005135661355795764, 0.10448776241579012, 0.0900504026666982, -0.12191216146522676, -0.10622351349239818, -0.008937302319595428, -0.01245157153674722, 0.011334226060815647, -0.06141630907040316, 0.021003670375975083, -0.07832117441123329, -0.05848136916484101, -0.05697287770477284, 0.02593401835055317, 0.0882449538678067, -0.12101286007299492, 0.0372590167039101, 0.005930423691306824, 0.025534921584672264, -0.029794160973095524, 0.04278389895404373, -0.1425372878059792, 0.10765866694770804, 0.010587343107551633, 0.11909820684568204, 0.043151656645167985, 0.028789602142483547, -0.08694884484813015, -0.03819235173021245, 0.020771589781296448, -0.09664328947803111, 0.034080040583216814, -0.09033829927482684, -0.050877153213624494, 0.141866282979748, -0.06864818638295925, -0.029884075739677673, 0.038534991015811516, -0.12996505087211538, -0.015539385884840967, -0.018298246090941963, 0.04144059926577535, -0.12056916025659377, -0.09012215345055864, -0.017330170807101742, 0.052637965547024365, -0.07466583363299273, -0.04926497774274089, -0.031002101153702135, -0.02398408242396726, 0.03282626240941302, 0.036317531236277734, -0.03558415124348346, -0.0066488570289470065, -0.1321893913004682, -0.09069182907638497, 0.06512353117581779, 0.013121797488254372, -0.10675765066722759, -0.025146859581291994, 0.011567362667225876, 0.10611074992190081, 0.0452861565405459, 0.017940118196391016, -0.060822194245329704, -0.03752753973200096, -0.044027158428460465, 0.09259855816708028, -0.0314158383233715, 0.10996367938194113, 0.01625019670917978, -0.03804810002945231, -0.028953663079491213, -0.004713392568286735, 0.05941459904715859, 0.047554519583720714, 0.08821234595118121, -0.047872455056088274, -0.00148045399796346, 0.04622285646945752, -0.011008571717366612, -0.06982286374666034, -0.032938387611659466, 0.04927673235775003, 0.06588532443371199, -0.031043094800691794, 0.02175111321856626, -0.044616146253236066, -0.08669653973265688, -0.0011221480037167067, 0.05978537801015246, -0.009051597760678983, -0.004174793235310802, -0.029382595551346355, -0.002196685458344612, 0.051058056617895295, 0.10566393146104547, -0.039350468322405865, 0.027032543342528772, -0.009275821337994686, -0.080504696661759, -0.01977431976483951, 0.08253024043149053, 0.09095797110709344, -0.06513444699002399, -0.039889956334472856, 0.1360881592508814, 0.037301354249623386, 0.012787025636212784, -0.08878766291086333, -0.01315224159006394, -0.10248729624237135, -0.08574783857784331, 0.00575495736538221, -0.05745220847895236, 0.006534603305702027, -0.00686387794078445, 0.03957507424406198, -0.14298598955253708, 0.14894173591180954, -0.07009311706106461, -0.07135354999647638, -0.02884173921150292, -0.062228952562061485, -0.06095270001036863, 0.018518850161956715, 0.10246028199346903, -0.11094037495348719, 0.1569455418211416, 0.08008933972219408, 0.01968686472582331, -0.02090100264480809, 0.06218531215624711, 0.01630633045759699, 0.02664398990484533, -0.11512575639862863, -0.09173531936494225, -0.012078923199727807, -0.06904024448701937, -0.061010848243856906, -0.11239865439514661, 0.047871960415412265, 0.1256391352870848, -0.062489619593340165, -0.07145386629972973, 0.11674042438986029, 0.008151889493582185, -0.17880301098763313, -0.060747713588691916, 0.05232895490800288, 0.026318021882543313, -0.03696950196792085, 0.07189536135091842, -0.042499123034752125, -0.057250179190992075, -0.037577847401578955, -0.05753808754416023, -0.06026287408580259, 0.05732615676860608, 0.09583452996019637, 0.038367271808826355, -0.12150151405929108, -0.0035239444921063277, -0.00482886394554044, -0.12771667927668545, 0.05683115061826249, 0.028418111666673792, 0.11568112071935407, -0.030654506142937778, -0.031221856126399612, 0.08437673934189126, -0.03538696378697864, -0.001474056444218728, 0.03892175652307922, 0.03486826647923863, 0.05444610698782397, 0.01888155511879015, -0.13105765124093546, -0.029311506960921364, -0.15443803401784445, -0.021860971414523808, -0.009917402341691424, -0.05135119477757807, -0.0780556326666652, 0.0016914694188270823, -0.033213544919932246, 0.02860415714649762, -0.058235724142700056, 0.10521500757911013, 0.05400288996318986, 0.05230854335962707, 0.044611250209641705, 0.06319894157433747, 0.1084259963300601, 0.013957159971308279, 0.10326005240992187, -0.03736160929859892, 0.034989874203571976, 0.023614262691585376, -0.02081411794191406, 0.022926225208456906, -0.027740623541851857, -0.035045752240326786, 0.16082510191294333, 0.003799847450770953, -0.03988520739921362, 0.047424068431020004, -0.15892626059622905, 0.055773402471412825, 0.1403421265045153, 0.0717028872715591, -0.0114351113493216, 7.80680631335199e-5, -0.045888015358677196, -0.05100852698567686, -0.008224549594746248, 0.06016539686233605, 0.0268223603125752, 0.044692068440691464, 0.002595748809342432, -0.031511591107708785, 0.012806150313737812, 0.0031770474035397365, -0.10909821477162175, 0.060556796975740304, 0.0674295674223277, -0.026474768068727943, -0.03824401160411868, 0.014986982989772947, -0.09754736162291261, 0.11473207008279637, -0.09532027161511249, 0.005033005360092535, 0.009723374686012903, -0.07654623918302876, 0.00888954360309592, -0.05023845448795942, -0.01674279975588066, 0.006174653936393519, 0.05341240316491312, -0.030789739406386095, 0.02666800024070235, 0.017750404867916878, 0.09141627297190905, -0.07107804566140974, 0.01472583014951807, -0.053764774216571054, -0.06726520015742843, 0.11462069694085987, -0.0701074114625038, -0.031014076452478005, 0.021451891039484163, 0.014461146271736494, 0.003316522072141358, 0.09151696361808262, -0.08118825383572266, 0.07973720918749118, -0.11547712249259866, 0.05055410850225415, 0.1250998563474935, -0.05706947667916922, -0.13442841608760842, -0.10825856233148577, 0.013115779548661286, 0.006338581909131106, 0.019046144588211557, -0.03821008405261984, -0.011536057383357548, -0.04924430336427634, -0.05987600010960676, 0.029915678414002344, -0.016693662073486503, 0.07675082633249655, -0.061951026464374, -0.03761140567305394, -0.06570836381427937, -0.08820592497500014, 0.048768238446246975, 0.044663207444523095, -0.0464913784151869, 0.04012348029041308, 0.001541079719434093, -0.09335888954981483, -0.13953011681939145, -0.025412176192657853, 0.050252700095041715, -0.021145747693947847, -0.02693353792726746, -0.027855379164863185, 0.10029035554991851, -0.030872483478731998, -0.028451999256795552, -0.05148587937917222, -0.1184019719066839, 0.036193836790066956, -0.054744221035320194, -0.039994228055846695, 0.034930538910295975, 0.034514989614643395, -0.01272118016223474, -0.07954142270673353, -0.035219934371813734, 0.04661405322581987, 0.06286931665405955, 0.06259928368713415, 0.019119947770626047, 0.01501377527494069, -0.06365086956527226, 0.013832386531285209, 0.007451036273911423, -0.12303687763533386, 0.05764984059621278, 0.02457353939550865, -0.0911435351517522, -0.1183847557167992, 0.04324266089193386, 0.08597699827822185, 0.024364062288719665, -0.03625260918713487, -0.047433194561371246, -0.018515088661435013, -0.007330004991913655, 0.10248965717430109, -0.10359422305588369, 0.01489418981362323, 0.08011183698866918, 0.04889460203887271, 0.028979807378060374, 0.0050462357700069675, 0.060493273275431206, -0.0602643811893547, 0.00934109512157404, 0.059149446899068765, 0.06676247439983514, 0.016053115079641817, -0.0077728173295474445, -0.048427496099253474, 0.08824912173590799, 0.00241087473586034, -0.06310630345558545, 0.08452085286266443, -0.035959176550125135, -0.07639390060845776, -0.04435607983069743, 0.06209844941642866, -0.09058746204287019, -0.0281283839432305, 0.005597585316116319, -0.05880625269788625, -0.0623566107463189, 0.03476467700858238, 0.06671481595076459, -0.054226085433603716, 0.05484930106603816, -0.0410344908069557, -0.07283415313086324, -0.02856180633257282, -0.010152042766543419, 0.09574742185761766, -0.008605924565531104, 0.08754552957859968, 0.06323064258571143, -0.011151506207893227, -0.001972105635985798, 0.09390187163862546, 0.059639153591313776, -0.1287558047585091, 0.0664488374690291, -0.07290580324183742, -0.08375628134541652, -0.07701414311058026, 0.08712468971094553, 0.023878289702937403, -0.08352986171483028, 0.006009454267844722, 0.011051866745695577, -0.030695202555160624, 0.09009192749692747, -0.035673922627101666, 0.035675279277045835, 0.05391631523274483, 0.04076127319644784, 0.08335895957654771, 0.02054060729373029, 0.04603722983418354, 0.046082954110531885, 0.07587174596757629, -0.1283599426985912, 0.06747567177975869, 0.13353280600860573, 7.363848580765245e-5, 0.1252647997012218, 0.017048631469712666, 0.027399512239915217, -0.018649505774630513, 0.10429797223762882, 0.016543493884253444, -0.051039257605565706, -0.011948420900155245, -0.1292390335325466, 0.018614190396207677, -0.020525265303555924, 0.004438451927939712, 0.03498723064034892, -0.05970197145804454, -0.050526013182337425, -0.08868836618635084, -0.03641599062357404, 0.14792141409747472, 0.08511639748663995, 0.1160154039285562, 0.005244152241818956, 0.06717612765181034, -0.1011545785560368, -0.00935753746333521, 0.19495055386521654, 0.025617674782125947, -0.1309958017958063, 0.030241789538318284, 0.11826048380481467, -0.044016598275125586, 0.02817517760488386, 0.014030473557400756, 0.03731716790655982, 0.08009864293015682, -0.019753775813175004, -0.06093072567400314, 0.09654110445713328, -0.09352204094524626, 0.12833873033532797, -0.05978980832282088, -0.014449810771077256, -0.05796895358938853, 0.03486380350308271, -0.07094795418367959, -0.022240073831624886, -0.10560380308019876, -0.19414297260085792, 0.018742275038447286, -0.13623262443082912, -0.014712633709252087, -0.022269033727553157, -0.09563189536950832, 0.16857408839223276, 0.045027467030988436, -0.06539250478588275, 0.03219112111485272, -0.18456750562213795, 0.10999696893644967, -0.038676401038375136, -0.06092184576914553, -0.026266542749277158, 0.07180596148358641, -0.014902256475356297, 0.13154420132610636, 0.09705288839769796, -0.025479545875671427, 0.05324900637859912, -0.02591569429509298, 0.049208593703418645, 0.08681884729396093, -0.05418746906280399, -0.10301104642995833, -0.10449501906067851, -0.003823863326300129, 0.08831010431120401, -0.08596218072595486, -0.020844890716994836, 0.005575325901958843, -0.04037727708535028, 0.03878755452044659, 0.06662860631472087, -0.06988796284365287, -0.036449448910641113, 0.0482353089760969, 0.013424965656390194, -0.12837083956035764, -0.08146258525704018, -0.04149977380774395, 0.07420419435798922, 0.013959810014456703, 0.041321471311498825, 0.03681692067144247, -0.1094330972519251, -0.012468795613279772, 0.12190725399762763, 0.030402434717844602, 0.06818223616531767, -0.204240987550174, -0.05501050977238773, 0.005380235552324324, -0.05176202281800623, 0.07765320596677232, 0.09609123604455021, 0.15605996350813414, -0.06408421296820858, -0.01781086770738397, 0.02086145841977286, 0.0322667919682973, 0.02890571467183288, -0.029366302188336246, 0.1750510735441167, 0.044715037405421904, 0.035471377721942254, 0.016810774049061356, -0.06306912272990219, 0.05116164640032211, -0.1552193482935634, 0.01310995977454974, 0.007066473300042755, 0.007957322891429936, -0.02815423640178153, -0.010763529435939693, 0.016918921334278768, 0.08574625327215529, 0.04784958256419769, 0.02848399752450921, 0.0027975734275103394, 0.022405613670371072, 0.006770217976194888, -0.031105055003542482, 0.0379952808304689, 0.10899804210240109, -0.022431752970432282, 0.05087734839917669, 0.023742622924228424, -0.07831684090241388, 0.015274097757942777, -0.06789235614949181, 0.09103271408491205, -0.027641535227141623, -0.0830363313263566, -0.14794959403999386, 0.0911181617111869, -0.04164630841849544, -0.20154054984803715, 0.02616935129965758, 0.09222785864950611, 0.10655804148909791, -0.0292033437744928, 0.02776030495944966, -0.0024958237359584182, 0.11789998139378073, 0.18141746668930134, -0.03699159832355702, -0.0909860817511059, -0.07444813948638797, -0.23911585290871595, -0.05183928654024478, -0.013927600308073029, 0.0391467781787712, 0.04989580363341252, -0.022187255456086484, -0.04081782510113957, 0.052910820354644, -0.007128354337359804, -0.042775779639160204, -0.04188985439274445, 0.056333748662540144, 0.0026573614227887654, 0.06476490129310299, -0.04564112588745875, 0.02756605118350527, 0.015353425421580151, 0.09960872143284503, 0.04956598789461472, 0.047043742490647356, 0.044526937222265486, 0.008364596851531464, -0.021364956429225895, -0.05318416581511438, -0.07148280808656093, 0.21788669379240133, -0.0400839156436782, -0.006605960260470441, -0.09768985960578129, -0.0053909725888714915, -0.06117968723779046, 0.02142088974203079, 0.040532215214033745, -0.04894076095142545, 0.03196359269780829, 0.022432479769156168, -0.03499402324573763, 0.13321556979084645, 0.08201498175756175, 0.06396479642384488, 0.10810277166509402, -0.12124004366652066, -0.012472260535408949, 0.08806134083220486, 0.07124697779416055, 0.03175765889587299, 0.055309030039207445, -0.09109117963152526, -0.06516258734198628, 0.07694674999497488, 0.017925682727779186, -0.11752960504967687, 0.046621742550695365, 0.07319181060256245, 0.02150965937785498, -0.027907968329239922, -0.08111245943215553, 0.027774117748810372, 0.025241144420102665, -0.03846202884513535, 0.03082301548232659, -0.09974419053691769, -0.019081789497225755, 0.1119253723018963, 0.09668462223473827, -0.09392752667285593, 0.18609545674959196, 0.07229512220042544, -0.04126957242982265, -0.025662271200644893, -0.00219589134233024, -0.09018702196959523, 0.11560225776652724, -0.15380521771344038, -0.04041267541273173, 0.005995270307684609, 0.014988445656050918, 0.050060332965321135, 0.041053204057111985, -0.06629698773606098, 0.014639078353962193, -0.08996718257866301, -0.05016458166912568, -0.012818440556612408, -0.04713104319362447, -0.019474261636255864, 0.07148345942002964, -0.004493307498419695, -0.12874363426848476, -0.09461732656199869, 0.005749288901810329, -0.12262375389536598, 0.1257912609948157, 0.058830794231823004, 0.1399179033051081, 0.061217584805595374, 0.018844807626020758, -0.10877846342597935, -0.11461187936115436, 0.04386642674523125, 0.18204174059760753, 0.05829119302971061, -0.020513052532349172, 0.0604439805122221, 0.047078640113575085, -0.10434063423754653, 0.042640969331103624, -0.03658037879110477, -0.10096793357065356, -0.03287228232079622, 0.017137089638310694, -6.649654600837456e-5, 0.14949359302954626, -0.09058134101097343, 0.12278829375549266, 0.0018248130483346963, -0.054404146291152984, 0.11346870981776559, -0.12178632926061919, 0.010249041845407746, -0.05944468879567902, -0.01432740904753785, -0.0102190490417289, 0.04765937821330619, 0.02708582810044231, 0.05312495799791647, -0.11065493292677762, -0.019687091312678685, -0.02131584993047326, 0.1142355016598875, 0.01779281779278682, 0.13520322806554727, -0.036529934153986525, 0.05151682104191755, -0.042824810686052116, 0.12008165516295716, 0.0374820857881244, -0.058575997139442774, -0.04529321158618697, -0.13753945840972231, 0.055131041402781716, -0.06728816752272974, -0.0431712289529861, 0.012730510730330921, 0.07023441688361945, -0.12162505025321066, 0.0471115648244532, 0.07898826626082668, 0.0660027822943332, -0.0844846552856129, 0.040105365072041606, 0.03242576807931454, -0.09010396868314907, 0.03884296993262433, -0.006855597510910599, -0.056897660688951766, -0.007003039252763985, -0.07483691436908069, 0.059147306652718855, -0.037145701517024, -0.10676042018861241, -0.007378878020458932, 0.05265726243753361, 0.04942323587225657, -0.06809882017656484, 0.09424336751082538, 0.07324558671093453, 0.0035482300999939848, -0.15329717224358427, 0.11673797249540337, 0.027876767739997373, 0.09722168278446709, 0.06954998998209887, 0.010391131103462964, -0.012083497606938672, -0.06785045809016108, 0.0667491018313011, -0.12383359280265746, 0.1332620001998347, -0.10499791522426484, 0.1107215522508544, 0.1790155061867434, 0.06045361346285001, -0.0524610247390597, -0.07127300195004145, 0.12435245848746976, -0.050321037803182846, -0.09842235464915315, 0.10292796919747414, -0.09466819160683651, 0.14126715744756663, 0.08969362181915194, -0.04761579490568341, 0.042485630222193434, -0.04863939227640482, 0.02072502624138959, -0.09534859552857468, -0.07878810695853564, -0.0013152388092538014, -0.011730372350346656, 0.1179581861800129, -0.004166326443669644, 0.07286677612194488, -0.010003358051691051, 0.07276757339673216, -0.12963759048215573, 0.19692651068193187, -0.030546868598470722, -0.034983743074074, -0.07769542430542564, -0.05639231820668733, -0.0252777988647875, -0.030599649608160362, -0.14379986903924868, 0.10407830222600574, 0.009811696573432163, -0.016082306667143692, -0.01324260654066018, 0.10009836279001023, -0.0752688577118189, -0.042429306153433446, 0.08960759808173331, -0.0734285625570109, -0.00934923324745458, 0.06216083477320307, -0.00906500110105785, -0.03635568697587457, 0.08795637522150329, 0.036763351124192316, 0.018057440040629116, -0.03922649959313568, -0.07758366949597552, 0.03650084217927197, -0.03273911532117887, -0.10399942293636613, -0.12089401291598201, 0.012751561917797593, -0.027021523544936414, 0.023896966159859253, 0.04677412045314162, -0.05380961655970772, 0.056819832034249264, -0.04578287189699574, -0.07376952923745694, -0.001964117256361062, -0.04778661990526856, 0.08778890747996446, 0.005200645019374431, -0.00039075901110494207, 0.0687091708837364, 0.004049862985781689, -0.04311892281600488, 0.03633915733940966, 0.017405341577857968, 0.0987054902693643, -0.12141119671853869, 0.07442807855067018, 0.14930514302465323, -0.0486948352989983, 0.05182549898217668, -0.2125938311176005, 0.036531890257597864, 0.08519719558817941, 0.0059336000029957545, -0.003211817925772828, -0.0009370050754468111, 0.10370204000286612, -0.10486722173368503, -0.10438284976843984, -0.02698821713526512, 0.06850788566402043, -0.0666875854168329, -0.015052897010966496, 0.004640414202238868, 0.025381359367645847, 0.004830513028664926, -0.07461904179590098, -0.03920592975789346, 0.1289790679167214, 0.09886376039978648, -0.06267840620884503, 0.09911491859190705, 0.031940248035785844, -0.06395086988814944, -0.032468519329213796, -0.12322433568425999, 0.03923464039698539, 0.03658507213911905, -0.13087935532266748, -0.1289238394157677, -0.013606945997546905, 0.0695696999736695, -0.007009477812984519, 0.08760211777513466, 0.14287951448327016, -0.07636447612854191, 0.03989255289727165, 0.017619077414893262, -0.06061288045289829, 0.05600101477589602, -0.08092759826115255, -0.0026141630174868024, 0.07976417704858321, -0.0825144219342916, 0.07869114456215522, -0.03624842525974287, -0.03976900731072152, 0.1192521114454754, -0.0337674145949331, -0.0023303597400997295, 0.15621047325881773, 0.028840743639316612, -0.052997933162844645, -0.013721837384038272, 0.05022524705248825, 0.024656007475586817, 0.007016066727964449, -0.13312032981476396, 0.04490102932475871, -0.018609773694105894, 0.04362204657807195, 0.011409261349602568, 0.08410292444725923, 0.02624817261020629, -0.017069732328950774, 0.03885990924332268, -0.007987341855816314, 0.13028478481156697, 0.0018065261965228327, 0.021353078373609082, 0.09952160063900765, 0.012287415528159533, -0.18900462007257668, -0.03765188417087355, 0.02126124465276565, 0.036106562245030394, 0.13222268323915543, 0.021630142372666805, -0.05536644376740319, 0.03754672269153856, 0.07839663404817292, -0.01834654459209564, 0.05972126379659658, 0.014418874307380982, 0.073853983700502, -0.009703604108994119, -0.016580308189512845, -0.08872679201342076, 0.14589230970494613, 0.16896712056286423, 0.037335733655202236, -0.08940345678155925, 0.03677326128048378, -0.023664638709591627, -0.0440360199463791, 0.026706782506065656, -0.026296379483463344, 0.12626714991403526, 0.08071187516606367, -0.11421016398975033, 0.1405110852962551, 0.027372538210500244, -0.11779264939959465, 0.09005187066868525, 0.02880657499754955, 0.05391758755683045, 0.011163403857792931, 0.07639695076124597, -0.07051387682366289, -0.033887726360018536, -0.024076633577437508, -0.04079724133215334, -0.01112311036949931, -0.09143537945992292, 0.015075934568576893, -0.06692765143367971, -0.07352613476720715, 0.013088522502072736, 0.04469201076271923, 0.06602409336742511, 0.020276513669213504, 0.15852984496970862, 0.019884224353214106, 0.03371949591510871, -0.03848152479328736, 0.054694698022043325, 0.010875358285286454, -0.02146793825290732, 0.03611048480099241, -0.06150938136004458, -0.07728334578182347, 0.051198269010626136, 0.03234214025630855, -0.1039191670312338, 0.09198984632545208, -0.011466439750487747, 0.01564021154152548, -0.07210732131362407, -0.11163346760269763, -0.07973757439369406, -0.008015902224977608, 0.0053886002787142985, -0.042328708130720274, -0.17405322240366675, 0.07018037718585764, 0.028870457761909933, -0.07119218163039215, 0.09608658332900046, -0.09526734843815436, -0.015050513850703339, -0.08469149239565826, 0.09624132506362941, -0.0034710873920910704, -0.06543131942350232, -0.009668617284085043, 0.12189831130977674, 0.08011185276163693, 0.0614588761183212, 0.0031138596102787404, -0.01856880704526347, -0.005970610876142637, 0.09299394340679319, 0.059415045480545166, -0.028647986297268932, 0.06812831149554524, 0.1565186337622815, 0.08777050350508543, -0.06274224966807601, -0.029166552381493755, -0.03801211904551132, -0.0931773530673722, 0.0797269747214667, -0.19068383488910093, -0.033623453259076404, -0.106073647465262, 0.015353330397445422, -0.006708229275570775, 0.08249216164302037, -0.12315431865915744, 0.04041098167055919, 0.031235625374580314, 0.02779367959908614, 0.08583352692533788, -0.09395884907475642, 0.06835038676059307, -0.016824992592431753, -0.07395989733745979, -0.06827735618307033, 0.10737560753276687, -0.19427682630822318, 0.07893666056981337, -0.016210397894760097, 0.034799854111679025, 0.07360978659484226, -0.08074114445083008, -0.004966106982940482, 0.010243422921982331, 0.005158878768708301, -0.059542492945638535, -0.11110014946081648, -0.007248498283219053, 0.10725519790787134, -0.11917955854724435, 0.06951649304154583, 0.022827535505070256, -0.12846532839571378, 0.004758460384615166, 0.05587529023252497, -0.08390812732669523, -0.1210086571471033, 0.06708740333360881, 0.03680128009170431, 0.016910087026805843, -0.06474133461576981, -0.09117546507325988, -0.037991138859846514, 0.0487188384277283, 0.03926813388005867, 0.029573754143726903, -0.016680222673378806, 0.030670223154775928, 0.14909376646392142, -0.07309547827756531, 0.07913619707210687, 0.020767232608969993, -0.19246666214499286, -0.02833681204628888, -0.06635286141549901, 0.14832370167362055, 0.07303270225359448, -0.027701605361369216, -0.015176541799959344, 0.10710050493315988, 0.0267908924742188, 0.04557018849342383, 0.039540081440508794, -0.07478874424526652, 0.055644513455964266, -0.02749595707587655, 0.023546833071929226, -0.1688836551225598, 0.046042412208426975, -0.020409325659964037, -0.05582922366255237, 0.11922766043353884, 0.13192233628776148, -0.017238977157769696, 0.05059736915190722, -0.055991242791341614, -0.09447806467713975, -0.1451366924432456, 0.0028863164772891118, 0.04194145452426738, 0.11479563776582885, 0.057198706026105724, 0.07724337732826946, 0.06023022406194024, 0.09818417135387911, 0.020320212499153046, 0.13142419261164745, 0.0412677764310121, -0.04959781881059963, -0.05032555167668126, -0.10631575471400988, -0.12218360400103122, 0.04523812776630038, -0.041791827100042214, 0.011326854698102578, -0.0598168192114331, 0.049904861589478634, 0.006991680370790451, 0.08103289799039187, 0.00527395874375184, 0.04735745604621475, -0.06896086853563282, 0.21065378196266432, 0.20776552425233805, 0.03821921670869682, 0.027133678448140866, -0.03499729847783625, 0.06266898114681536, -0.07650501600019746, -0.012260763798874377, 0.07751848147333233, 0.06700211496470455, 0.00033688596838643813, -0.021108199907093814, -0.05015741658325405, -0.1294123963610126, -0.05028473010424452, 0.04496406355630545, 0.056199052015306054, -0.012842823782315533, -0.1925316356327035, 0.04549969343730197, -0.10555282332983654, -0.10158688262346394, -0.11351003652086457, 0.0036988354221097687, 0.022036148678437905, 0.18415879236991892, 0.007512586449054675, 0.01550507169187488, 0.022229921041899987, -0.06919318429283924, -0.16159935524505142, -0.028967783583762504, -0.10108233526390102, 0.0002689673003107341, 0.08805940278814578, 0.07393529276934707, 0.0006013169319954237, -0.013850216723335659, 0.022849774847343858, -0.06649473038794894, 0.11291150647453894, -0.1266040558757311, -0.10846622858661203, -0.03524310227068675, -0.01472859858065715, -0.07453558518195008, 0.05182370272331324, -0.04444650637147262, 0.06624017991659367, 0.11380183051053433, -0.058343830552824986, -0.04076285470189978, 0.04805089573431295, -0.04331083186420838, -0.031284971838955726, -0.11100693703451954, -0.020393083396550438, -0.010084289332458527, 0.13726432426485335, 0.01987158857558204, -0.11256361924996625, 0.08440526807673576, -0.09288183865045924, 0.003895412655053728, 0.17711428511108843, -0.12796554685771624, 0.0021426058390776113, -0.03018974290978323, 0.014924283281959595, 0.07143874290522269, -0.03279398096044214, 0.08788243707746447, -0.020009990571992233, -0.11574855108877198, -0.02892344508851954, 0.07764322489846219, -0.1024006286227536, 0.019942865692744387, -0.06942471008501461, -0.004808509859087028, 0.05208868681719202, 0.05076892675280833, 0.008519810384812431, -0.02575047147435412, -0.025633700714061677, -0.016616117159992134, 0.025631378242387936, 0.03525985001204613, 0.0803452053213234, 0.13943087175256078, 0.11637136064590377, 0.013174775290957763, 0.08944032293714203, -0.036167937835736935, -0.028935969141831355, 0.0038393711028942763, -0.061959977855888713, 0.04007646415461648, 0.047622914280839736, 0.08188678084599083, -0.023966545026757605, 0.12043822588205687, -0.09052656916063481, 0.060244976439268856, -0.0668335895913988, 0.11830516933285143, -0.05883393221024601, -0.0700107173845203, 0.03335560739434561, -0.06129973920438602, 0.14456924511640085, 0.09836230936111723, -0.02033354399899491, 0.13455149586674495, -0.06276202039882224, -0.059894053138617405, 0.10750661774987205, -0.04460467041501264, 0.055039346295876056, -0.08800843674643122, -0.10721620325362886, -0.11774716040718533, 0.08859226457113878, 0.015135229932008375, -0.04461018721390439, 0.0325797907777166, -0.1905203492958626, -0.05002122531455199, -0.08457618470436346, 0.01446757189934312, 0.008647801963226466, -0.05515030674402942, -0.05965719087076488, -0.001687227659002151, -0.006420934642311659, -0.013323814383265086, -0.09630443909830154, 0.1312133471714232, 0.03307021795811975, 0.07373136669981734, 0.15904694433982447, 0.0690117777489095, 0.04829915528038042, 0.07010215394503685, -0.08392540611530792, -0.015379768730761885, -0.05505653039266748, 0.07037452041046838, -0.05443085513265965, 0.058648801374600684, -0.03616330946297696, 0.11393452948708611, -0.002299791967686604, -0.049616290636397935, 0.017433443746316505, 0.0971640693377071, 0.1239460669808092, 0.03583970833549852, 0.028366293048343226, 0.029730900646447476, -0.04769859978689996, 0.003168780850013438, 0.06804577162655694, -0.0198128413530059, -0.01798921177947729, -0.009922889006226765, 0.057537968738931966, -0.03317984606110055, -0.10870369855785879, -0.08164660898571853, -0.0798153701755885, -0.007855294406329176, 0.021708701283831372, -0.025844545770260467, -0.009394258003552732, 0.08562776435713607, -0.0015639660898351035, -0.009744737540275791, 0.07036689287233817, 0.08963354478892288, 0.07681345415644861, 0.11957467785498876, 0.08614924916091382, -0.038725006650832235, -0.0509717018239061, 0.02014808362619056, 0.023087550178671375, 0.04966035037192459, -0.12339509024371807, -0.013977741491134229, -0.03713056563489663, 0.0506732854465271, -0.06382425358631132, 0.005865683922865458, -0.043256851468933384, -0.05783112461366823, 0.060500621664222425, -0.03800238384555902, -0.04755870232085406, -0.10676568036677249, 0.07021880832970774, -0.0427040681133759, -0.07475539470173868, 0.023722356735012974, 0.026832687952457423, 0.00918347937116132, 0.0603070649707925, 0.046773427645530725, -0.07745309015301675, -0.057093205253359106, 0.0795418686563248, -0.0022406832452484043, 0.04454661607249594, -0.21034732209749957, -0.09377809181044656, -0.07582783752156647, 0.058953608409864106, -0.03430104989684815, 0.026602890607471282, 0.14784787255587944, 0.006900441484691419, -0.15791542753992122, -0.10810103462997432, -0.014925196397362532, 0.028571124575328243, 0.013533712284777939, 0.03156697001201972, -0.07744821702674522, 0.1857439957130853, -0.15883701492867894, -0.0638544968733591, -0.0912128347096945, -0.008994680635767624, -0.011467135090806971, 0.10504990375367929, 0.057927381992006644, -0.06401698181797827, -0.05736271386297416, -0.0011407903015506848, 0.02190762008252522, 0.07766933984951645, 0.06441483624028119, -0.006835729842329191, 0.03173843246064068, 0.08812919102316992, -0.036442675600548084, -0.013656028743671087, 0.06178662755749994, 0.06943042097294397, -0.13539974834209292, 0.028018261487882876, -0.029024264441554155, 0.11957696343798314, -0.06182690957242366, 0.07779459132991269, 0.015692886509059124, 0.08495609378753063, -0.02131636251605713, 0.06652693532078886, -0.03778412072730707, -0.013452127332664859, -0.08308202797882892, -0.08096251034275262, -0.00021876353620000667, 0.06112515506405959, 0.04689491324320121, 0.03847206204856999, 0.12240225347127984, -0.048282311980845816, 0.05498219793875335, 0.10555876588247805, -0.0019054795018737145, -0.060339709709281875, -0.011233072396736671, -0.0825747987047873, -0.09318133545070573, 0.009179491338297223, -0.0472681822017267, -0.0351220599239107, 0.12171324904393933, -0.07592218580686265, 0.12745343488719826, -0.0360393354129197, 0.000715971467301293, 0.11771634522246184, -0.008631658303935254, -0.09762850032325229, 0.04317370333705205, 0.07660239303321877, -0.01681551254499529, 0.06667712935592597, 0.010528314114297815, -0.03543650753711567, -0.0025431325112765476, 0.033715007740742084, 0.04862073271821845, 0.00853772115032813, -0.028686153896816034, -0.06534264164526549, 0.03367657457791599, 0.09749106554591908, -0.018176739706511433, -0.07179001090510487, -0.03458135806016201, 0.0012054066385799517, -0.013608733375496422, -0.04110042118211595, -0.06773352091556947, 0.09015155656647646, -0.06878222964610564, 0.06007874844935926, -0.0037087226545217916, 0.019057504390672, 0.09340681299927966, -0.043722926704390756, -0.03700686933808983, -0.16623742775558015, -0.03406440606081315, 0.05011599925104813, -0.029985886925563345, 0.03940089198048858, 0.05567689652691282, 0.1227366790317689, -0.005576511558342011, -0.01438758007288147, 0.039494366686310886, -0.0012289678069272718, -0.009110177616686626, -0.12876940141033194, 0.04087483702471434, -0.011918086413300576, 0.01755968741101617, 0.05515285404817704, -0.11906920453654178, -0.04452833511874157, -0.021821768783515914, 0.023030573273907173, 0.0023227564991320808, -0.005256739837984386, 0.025098488400736026, -0.007953877315806551, 0.09671338378169388, -0.01976427028409784, -0.08726146908118722, -0.02239170976778491, -0.07038899298770765, 0.01793977536262463, 0.0030180643586380966, 0.048358331239682105, -0.11996033359705773, 0.0003881446680512286, 0.07050268791060844, 0.05982917335719972, -0.032404049870844304, 0.1076414198075103, 0.13120997366928347, 0.09260976796027978, -0.07580390838536269, -0.09319516677625285, -0.03579097876916275, 0.01946770905727681, 0.03470265625837541, -0.0018008083793670869, -0.024302207456482226, -0.07690384221076277, 0.10197425621543169, -0.08731567296556306, -0.08667320026913232, 0.09256889388328031, -0.047059479296307084, 0.054886535101417444, -0.07452908023731117, -0.15919671367995336, 0.05166304833223771, 0.0324798802406132, -0.055383862829701924, -0.15236129001933807, 0.06807342520700517, -0.035469158830154966, 0.006292450957184192, 0.004503600121096268, -0.011490274671937292, -0.06693575332647063, -0.014252503267812454, 0.11192315579974751, 0.040913645582592875, -0.013471484617635317, 0.01600370515265488, 0.03030564831562661, 0.10643813626843479, -1.1888566818382837, 0.025904382382335455, -0.01858364339519164, -0.037068515373960546, -0.04151484076380678, 0.12456045430209507, -0.060217313337113804, 0.1464807553267895, 0.08740998137008946, -0.13837307763648696, 0.008383229196721687, -0.09459162474774623, -0.012078830005637172, -0.017787981117177804, 0.06952599743514253, 0.018925616584179825, -0.01618791270206714, 0.008334207189770832, -0.03789770663812126, 0.09870578274846795, -0.04327166268086886, -0.07528305159379785, 0.003481319902778023, 0.02074106492734867, -0.015458011146029084, 0.06265031550124082, 0.03861820550180269, -0.03599673942728344, -0.04464260660273877, -0.038409287397570525, -0.09751330577317273, 0.09179959962743295, -0.009886252883070415, -0.027970784312761957, -0.06751016715954766, -0.02083549440622267, 0.040498686871017556, -0.1178628069025428, -0.048226513975728594, -0.08345424372569922, -0.01515001128617849, -0.1309317120496876, -0.11195260386803134, -0.004048848691839838, 0.08933369886874881, -0.12018628724239451, -0.018699017640016514, 0.02667092173371984, 0.0031114746684386776, 0.06315500782050956, 0.05681594121742315, 0.03759823806091875, -0.035540663893460535, 0.07303946165222633, 0.07712367880776928, 0.0434081366296146, 0.06715445026663286, -0.12275416548274679, -0.13088793732204573, 0.07826083556492971, 0.008854848977708319, -0.08635124966521322, 0.002134281995083691, -0.004191958597874943, -0.05727458486641364, 0.005995858407973097, 0.027106737459192676, -0.08044323042287062, 0.016691869206570085, -0.11608382466411683, -0.0333760301042658, -0.03683262138430408, -0.05739804698717912, 0.039410100927199765, -0.06050601650216471, -0.02582828618944112, 0.033550512150921884, 0.020032324483405105, 0.05621558088796364, 0.0103959224285838, 0.1544961335183977, 0.005844692211695556, 0.041411825813740415, 0.020505033729485103, 0.06044000549499448, -0.05961254869779694, -0.037746340395821025, 0.07425038478838003, 0.05622962966561701, 0.03447607018999315, -0.017647882062420007, -0.02786469296124364, 0.004708080177133805, 0.0018898562876866918, 0.003504922992172075, 0.012981164308121927, 0.05717517974929901, 0.06293041937033422, 0.028443289086545868, -0.07556411637200201, -0.0689745719744047, 0.07431882430410351, 0.029445078823848687, -0.009987053156581336, 0.08634561042613155, 0.013214492079476581, 0.03464250759063705, -0.029828133613238092, 0.002410839940045961, -0.02034365938054219, 0.08524862467940203, 0.012684588916251393, 0.06220112038138626, 0.08034356197581885, -0.015444294889617426, -0.17814319806170587, 0.044786335750841275, -0.13971501087947305, -0.08749454827024479, 0.08874219210092923, -0.024601602565518417, 0.07068181310433293, -0.08549377391461689, 0.051447479307202476, 0.0017876642155028035, 0.0913719308818428, -0.1482715536561605, 0.05213403910009441, -0.12918708888876615, 0.0017836334559627666, 0.046904470865936035, -0.1227444649077115, 0.027195871998908938, -0.011002690642867477, 0.08220490512162315, 0.033643111826522125, -0.04533980495831928, -0.14326043879115005, -0.03616033020464874, 0.013661339101899995, 0.03148845566465838, -0.11922898688027266, -0.123524547673313, -0.016529255365005734, 0.10225582386286744, -0.03663845875606511, -0.014508632968212058, -0.030222140706250507, -0.04078184307772665, -0.027435446588850075, -0.04022574902933672, 0.0708174321871408, -0.05550037649359295, -0.0275664094049632, -0.06327322616681193, 0.030673244084626163, -0.011833420035668767, -0.05733187481229804, -0.046078184407637474, -0.0382062489923961, 0.04309839599671038, -0.05625063842238591, -0.02328235418137407, -0.08450461163729828, 0.0349501075938395, -0.04146485209923234, -0.02471083629128277, -0.10003960232325433, 0.02967011700830558, -0.056198742660212755, -0.11171977305723223, 0.12713286240530608, -0.03045138159354853, 0.07812718574449101, 0.1334558611501203, -0.0833629287778874, -0.05573553481223166, 0.0923786983277744, 0.008369291030138244, 0.0037447661310776835, -0.11540455814178473, -0.10534942483784566, 0.02923436282824193, -0.0013455469954560264, 0.10273023685928172, -0.0600653531680662, 0.030208895495854992, -0.11869986377808749, -0.025525464714636917, 0.05468411038317133, 0.0036420052538583742, 0.1439630512231326, 0.06782166014405079, -0.039978324735626285, -0.049318312132366926, -0.0940352428549212, -0.07429902668050577, -0.012745628473816207, -0.01723530992453595, 0.009370128378092701, 0.04428967410795562, -0.024049177072570992, 0.08461224979070577, 0.012133362949087975, -0.027566564590615145, 0.034334460836996664, 0.04726836787793548, -0.023191185332421205, 0.012313027575681116, 0.11642613800244105, 0.10221769686408831, 0.053624296757456606, -0.17942155193664777, 0.04308474774759327, 0.04033300391562007, 0.022097024705122828, -0.17694672051799373, -0.0002115851008280738, 0.05637793143751507, -0.07790676173198405, 0.027219775339664447, 0.036007326995739046, 0.11642861673654102, -0.06051575644548924, 0.017459579756810714, -0.05170967446638449, -0.11878407673488213, 0.06462429914726904, -0.06343404987954496, 0.14695809983332778, -0.15811843254125615, -0.04662857432321903, -0.005160222505435448, 0.007120054143098113, 0.10171434806938237, -0.020770014703576097, 0.014703732211512631, 0.02206524907882017, -0.044053111645433136, 0.15299187324888824, 0.05151683302080372, -0.07845484057940631, 0.0731875423779463, 0.015138144214366264, 0.0729445073592924, 0.07064260598215817, 0.07778741536243203, 0.027837022691894704, 0.020161370676490563, -0.16940365209958397, -0.0969325459175922, -0.14576402611389896, -0.0209173785924683, -0.004448184424339003, 0.056873541370878404, -0.06092663316691106, 0.11023364470954204, -0.0896703699258555, -0.0510998235152264, 0.10125907552971283, 0.04444134046271333, 0.01190668409336072, 0.09584230848356812, 0.03018396017349593, -0.04218610529679659, -0.0657178166144104, 0.019171631392344488, -0.0713185556298949, 0.02384880310408605, -0.08956687990608189, -0.07176498168674245, -0.026300169578280953, 0.05995647042609425, -0.03228385357199365, -0.01231079364537307, -0.014848046935034517, -0.15575520978929466, 0.09125014433128495, -0.10120357839389378, 0.03593133783985124, 0.0777677770781532, -0.019697155216933417, -0.07077749649912556, -0.09281932147716657, -0.14034396002511998, 0.0357713344015031, 0.0036958135428835667, 0.12171617975080258, 0.04655834803078949, -0.003200826380231166, -0.09489811615202792, -0.038526931186514304, 0.14158054626522854, -0.11476486186947756, 0.0018751553331432262, -0.025191465864509502, 0.0015883959430608274, -0.014053925211185075, -0.08087643511932431, -0.08766333315095516, 0.008390965174215054, 0.010961233076639775, -0.06084512893591371, -0.04921609403472046, 0.12843190060077295, 0.09584085466257769, 0.024303247889533433, -0.014038203911526114, -0.06854692519067301, -0.027200027136722708, 0.016430565773963027, -0.08130149850314775, -0.08118588557276073, 0.05912764191681839, 0.020318251885892785, 0.09314238487032513, 0.016000987344093845, 0.0209648946073509, -0.04018745601600144, -0.013606370659201877, 0.046855367969082955, -0.07939260816406224, 0.16977171458712031, -0.05540083792240497, 0.10413239309424127, 0.021942994770252616, 0.04593316925354632, -0.036719773660730384, -0.09636943265318375, -0.1998576109195908, -0.011880622018538544, 0.06824863449902414, 0.05447151146388648, -0.04634687820828464, -0.1061453730041743, -0.04426788484003298, 0.0468943638298348, 0.07168800656481439, -0.09297003121588819, -0.0852669082166991, -0.08725181050313553, -0.00045698306736001665, 0.032967462345132835, 0.0706849266542803, -0.1095007573307371, 0.08997902223693564, -0.05384205229255744, -0.17619297721539082, -0.14907642408174543, -0.09331130009732849, 0.10761417675354153, -0.13337006437188167, -0.026000021744103236, 0.115192802764158, -0.025182175240508527, -0.08547174950064344, 0.020378929750968087, -0.07007442515794883, -0.04831829603643619, 0.02342613324491774, -0.03141396725523725, 0.03752965745452954, 0.01737589611879338, -0.0938976321396601, 0.006322469671902463, 0.016581838907870856, -0.07275900156287526, -0.10932764467724995, -0.10437221699334903, -0.11070603560091077, -0.10385633018958396, 0.08913712933192729, 0.1401591583550534, 0.05111818488426939, 0.0005867142187048743, 0.09520484902325217, 0.005796977266307959, -0.13312854400587826, -0.05971915138378682, 0.11521010686633316, -0.01920442153044753, -0.002820645861647557, -0.0514286663618298, 0.09561085874303209, -0.09293109964577174, 0.07173307583205399, 0.08249134643041663, 0.11333141643107192, 0.04598487933031239, 0.046036386758622636, -0.0072147450612625235, -0.14373324492906048, 0.11076500912096282, -0.1531417183421385, -0.006746285926279964, -0.12673013103685313, 0.12107012779958652, -0.06557822963295111, -0.0834590659256034, 0.0994325379788981, -0.02449729878094053, 0.1791486911440048, 0.021081298295434165, 0.06010661176447575, 0.06052755964941134, -0.06132101788410645, -0.060489967313966886, 0.0717266671331158, -0.11303805908983137, -0.05666029467781352, -0.0003268491172574664, 0.1242524971503699, -0.015227217582789675, -0.06675864517387252, 0.01911840457084139, -0.03649117384256229, -0.09582411671207955, -0.012776283345399543, 0.056846279022914725, 0.005690824675473114, -0.032867061443905686, -0.0768281339310384, 0.06505566064032527, -0.05606107543224449, 0.013698128819261593, -0.16939685850250796, 0.054550992040574495, -0.03499019930777109, -0.20364775597755344, -0.02654942292651879, -0.07963239957818255, 0.052310721997205045, 0.1063333877841703, -0.0811689831442786, 0.05989370994936562, -0.11363405151033748, 0.07655143805258965, 0.11606925243897218, 0.1983324124685443, 0.033168320569617354, 0.0013476190985746857, -0.009178536666120769, 0.11170143647704575, -0.011878641768516116, -0.1381329059504725, 0.03274509948571216, 0.09300135479871485, 0.0732800776737609, 0.05445079058114863, -0.10917499738010356, -0.06472003603332353, -0.07015693980633896, -0.002820280865784008, 0.13291273972593912, 0.044702911169864364, 0.159166045503863, 0.10119516946830101, 0.06682250354865836, 0.022236585985009504, 0.06481710667378615, -0.12420134972137434, 0.05834460096318371, 0.05148641389706461, 0.09327797795592194, 0.05181483864409571, 0.0009794580852161507, -0.0914657069042141, 0.03193253506927621, -0.029237769573748164, 0.058405012083691595, -0.023948261421699826, 0.01444191655790289, -0.15787971767948866, -0.00555309647369978, -0.14943345434436303, -0.04223536655446057, 0.10185783152354771, -0.02403451600717801, -0.07697540717241118, 0.15017662647029573, -0.04358822743247126, 0.018805769388628126, 0.01333256007184462, -0.07393895503114205, -0.02241265962992797, 0.030369910122236186, -0.04420065724089253, 0.0085316062991048, 0.06357318441153514, 0.018245792977563045, 0.023465559928006106, -0.01799072608247405, -0.029157393534407335, 0.010618466395861842, 0.13316196767871558, -0.048887133528873014, 0.004622009779756292, -0.09412720864331796, 0.08322741239819693, -0.08588871404722383, 0.048450652794886306, -0.02861224773075691, 0.03862217451462992, -0.05877853846045182, -0.052241895959938986, 0.05388304060369382, 0.09351020979495986, -0.04290065038173358, -0.02547772579393539, 0.0006381626991236376, 0.0582257726704261, -0.01713232966977509, 0.005787236592650721, -0.006560261981458871, -0.09934881720780772, 0.023353602238283585, -0.06041235492629347, 0.06459463465411511, 0.003938052067884208, 0.019060151991990286, -0.04119124118043725, 0.12752230837719847, -0.0021429239394499516, -0.06073236553350507, -0.20318877152472603, -0.013283397100927512, -0.00046355914374150215, -0.05649257571771867, -0.056719944345176715, 0.10266219098976759, 0.07789502905177248, -0.27765366598497093, 0.08528800204099501, -0.0640802327542779, 0.09055470795102509, -0.12364358186227714, 0.0614948097828671, -0.14875341472617082, -0.1463367336233539, -0.019505246190034072, 0.04629716547630133, -0.0059443043588861735, 0.00025091463636702253, 0.05405021573486903, 0.00975213628328549, 0.17844580342396685, 0.1462369673803276, -0.035974659705709094, 0.06200313601246959, -0.11310709719041433, 0.011389002841433482, 0.0493728940671112, -0.04847097827569042, -0.020116892953917675, 0.07693092228841679, -0.05543196065044174, 0.06979154183866446, 0.051776871590163, 0.10797397243753505, 0.07027831090123371, 0.013495074681457053, 0.09384537009738217, -0.039762738685216556, -0.0720984927699603, -0.053993659409730235, 0.12164046509332221, -0.0861053484320755, 0.07036829798042159, -0.028889851374063094, 0.058277070798223744, 0.0019313325664347984, 0.03407482477983068, 0.0015926117854525408, 0.03550546136823874, -0.004610460033136182, 0.021587585900524394, -0.019223456893863306, 0.07421136440569631, -0.030675662506948004, 0.01194954694365605, -0.10135384395748995, 0.08343879581584454, 0.052548259624992336, -0.017042220811195326, 0.04471353883114553, 0.018642764137650994, -0.021214569974004695, -0.036766322627173886, 0.06303888905388612, -0.12915216825464385, -0.06088274856370174, 0.1145478472836448, -0.035806246241528124, 0.10940497819002575, 0.008035701512648475, -0.03995843120144454, 0.009021812373184916, 0.025872088249154664, 0.04742728992962629, -0.00927086830470405, -0.09025520451289133, -0.015643988785754605, 0.016570356485606435, -0.04763313973291705, 0.021316673214145813, 0.029482705860014752, -0.209293674700332, 0.07241878023458082, 0.034083660416432064, 0.01490436314665399, 0.06920417288629621, 0.10525230715703686, 0.13864570826141753, 0.010636916513514418, -0.012513597432862681, -0.11882984420371326, -0.05317305008785948, -0.07561157357334718, -0.11911522824849749, -0.024694697136550228, 0.042919704124301856, -0.025995701119543965, 0.0598014227244496, -0.16992221544505748, -0.01979249025439037, 0.04676442347360932, 0.0665335701922058, -0.06312778093682117, -0.07061988898724214, -0.04784646186694797, 0.03774440920908941, -0.11663240859992793, 0.018385515228215923, 0.08907941690796031, 0.056698100900979986, 0.039989477386700195, -0.0815419196228621, -0.11230283100494115, -0.01994837266990335, 0.05358100741073638, 0.044275278462063306, -0.06089975589342579, 0.1703439356424448, -0.0055783106954719625, -0.18778482189036277, -0.038308365112162165, -0.05688585711056695, -0.08217944405729727, 0.010441993985639648, 0.016089467940181155, 0.04406109639453955, -0.15742959203238627, -0.016567389971098325, 0.0689954577902581, 0.07584854780415075, -0.0443219947183634, -0.013846985913756004, 0.0656316187871964, -0.027677600709085478, -0.03160372839722563, -0.02785493740353896, -0.0038487890320416766, 0.134462668489447, -0.04575407853940021, 0.00045004785168547266, 0.10903860073255812, -0.165692151530896, 0.08576610590546564, -0.005028460668785404, 0.014896321025111375, -0.00508231419935123, -0.0071579246071136, 0.042973051465885564, 0.08946962581035413, 0.019448540718573167, -0.0028987023158246476, 0.06554517454909808, -0.07616988648287698, 0.022523188941005908, -0.029785665059534932, -0.06527912377161735, -0.12376455014785982, 0.04529103938182615, 0.04530409904852863, 0.10269718072685641, -0.0022515975496904656, -0.01018596296582695, 0.0128630897829878, -0.10911590676522706, -0.06725338520656317, 0.09807527325626304, 0.10294570160972147, 0.024630635559113465, -0.054242298006598806, 0.03915360892955426, 0.002952791998899049, 0.03745837051511235, -0.1543978357229098, -0.12267651579951512, 0.04361605497646826, 0.04515376869094698, -0.037660666431518296, -0.006728405791729152, -0.07163455692921249, -0.018724288626463827, -0.06347611075480268, 0.012420286116071025, -0.004324709369228523, -0.11650955089126652, -0.04200675753056328, 0.024816952292320858, 0.07429480766434134, 0.0055254614718127475, 0.031324583243645884, -0.059098317656248345, 0.011659789340520296, -0.03817305751329434, -0.01189018873334292, 0.012912461375946961, -0.09864580549564206, 0.02792395232383767, 0.033653572811200995, 0.0798574825059104, 0.1922114532750674, -0.16408662850919511, 0.037648974225910195, 0.00426839518341361, 0.026395207527764513, -0.08373929992246201, -0.05555577244342818, -0.03769651391060161, -0.012791815576949634, 0.011545222391855951, 0.05200078619116563, 0.003583030120158254, 0.06470344099894723, -0.08149808036337014, -0.016917660083341626, 0.030364685774950318, -0.04734272157029468, 0.057580964823839245, 0.096819910288136, 0.06409283658715044, -0.23791601848582852, 0.03664334424095855, 0.040221418431362115, -0.20125847396552798, -0.061829667155198755, 0.1719141864949903, -0.016976402460437308, -0.02199063749819901, -0.05495903343622688, 0.005688192083528562, -0.06449147456213775, 0.03083355001024698, -0.02345142465802648, 0.06522823515128255, 0.1048931546884564, 0.006641177362599338, -0.0807814064689327, -0.06616255944755171, -0.13943490053864147, 0.013236141814950118, 0.019156629780986006, 0.088302907150883, 0.053467530691402074, 0.06508672747455223, 0.04389971774834798, -0.21837935720968213, -0.06775107523693609, -0.08667666663787124, 0.05093330585710586, -0.16428084049532277, -0.026811529186416938, 0.013368511608138305, 0.025871053828478912, 0.08856032515556614, -0.0359113247085689, 0.10147399162428944, 0.055483007169024635, -0.08029890305672305, -0.025538431450837944, -0.04880402188710271, 0.054416953385382695, -0.010263833749915038, 0.03193142230373406, -0.05160899252789029, -0.12641191011800795, 0.004706099695544938, -0.03454550121401199, 0.11710373519215596, 0.1463344398645004, 0.1070718657262753, 0.02398745824952939, 0.004199434976643428, 0.10291869059571276, 0.06491027453307385, 0.011625595346344894, -0.02029894611247502, 0.03335498912379213, -0.06701398649319835, -0.023674773008949854, -0.0045797785642309715, 0.011034303539121524, 0.1375039262365188, -0.06083710696272707, -0.1039054943160677, 0.08016539853736455, -0.015513626816666801, 9.281742668656386e-5, -0.02107986796081636, 0.07235115175583459, -0.007923448653118914, -0.027700228096266172, 0.07082684872602285, -0.1086285570026918, 0.013040473296626652, 0.030584416840658477, -0.04126936325778312, -0.02056980228283751, -0.0510636288663767, 0.07941884524579065, 0.051027002420385895, -0.07351254713839146, 0.10499998145794495, -0.004708388459722396, 0.09422688945259584, -0.020468399556513524, -0.12087571344396728, 0.0640973371626066, 0.014542544517251647, 0.031283636356284596, 0.19618316104828323, 0.06363752542546201, -0.08225946336133978, 0.10484294787307882, 0.06888284529373226, -0.09454559307370816, 0.09323441820640506, -0.06910371020601998, -0.0712391258509409, 0.009609636341842157, 0.05912702743145968, 0.0956900889440439, 0.07479464511665607, -0.0046513755576389326, -0.06149313657134307, -0.045134550517353794, 0.05372702739586059, -0.18320359793983676, 0.10626709043880604, 0.04823837420066047, -0.08977466062185237, 0.06123435704489307, -0.165892893780535, -0.01242970655893443, -0.031343318582250895, -0.017073826841405938, 0.012927262394057993, 0.03402279691360086, 0.08491283214685959, 0.013001974926500204, 0.03631230520224314, 0.00889634574461439, -0.03254435914519322, 0.052751012088174946, 0.024435860592542644, 0.029859447915017243, 0.04248488681858185, -0.041029641734145174, -0.04527871134159917, -0.059109119462087474, -0.03183716343187942, 0.16056641552422907, -1.810568527608188e-5, 0.04149550473851783, 0.03253252063335891, -0.10877797338241299, -0.04149685583031129, -0.1536218643482364, -0.053983435675758196, -0.04460657336131355, 0.15717090612619564, 0.026656821682162025, 0.02458065288023218, -0.049067819328270965, 0.06224629594240297, 0.015693116530061255, -0.005593161977411308, -0.14325008072494794, -0.013995033903468712, -0.13701736910888052, -0.08828842092833684, -0.013250488081678396, 0.09005813843273815, -0.002359399004706382, 0.12418946134680568, 0.05549952027430186, -0.01648139045465774, -0.0398938415516403, 0.043610121324731826, 0.036963277496761336, -0.02077722683981635, -0.01518427873773223, -0.03525252176426279, 0.08128573252368843, -0.11104434732261932, -0.059407809668752656, 0.05150577272269922, -0.02795716711128348, 0.04570872664118656, -0.0006159569073738096, -0.09818544842708306, 0.05067357835194314, -0.04262582118324527, -0.11296001692215257, 0.09727791890514374, -0.1373300418282972, -0.011725663575484659, 0.07530591793142817, 0.02340638353303179, 0.09500797066975791, 0.06419202216273547, 0.14915462826417059, -0.024861027767412648, 0.0007439818232614462, 0.22187822557633433, -0.05990401747922519, -0.09895703263578404, -0.0149481958116403, 0.05384738945415892, -0.006206520539525371, 0.04785700684375544, 0.015450192867260439, 0.07862863473402454, -0.034314407977574825, -0.033808284609358114, 0.006494150535047166, -0.061200141566094365, -0.11730865781414483, 0.027974196552912906, -0.055608760302283136, 0.07938100724487686, 0.02796722291152858, -0.04027746900381951, -0.08380353423901256, 0.0006476889683026406, -0.11469584412431953, -0.0022047623735083404, -0.11881015908476797, 0.10546901320206196, -0.044848682450577215, 0.03007696395591012, 0.1245787358141495, 0.05971110721654101, 0.035347760784576215, -0.07065197183952321, 0.0010406932363302877, 0.0889042454978907, -0.06620058506083491, 0.03165235221207374, -0.04223625027894356, 0.07628864675579106, -0.053750802316685026, -0.015912729318901978, -0.02656461075616472, 0.14124988986738435, 0.09518867200729736, 0.03562920705351829, 0.0827330503369532, 0.02627476147985334, -0.03003389820068522, 0.079330526399676, 0.010886137069606639, -0.20865088467753323, -0.010307434520550385, -0.12491220189380074, 0.010820462861708992, -0.00184821314664013, -0.04388911824160038, 0.07627149923912739, -0.005066883522141455, -0.039575723826178834, -0.05497023990380913, -0.00444906942745508, -0.08302963008217532, -0.10831432386010986, -0.01469988378649997, -0.009627068847280075, -0.006144860342771632, 0.19242834638862882, -0.024609108966688383, -0.03247536267637, 0.07259338965985764, 0.09384151924124037, 0.06735141873886819, -0.07247144446464614, -0.0949468650392844, 0.01454336914394318, -0.00325452992639108, -0.037179135126655743, 0.03791710600156882, -0.10851951565757613, 0.10554295443123439, 0.007296211348121118, 0.008180041383097292, -0.007661928008209026, 0.0620655589130124, -0.0833709621494284, -0.023400847867521477, -0.08954114291073881, -0.08507058549598875, -0.04410987188238692, -0.04567558006180545, 0.030075773492742024, 0.09963887569833363, -0.07137099638170809, -0.031633138527760854, -0.04739574984860644, 0.1148254665159355, -0.007358023452666843, 0.061342123046580564, -0.07880948601487077, 0.15029766129800595, -0.03534787500846051, 0.04484909742771711, 0.05602629521380309, -0.13601516194554567, 0.04512637134654983, 0.1552756792595285, 0.0659871820145302, 0.03675410652449384, -0.011738846716768498, 0.011637719567888092, -0.10237046797608045, -0.013422982836802344, 0.04418559852957397, -0.011517020677927519, 0.09256503307640482, -0.004900183781210548, -0.11282915052446711, 0.031039448587090693, -0.09620610263504435, 0.03163038697594842, 0.011280440454135631, -0.0662296993480617, -0.055856990555254143, -0.08052044178289455, 0.06442587123988976, 0.01377825375458515, -0.008259433822248986, 0.024792098415292504, 0.04847710639139658, -0.08308307051739333, 0.036369379374840534, -0.1255938494376646, -0.012066803432863683, 0.12353847052442786, 0.000915618885183478, 0.02499497682479956, 0.11771701383632385, -0.142688009119691, -0.09452683860151592, -0.015061023043118437, 0.08445370712553839, -0.17834392213458228, -0.019047309462443058, 0.04545990218577587, -0.14621791062402764, -0.08176809507337708, -0.001467909758111318, -0.06343508433165165, 0.07288130744176527, 0.03872236376769656, 0.03725526380069682, 0.03955708714644905, 0.011712470951658132, 0.1205645599056404, -0.10204370548659768, -0.0028497097536361463, 0.08360410249621172, -0.10476621719254052, 0.02682580794771782, 0.030216084614026948, -0.05893783279535079, -0.007369874784874768, 0.08956701008784561, -0.08480777016248137, -0.03625975891346894, 0.10735990201834963, 0.03532135818033426, 0.13239160478143244, -0.04879319126457949, 0.1678154591711613, -0.07984130888514757, 0.027328444947796907, -0.033657279053299606, -0.053324227512751024, 0.035660841854334616, 0.127512242583776, -0.20301145648075156, -0.015117417471859322, -0.015737611487417296, -0.036189057005515765, -0.09073330172679771, 0.012344086152207251, 0.04267736120512991, -0.035801206038835486, -0.028248197753464675, -0.007421745643333161, -0.06625815197033857, 0.06451301465708933, -0.028899332561951385, 0.04337867451929328, -0.10078954144942631, 0.02874331282173321, -0.045262096508005736, -0.11128402034704302, 0.15738907668419827, -0.13251114129059477, 0.053865085783531295, 0.014890746003232595, 0.03585324191981895, 0.022551927272419624, -0.0319474223330042, 0.001400984335744182, -0.11071645276588886, -0.0025763118339194647, -0.02508075310150658, -0.16973987478735209, -0.09833992891259055, -0.005112091438683315, 0.057398503861882076, 0.009071087359921737, 0.051425621759040634, -0.009347094332986803, -0.017139047748422075, 0.07670374669925273, 0.024025094782920607, 0.01222881944731959, 0.017099426433939247, 0.023491378756121232, -0.0376443807810971, 0.092829208418929, -0.0711067084605203, 0.06413143579970151, 0.02612254368352085, 0.11878073876129683, -0.07851530956996969, -0.09972640298235369, -0.0950999686690982, -0.01717519886199833, 0.0034459948999994417, -0.041182519034022695, -0.13402943632503575, 0.012125465755501193, 0.015179158184444092, 0.05612404834265061, 0.039047021220766136, -0.03853204949470539, -0.07571622546104417, -0.03637101213189438, 0.12791579407681422, 0.15493343911045043, -0.07513081754229667, -0.05495396122002476, -0.016971295493441525, -0.07103030193709745, -0.16319324431061097, 0.024099763661203748, 0.07736744691104819, -0.07610644235794002, -0.09210957568332523, -0.11813325809501532, -0.12092298714152118, -0.05019427071280471, -0.11588568311403497, 0.014196690477665036, -0.24696124430668837, -0.0991285718039565, -0.09063443726991552, -0.05911459337704485, -0.028216236204854885, -0.10285124206681288, 0.06776264627399646, -0.11029818336901791, -0.07979139125657791, -0.12692525525969559, 0.12414895262528894, 0.01638018540462199, 0.027851424930558567, -0.18201449754592358, 0.02292076252855532, 0.0034381397393497357, -0.07141604829680001, -0.0660934079440111, -0.029275307790757158, 0.0991767666405516, 0.04393136117383468, -0.1344236264060253, -0.03765981579344742, -0.053258533379199974, 0.048020992776614896, 0.054409359632275084, -0.03809161137521612, -0.07766465410276656, 0.0648869993601486, -0.05452450341215784, -0.07517193968825626, 0.10085437996043929, -0.0007772740232185676, 0.14267586933214282, -0.0357369183087887, 0.0055285935755526264, -0.05172863360146312, -0.00587751434408326, -0.02275218371172863, -0.08398751945242845, -0.04149947220846718, -0.005019937117671477, 0.09107025743305273, 0.03881068091374261, 0.02046853213234503, -0.08433347460575982, 0.05038149582684518, -0.003617030188961823, 0.0456724883978279, 0.024546036532192218, -0.07664677575371816, -0.024612183660799914, -0.21470848247706947, -0.03244993490348177, -0.056670123553286995, -0.1442854821741349, 0.02530298792355932, -0.05801116389356413, 0.0038879710428121692, 0.013854820307542308, 0.016968683731192123, -0.05730080125735848, 0.01464827171205531, 0.03780183942922814, -0.02147588351651647, 0.02538797576990708, 0.04787106109942024, 0.04781206558223849, -0.0592146146795854, -0.06340052207858671, 0.099964339476778, 0.108050697873929, -0.01829922629821075, -0.08768670378900766, 0.044667223898300135, 0.07515926648426184, 0.11097970199962508, -0.0425742965127581, -0.027259195522198184, -0.05563336863157944, -0.19708830779010336, -0.07998825400275783, 0.054369130128066136, 0.0006659635347169257, -0.1288567213671736, -0.0030076329067453534, 0.012031963906098564, 0.012075329482548296, 0.05233303964071974, 0.01759315845274713, 0.12816828959880244, -0.009226509974361035, -0.02810234411517935, -0.032668476562689334, 0.017584925873949258, 0.013420046512382702, 0.03597175181291804, -0.023801614354921004, 0.01973341231488825, -0.043224234891390795, -0.007993143760146773, 0.11734740308610853, 0.0014806724338295276, -0.1383780862570554, -0.13558276747482362, 0.2188330433536464, -0.0007996780956618964, -0.01094207570044741, 0.003957370077428587, 0.07210562856938767, -0.027698506988938255, 0.0064430894269130266, -0.14021430212312982, 0.07464715403246992, 0.057885365384854344, 0.007976015161044644, -0.05176688969148669, 0.10278817126182938, -0.14598798945165728, -0.13289032336035272, -0.059488882921186925, 0.014699356690087116, -0.23037923450180814, 0.028578374009415177, -0.18444894335474504, -0.16227320077439322, -0.07560932018303754, 0.131211819973393, 0.037564099970407855, 0.005536788347937532, 0.04974445888785047, -0.22810993933289186, 0.010450971294917826, -0.013668710070215508, -0.04112634183521023, 0.037885632083017935, 0.09523394621869219, -0.09677644786660461, 0.1391282106653328, -0.16242473691234047, 0.01379528828475947, -0.03884360706919215, -0.05238811318743045, -0.06845893422599526, 0.054672848865560064, 0.04740984624403291, 0.0004228703769059643, -0.13472948818197097, 0.031805440837581134, 0.05517858927098087, -0.1963660534417017, 0.060636636915178274, -0.04560333980712085, 0.004002502097525635, 0.07742929965149355, 0.008797304507258285, -0.0017083078751473943, 0.01699300465215946, 0.10194084846344427, -0.01071185469475341, 0.030814515412269865, -0.08733188101067263, -0.00433012141587245, -0.03892961148669383, -0.019275341486219698, -0.14761084144738212, -0.008073798098982825, -0.019542616858196155, 0.037456726989324965, -0.0211427538498242, 0.05515918944270961, 0.05575244680298024, -0.08720199520059717, -0.022112516679522264, -0.059836817560811995, 0.010612616502247894, -0.042636776367322236, 0.0017559368241303055, -0.10406857988128212, 0.08775129719765816, -0.05187991290439483, -0.012858627988639019, -0.04432910887045517, -0.032647978820264704, 0.10991665846637026, -0.026596252927431268, -0.015695481955677743, -0.016716379832613273, 0.02689662781890167, 0.012326462996922471, -0.013919494657201359, 0.05000801698621304, 0.1272850348468658, -0.08861901245878435, -0.21941318146746827, 0.04151527658683903, 0.06746724950039946, -0.11394771065795692, 0.004824598130037371, -0.14539547825845828, -0.05239027469332926, -0.09214736715261425, -0.03982434190608283, 0.004884734259232094, -0.019276442798898036, -0.06744635800735309, -0.054791789608628, -0.06279819906246514, 0.004378761978925835, -0.05832044767889659, 0.11524502529325986, 0.0027979044509238247, -0.07136191425891947, -0.024310985204300146, -0.13086107953488701, -0.07625712384304764, -0.006070304676517793, -0.015048047135879615, -0.024266370359519084, 0.04656332330933458, -0.005399750303697871, -0.06606042879137329, -0.07001143315615685, -0.021272564618745656, -0.030228314846578404, -0.005135145697446921, 0.0262303297380795, 0.07406515262298383, -0.05864962949239655, 0.03944315573892696, 0.08464305905195683, -0.0575418604667333, -0.06945964440922672, 0.11077790714018587, -0.11184242157335732, -0.0702103082283906, 0.03145080932829021, -0.03461278630931507, 0.03037701598532627, 0.04040266573095344, 0.03860464553653892, 0.023879288754204878, 0.097884347795974, -0.05146645276296279, -0.0016733091688027634, -0.00334321130118093, -0.03191897048517533, -0.03886839848240885, -0.009108701292760465, 0.003282738575730428, -0.10175416275192173, 0.028284745410451204, 0.019466862726168946, 0.02377064396859244, 0.0949465418186865, 0.0296160080381805, 0.026661538992117824, -0.11361310892019512, -0.10904308401577077, 0.10253252787982493, 0.02109523026233204, 0.0635099653526925, 0.024095315453262052, -0.040309432096977786, -0.11967285412723933, -0.10422463892633713, 0.007274262972302024, -0.12445057503256585, 0.025810967239210114, -0.11516029712606661, 0.0029321314978066684, 0.019387734636908578, -0.030188426321626983, -0.09260117145179897, 0.011086472170820325, -0.02025106803114961, 0.09541566088386802, 0.13579743621140283, -0.14905413387545602, 0.06791477181173478, 0.03762387140359578, 0.055343407260451005, 0.11972343319058391, -0.11137717155448362, 0.006337301220528053, -0.08586423064464702, 0.07852264825741377, -0.088141055715123, -0.060543643486911464, -0.0871009108876813, -0.01297263255706214, -0.15437212693923788, -0.000521997877262626, -0.014360221492664563, -0.02801352016607069, 0.006292418541433444, 0.06577288897421307, -0.02376660756591484, -0.08067177769354258, 0.14145491555434017, -0.014852486431262111, -0.03937931344278851, -0.06491291352508136, -0.10512875359645078, -0.0475114595690638, 0.003989060429933074, -0.020287928057284837, 0.07084668991912242, 0.06388580342241557, -0.025105443580961635, -0.04682217938436899, 0.15292085302877154, 0.022148504415654587, -0.04946825832760583, -0.09479504443757107, -0.030055483686044118, -0.01764153230002344, -0.10394249293420355, 0.060178712611386326, 0.21228342410852466, -0.04292824447139933, 0.010924500401633295, 0.04200763241762132, 0.09417364217189392, -0.17672466055247193, 0.11213801356515725, -0.016009889971942477, 0.018597546698366663, -0.03253013226629217, 0.053174971436492524, -0.0585818032990296, 0.0463315653816457, -0.0033614718988283095, -0.05430712865249567, 0.06231212636395235, 0.03333989860967411, 0.06149668376036315, -0.041744073303878526, 0.029155408757914017, 0.02992031192716862, -0.06468476326655061, -0.04512006308997262, 0.12826434980434398, 0.031253326564008914, 0.029427181269608544, 0.020874381639880445, 0.08762604279126487, -0.06506986475592914, -0.05699195993843398, -0.0698603339565041, -0.02945653345198159, 0.13200151359144402, -0.13845044486903318, -0.00633934983811034, 0.09470197279816021, -0.012599104607892956, -0.15335542032494215, 0.12288182323052359, 0.16246851156372952, -0.0282761483720472, 0.04549618676699821, -0.04438006615477788, 0.03171365475511823, -0.0699043549809109, 0.21570520884633415, -0.04329987794999147, -0.18052220763078689, -0.10040762569756176, 0.009976520356036663, 0.08639079121377077, 0.07257752635276538, -0.007624448782683564, -0.032738175665674094, 0.09593922863257785, 0.02596379880692031, -0.029529665266724944, 0.022147158685039203, 0.009210484038580559, -0.02664346934549955, 0.11139455118400897, -0.017330354753250305, 0.06999496018670495, 0.051644777283535066, -0.07226959083878638, 0.033895354000390385, 0.005835885206059678, -0.025037507300588888, -0.12357085993868006, 0.1423584402946526, -0.0693342640686793, -0.015664137518020236, 0.03148263753244758, -0.00034669791280730227, 0.09970575429859636, 0.09587187668202075, 0.037377070917042045, 0.014850363787660152, -0.07637041148639226, -0.10436497382903852, -0.043159220629263265, 0.08987376134971505, -0.060182545525773526, 0.13750515475658773, -0.07971558016241682, -0.010051788263632401, -0.0025413399710101847, -0.04947785157246875, -0.05901859693426744, -0.053426321404483466, 0.046827827294870075, 0.13069736528556167, 0.041899596639337296, -0.06697890071848249, -0.019141293786430154, -0.07138805579386577, 0.08584798711051017, 0.06809168107258837, -0.08508254023170007, 0.09814303724721336, -0.08150767313610062, -0.028606242496165134, 0.005637201464066338, -0.05057096139621995, -0.02654487727275967, -0.006150676639064943, -0.06203729615313618, -0.04772922267561595, -0.07314768340943963, -0.029361348298815926, 0.08422913810838813, 0.06489480900167606, 0.02424677613999484, 0.12631662429768842, -0.06527820904592163, -0.06564733417398798, -0.021413728749882878, -0.009276581501631772, 0.0011444935020151756, -0.07952086823127072, 0.07734854650558644, 0.015043458097237344, -0.06600696113831049, -0.01612418933812773, 0.04663404720369107, -0.0024527311336025024, 0.07270307715897995, 0.05741345016320678, 0.03870844436842471, 0.08002969931780644, -0.033828344278458204, 0.12741964849182863, -0.04132855015164998, 0.033386334104439266, 0.02104815052555482, -0.009377160777909512, 0.023464713640125798, -0.030489570791553074, 0.08974398579850582, -0.010187157052018147, -0.018165202637181785, -0.06345751131064599, -0.030110509131554976, 0.03704314013968222, -0.05184275427589883, 0.08422683211023807, 0.0052103119034618795, -0.052987274049886145, -0.006835674837433343, 0.11275694150643303, -0.04007518753942522, -0.020331261528143622, -0.013634628844904413, 0.1615031818164756, 0.07845620565271465, 0.027234071470588855, 0.08353988017151004, 0.04404627800599173, 0.11642013216735068, -0.13177881308483635, 0.028394399842367947, -0.06267309920752862, 0.05350345587054016, -0.066190329193123, 0.01235696439926335, 0.08562916808356083, -0.0028018020243021444, -0.09844421240102708, -0.06210004753069277, 0.044646043599361064, 0.04476024582579116, -0.044765109983608706, -0.11759030381118965, -0.017071479267915038, 0.11715640967604848, 0.01450440677779945, 0.08196669035192486, 0.024355480644263117, 0.020712053132444088, 0.04757082917558732, 0.04311958665432869, -0.04300150829944927, 0.021306623382221317, 0.0068734228885758385, -0.06238940695574244, -0.07184566217435251, 0.0399733727474069, 0.02204782395414655, 0.06329873208500574, 0.024810531938477313, -0.044636513394511176, 0.07346023157361174, -0.05085578988807011, -0.00947366517688083, 0.1924137727022325, -0.02382845112641868, 0.08845137417548908, 0.09316256247647747, -0.024047283599074126, -0.07796520348478138, -0.09977711626560283, 0.07650859218675006, -0.09809069169950212, -0.03758937691120526, 0.03619359220058721, -0.07595456071035575, 0.0037500467331838916, -0.00187286546796176, -0.02725366099887532, -0.053811027812746784, -0.03524448342764551, 0.07173237979789064, -0.056888376384070106, -0.06771915934831388, -0.015513722787658626, 0.05901209103249037, 0.029170260760342535, -0.07096687731548183, -0.11822371816027566, -0.13653891501180263, -0.03758544598866388, -0.0034764920116045523, 0.15628637650666016, 0.0358936216761115, -0.1322933193347048, -0.02014271722889097, 0.03243849343184835, 0.16949905194143092, -0.16820478678896786, -0.0014755333199480323, 0.021857463393715772, -0.01139428400038427, 0.006871071702223842, -0.03330108903724426, -0.0013668968649374381, -0.054900117425387605, 0.005389183598784118, -0.07895535415686032, -0.10873202977874045, -0.12398112995304582, 0.012301447130550291, -0.07218699951231113, 0.054982131541160176, 0.005229896044848986, 0.049266712256122974, 0.08158994159760591, 0.00926504664965068, 0.0662055124017836, -0.011147235355502176, -0.01094506500593934, 0.09328895307073268, -0.1134934378595106, 0.025259029598734453, -0.01246385331490133, -0.015294822196698506, -0.025793456527335296, -0.16149913392107856, 0.06899468116663511, -0.13637289981488035, -0.013441052928678046, 0.0361565177444121, 0.1372256761671538, -0.16077685155693186, -0.0237873807502019, -0.00028651830953742654, -0.023201532218616337, 0.08694604322216351, -0.057096872695753864, -0.0072165904500027045, -0.05871574639315838, -0.05386006402050541, 0.004385068515807666, -0.11179291486161423, 0.02210654601348108, -0.027680107031110825, 0.0019245430207740586, -0.10363612869355945, -0.15652292450258945, 0.09653083596437984, 0.09780903904160224, 0.028521409650397976, -0.06552579437444245, 0.19076801368619106, -0.0015261481622562652, -0.013368616787586829, -0.011673951339718996, -0.08113338901715784, -0.005143550604547566, -0.02553165449677642, -0.05200966051655984, -0.0371497009858499, 0.023178924740999245, -0.035533846319548305, -0.05522207741705092, 0.02999674490738479, 0.016921940003563698, -0.08543324302285449, 0.0722354208109651, 0.0880134601520802, 0.07593168536963252, 0.03149893589583702, -0.010112846917350064, -0.010392110698866596, 0.08717066543003545, 0.11748812258868109, 0.0352049307052035, 0.05833537940852633, 0.018896562735629948, -0.07831652071788944, 0.0845425094597661, -0.08313572538009983, 0.015168758466602475, 0.08571684200726673, 0.014876783157024577, -0.04195349456700774, -0.012367844437670613, -0.041510675882928554, -0.07231524972032032, -0.048274503436370124, 0.09395824529837014, -0.06565353504990329, 0.0009755173490474449, 0.12012674895454338, 0.05252125467407815, -0.0130919255098922, 0.07933092561791003, 0.026065491401058245, 0.06730102645672384, -0.09140438842197122, 0.023723418722150485, 0.0005713722460122673, -0.13407608996945955, -0.08081571072096841, -0.16167230786349474, 0.07663937066095727, 0.0038360251980533917, 0.04974915981539856, 0.05570011379497582, 0.019751070414037657, 0.021949495428548984, -0.09142923061867549, 0.006919095338510525, -0.017239747348639645, -0.057640903590569705, 0.021831312456678468, 0.09619079645150984, -0.042766473126829144, 0.04206978784439364, 0.17427514645628286, 0.06528200339352437, -0.025732130579457738, -0.01494867229079602, -0.16563743716533114, 0.03006710183568177, 0.021521306610359635, 0.13885754501425301, -0.02811650338482696, 0.051543107664817366, -0.0564177839946177, 0.09598620943879216, 0.024036309374269062, 0.052559825640953774, -0.06021556261419157, 0.04821203451448068, 0.005358368422156772, 0.0056054377556192, -0.04317800998028425, 0.007709992804732104, -0.04102229905035458, 0.06810335962899293, -0.0824346304131347, -0.07158385721064137, 0.03378766842765732, 0.00027106738688958287, -0.05119115164755928, 0.042185173050866054, 0.08619666718197862, 0.07426898431306415, -0.1650700470632143, -0.01963917714845744, 0.025774475810621297, 0.07477684462701585, -0.13710838987326066, -0.0920462183998427, -0.011676238501392358, -0.03552911733795665, -0.04268592721414078, 0.019044409376121854, -0.0573629913170978, -0.01572590366863141, 0.030410724808095764, -0.0662719549360902, 0.033433342881450535, 0.05384454733310143, -0.006289696839243338, 0.01551501736021288, -0.0018764202141899017, 0.10020586959402544, -0.02115913114065083, -0.167511103826744, -0.03217219259341748, 0.05884953526205224, 0.02224761562250918, 0.010965985068944625, 0.06369510680175168, 0.06540514661389638, -0.012970917720036612, -0.022675912401709204, -0.028021343905657147, -0.0006652750942568508, -0.009213818204501272, 0.09633501437526547, -0.0002632752262006693, -0.07281443197681393, -0.10378126842761841, -0.06134679707499761, 0.03904946220067093, 0.006320480472020797, -0.0349107931340687, -0.07552229012096061, 0.061262476895344385, 0.022700219930468356, 0.020426955517978716, 0.13460733302688713, 0.1035247083519105, -0.02234744929021029, -0.016100522768104465, 0.008195929313782388, -0.002216312974792262, -0.06194985940513332, -0.17482988821517637, -0.027848861676280253, -0.04558994681686324, 0.04376635119947438, -0.0803392906190052, -0.075905501629717, -0.08873042856762968, 0.08598287004174882, 0.04570621552144271, 0.1022016147728532, 0.01805302306071323, 0.00233853977978267, -0.051017710574042405, -0.1233511611081547, -0.10814972199037527, 0.10896929684240501, -0.16239183685063657, -0.033492347226866226, -0.05097223722784286, -0.12427443137537848, -0.12301837057661834, -0.11752920006775716, 0.0980302878409272, 0.07669574605637053, -0.05436190921084818, 0.22778873321135185, -0.0275159320368603, 0.021809905685185354, 0.09257097351356516, -0.17978566724113276, 0.09042148057269085, -0.039458476448649, -0.04832175698376189, -0.028114096803746658, -0.11880775962258754, -0.05315775852102746, -0.11175402873821666, 0.06461370345052746, 0.11471460243815586, -0.03709745996060331, -0.0013757991328639794, 0.02363152134149809, -0.08907512768935764, -0.12203106966113834, -0.0768726874560877, 0.10838381576876746, -0.04406844660343166, -0.06838234817813511, 0.01666858376411305, 0.0845114274689447, 0.046065020283220764, -0.005188520752140577, 0.10369914717492179, 0.12688188462681244, -0.0023501425386680827, 0.1336476902339533, 0.08376259819967959, 0.09116279534648598, -0.03798139768920911, 0.14156627418679651, -0.062209769019058926, -0.060239370493459704, 0.0794591385706962, -0.008817691666466924, 0.012920346496073324, -0.037313654004748245, 0.014340368993225536, -0.02725931443359953, 0.006738870610328055, -0.14556162865953762, 0.03594254361763041, -0.05701973338998936, 0.07673306051444874, 0.10751699551778947, 0.013858765300099022, 0.058918850432602456, 0.11818396076904221, -0.0018523727194887439, 0.07212633478694162, 0.035855057125560974, 0.01350873387673232, 0.043328673905946594, -0.08305541083779346, 0.05202892416096743, -0.1082124740861652, 0.10058440751287814, -0.07412093461269054, 0.024428147745086424, 0.06321637208740376, -0.012539978265617426, 0.03028841631514544, -0.046918592194875, 0.11440436304863916, -0.044668689613096745, 0.06356071963553812, -0.10871247499084982, 0.04846415137521775, -0.0016613054190285689, 0.16005730348964245, -0.16639026427252657, 0.08354924653998443, 0.13414576558379296, -0.062065646431044626, 0.07925872674948428, -0.020078482938342763, 0.043951241572535125, -0.14793742266021587, 0.017581319306476228, -0.02801045417747938, -0.049180488410844596, -0.048010197363126256, 0.03940188724305884, -0.004012086317212499, 0.10809436973901136, -0.05918858140144349, 0.09786439388651963, -0.03410360438181554, -0.030103630846308228, -0.104163742772407, 0.03883943578449946, -0.02281485879467897, -0.07508251716949797, -0.026897510988686292, 0.08029198594455526, 0.017902106862030056, -0.06842100729897742, -0.07581271257825768, 0.14998775835308878, 0.008935806042978488, -0.03982325733778806, -0.05120646133437719, 0.03235652914089138, 0.12428039337207905, -0.04505661854054001, 0.024274413296976802, -0.13222708955230242, -0.015209487141084483, 0.02005167033619914, -0.005795759739873482, -0.0045702099536311745, 0.07429413902123778, 0.04178290179961632, -0.06441141370516405, -0.08406311164587213, -0.007939711893371325, 0.07753570740335292, -0.023714339014802396, 0.12895869481711678, -0.00532582837696279, -0.07313858853004153, 0.014764075802071844, 0.011037295557665803, -0.0741950798207792, -0.05455894993853296, 0.05992488427095827, 0.02256190342729738, 0.09198121286084388, -0.17192881097438104, 0.2074267748255055, -0.022907137431762082, 0.009172662556968069, 0.03863528103939554, 0.1332864344112686, -0.02168586639608919, 0.02938462608148993, -0.010474352593126674, -0.09187687898634497, 0.06572563206082675, 0.08418052740754413, -0.1348754400821947, -0.053810121429867006, -0.023793696950604393, 0.08776942524280743, 0.04098198961063939, -0.04764090668270893, 0.01621869528750269, -0.004305225533952881, 0.03349054553173595, -0.017181754218831714, -0.08384636739114425, 0.010528843332662944, -0.06776295515541758, 0.04204876261683484, 0.022977688494036447, -0.08686806817033871, 0.07035825955613123, -0.08444857924050414, 0.02578921367317307, 0.12418759015558281, -0.04301887905913948, -0.15563716878249176, 0.09600244208440668, 0.034721603257853964, 0.07980771912545415, -0.008817047430688837, -0.013970020002680963, 0.008770829903841091, 0.07864031289265384, 0.06487881397526712, 0.031898686461571686, -0.2452745214608476, -0.04999994756304601, 0.05519807240781848, -0.033193184032437484, -0.02153102868564845, -0.01014366946346589, 0.011964418310838427, -0.14559542630572797, 0.0334737891255424, -0.05995282938239853, -0.01522118457462416, 0.08937961828547801, -0.0614378545155015, 0.006175773098570108, -0.05111023737558967, -0.04194109407048542, -0.03468309265922674, 0.05640950057141451, 0.04495870437290665, 0.019214341593718945, 0.07964756385919876, -0.06128198227006044, 0.029109381901398865, -0.00029127367889243797, -0.02695154810342181, -0.00021410101930679958, 0.019270482466904226, -0.060844515129815854, -0.03029116010234744, -0.018425571054396274, -0.03602502470562241, 0.030306503879495236, -0.10736954501087066, -0.04544883577445728, 0.05456377859552962, -0.09980751062066172, -0.004908556950448859, -0.09016492316977588, -0.10420971606733404, 0.09570957933418285, -0.05038511745306976, -0.013188647025535653, 0.0592758750416811, 0.1331826669040103, 0.08041327075045297, 0.049208834945759394, -0.003205195187852812, -0.008016047400839912, -0.057403520138931165, -0.1007487581171927, 0.055477190897430244, -0.11072439773780407, -0.10652443359890874, 0.12281180212594256, -0.08429739554202277, 0.043624039897520665, 0.09117382695665985, 0.19781967765388955, -0.028634221655682152, -0.012623898082737823, 0.021847189930024286, 0.023709992707304176, -0.04766880513870386, 0.02270742208979758, -0.13552275875614792, -0.06409505707193676, 0.02114427465946226, 0.06130655894612356, 0.020649188166299837, 0.0038028334016743446, -0.02370356487285526, -0.04171525957123282, 0.055113296445072776, -0.043688742733391635, 0.0009184937291001525, -0.10912853386478447, -0.07960078039642222, 0.15478676726253426, -0.034113988379373864, -0.12606964228557857, -0.06257380946597324, 0.0849542588726041, -0.052246717076989524, 0.07648924711682273, -0.10956309101207536, -0.28207001369728474, 0.022484603905470524, 0.027617380661256238, -0.006774047631248962, -0.1574278017441424, -0.06147189953636445, 0.04309064406667504, 0.09672463176031978, -0.04286895420795, 0.039381127743238195, -0.021917519883460704, -0.05175660576482045, 0.06001687596721329, -0.1697143760881299, 0.01874399489235744, 0.050000367420701894, 0.0005482196504233053, -0.04445351062561434, 0.031034017738447554, 0.10539047421037737, 0.051711923077639596, -0.08032717661200173, 0.04669804489321302, 0.10922013972892998, -0.032539563768862925, 0.039560619439527656, 0.06911975161475481, -0.021785546450132403, -0.10563109274615573, 0.0462884888694418, 0.05237508269133205, 0.11125026680173492, 0.09553760157064262, -0.08472734600999805, -0.09717371634903146, -0.033884433039928885, -0.031074128507752534, -0.025517724684719502, 0.06535416440602106, 0.05952056874248358, -0.00993374986740705, -0.04395540358037401, 0.03896928149176565, -0.013053201658843033, 0.16070056214547057, 0.10439292305374025, 0.04956311803709721, -0.024591157823729774, 0.025835728245710583, -0.10810487573524231, 0.09696478845601902, 0.01091354818304772, 0.05687816436852533, 0.09542812098951844, -0.18784677050398169, -0.015891303556260603, -0.06386649193319796, -0.006951065990077163, -0.027500217480042183, 0.007662310803967353, -0.05517998862076557, 0.10123053892292447, 0.18009698775114835, 0.007359141076846273, 0.004048470248734508, 0.0031584580921664445, -0.0663965368064889, 0.027008919282011105, -0.054473027528587736, -0.11282936889724554, -0.011671599414786469, -0.008227575154240259, -0.0062443501668642, -0.07511207639068147, -0.022102548019346763, -0.008422804818573942, 0.02301951030878926, -0.01613018797994562, 0.08248230047168119, 0.009297629081454714, -0.08451591470976333, -0.03068435693581472, -0.004093275006200309, 0.044134600743750405, -0.06710093904442671, 0.1767191929542516, 0.11694450448408927, -0.08811417288528961, 0.03576123685217282, 0.00535642655293163, 0.006237286489803379, 0.07712905942607351, 0.035759439130413406, -0.10969919610224559, 0.01578843651084065, -0.05367017605033081, -0.04614782685311844, 0.08671414568469549, 0.1438282008195337, -0.0212871939486435, -0.07762673031043842, -0.09002246859802251, -0.022952936854609193, 0.005677313176803801, 0.02097552664120628, 0.06171151046987223, -0.013968382957854542, -0.0010720731768223007, -0.013197325004094174, -0.07202226527990575, -0.013112869725668717, 0.03968413039609869, -0.17890601853770297, -0.08706553454251703, -0.08343622137985164, 0.002328348487743058, -0.13964273791942772, -0.08509828843677651, 0.011672839050922076, -0.03806945002163694, 0.01692417902528101, 0.04347888535364836, 0.05191679488727927, -0.06255475355159529, -0.01334331035368848, -0.02939838506721982, 0.027692264023807676, -0.023931607924829335, -0.012967419488963244, 0.10458726048539425, -0.03383787761054207, -0.05450046556546021, 0.021796986771371818, -0.08951580780603118, 0.06748749538622538, -0.059457862976176876, -0.013854935200620741, 0.03464939296196445, -0.045131492873948055, 0.12974684300940498, 0.012336023854737936, -0.07482279142500282, 0.03541984266979618, 0.05615221413317432, 0.0584787383872387, 0.15528983835751892, -0.20173412826530676, 0.03401150232002261, 0.07754362779608011, -0.09796059831985919, 0.08238384859348331, -0.01986126200776286, 0.0782073722222816, 0.09002691056486, -0.16274312881480232, 0.09215075103534372, 0.011789674447951563, -0.011491797467628441, 0.012566919698000834, -0.05736396025096001, -0.00015578144482404337, 0.0528510862850408, 0.10020783807653713, -0.053104048208741456, -0.014300772634587202, -0.07686392025865268, 0.04218335254061487, -0.14559118756242206, 0.01898064330628492, 0.044315454567000104, -0.03550404704761097, -0.012664384983830481, -0.10122508511631698, -0.08553260767238884, 0.1309229577979946, -0.10682784299992722, -0.06429435992346949, -0.0761383382009719, -0.07565257125683553, 0.1651043150459566, -0.024842588181167642, -0.04713975290463425, 0.014136020867850903, 0.02454748883757318, -0.19782186877770405, -0.04673155953312825, 0.053293094852972324, 0.0748439192887805, 0.07810532623312624, -0.072834500253196, -0.05532933227172406, -0.08396364251548748, -0.04649653813837366, 0.013907324815263028, 0.0841831772619278, -0.08398131021055028, -0.0035105165864354445, 0.017935141065616007, -0.0209413088224811, 0.06299192011902834, -0.1818340022041858, 0.06913996565390249, -0.09483345856858394, 0.1398091376633119, -0.026894570029643336, -0.13911592928762942, -0.08138016331167741, 0.0833235021764781, -0.12251156671672574, -0.04263147241450373, 0.05434162642247029, 0.011783648398003133, 0.024242925866621568, -0.14363235794263501, 0.01947003742633395, -0.06684969012420933, -0.07662821413637844, -0.12378213749843911, -0.07993343058593015, -0.07621456927790082, -0.1429648690529758, -0.014738619198510721, -0.06445274162054847, -0.029279746115927662, -0.11247495469104205, 0.005417219457952601, 0.06783167475018108, 0.016319569000152048, 0.10702119371356261, 0.0644858892243478, -0.10938145092972788, 0.011591731694500089, 0.03633829022430113, 0.022954530039298156, 0.02416701607394362, -0.06942798994267586, -0.03171871038129916, -0.09846501382961897, 0.07680750416722722, -0.1025163573018976, -0.011297768724694976, -0.11999941483526089, 0.050634308897611495, -0.11588819771135854, 0.05818484403267871, -0.08616339699490567, 0.03439616345995043, -0.022380441599587807, -0.07399039334027922, 0.006277827422448743, -0.026584742518793628, -0.03226262902616402, -0.03169087697465753, -0.036091541727793164, -0.02990465083251719, -0.03913718201873618, -0.03681452724926775, 0.05877013604626568, 0.1022094007229836, 0.05251555157604122, 0.015222012368997723, 0.07034317165693613, -0.017103543384683943, -0.04222282448926612, -0.09515031865227368, 0.063417733520284, 0.05439890568110844, 0.07181501434620567, -0.0629455528287942, -0.006161899310751245, -0.06973720094961357, -0.04845140914137588, 0.09706650149085709, 0.0239799162563712, -0.05581054920116777, 0.16040623387902755, -0.07495674354694255, 0.013373460594419512, 0.0380523857470755, -0.02194448315017903, -0.016557674314353014, -0.050956394009503456, 0.08722262766763869, -0.05832694604143813, -0.0076917287100257935, -0.030391863100702885, 0.0066321893159624985, -0.014055018075716142, -0.06558070466105025, 0.018315366056592244, -0.03154611960139651, 0.08553667428509268, -0.07392875978187573, 0.06330672711827486, 0.11341355739619455, -0.0018907314824813204, 0.013461909596114935, 0.024982305492057103, -0.01252903317743505, -0.0022174694008559315, 0.07160326834856492, -0.024633505154978596, 0.08057702892965767, 0.06882697665181031, 0.013447259643171302, 0.00016341911993160907, 0.08294374856315316, 0.0033501001896480234, -0.14668573469861185, -0.11167044239989886, 0.12824749323267598, 0.03466116974586102, -0.03622038612084144, -0.01070314254506203, 0.09592295660407368, -0.09093518867502226, 0.053900939241900786, 0.03373746841252006, -0.053881700793947274, -0.023682748920223238, -0.012222325504916417, -0.0332853879138699, 0.06682768595006537, -0.010009394389941246, -0.05661006594088287, 0.038631614453279985, 0.001865946396281145, -0.08163830187418676, -0.043177622151698576, -0.010356747519783542, -0.048514662618879247, 0.033488478532475906, -0.008030045132956163, -0.11658593089630535, -0.1036108253746012, 0.14448806237077055, -0.002199564600693126, -0.011839202632911515, 0.0011849644907977923, -0.03680863866350988, -0.012786535264885296, 0.014265807440876658, -0.039532711764579224, -0.025949024783516864, 0.008847424291573758, -0.0677640019951645, -0.03535666839892794, 0.056957098012939834, -0.13030675220366378, 0.027710115984584022, 0.22435147757766807, -0.1278615407917833, -0.015877778836061694, 0.1010663678156004, -0.01309183630010078, 0.11254928443960606, 0.05233025114963181, 0.04526062500893407, 0.08248954296471815, 0.04256642242395127, -0.10787517260447566, 0.03554839914124673, 0.04033695050671685, 0.030761141066488872, -0.04956513086162408, -0.14640308172072058, -0.08083702507261356, 0.050749854290386406, 0.015026445871675212, -0.054255013866000755, -0.14904040196496532, -0.023506268291970493, 0.027426361135149402, 0.1122176590584655, -0.15048651913840266, 0.030380584439559218, -0.04289013704738403, -0.09760003432400755, -0.08690085893611231, 0.029231875221849273, 0.018339731208713378, 0.17623585510805095, 0.11911402880703877, -0.01130758197692907, -0.06378541614795644, -0.07352366894183955, 0.05487861716217607, -0.050701443715714035, -0.02961176313003976, -0.03770661931762927, -0.07026695112516992, -0.002697780400620785, -0.07313130236610928, -0.06888089751940245, -0.036861571033036875, 0.03403214079392783, -0.02131534028830922, 0.07844618518011441, -0.05275590322668345, -0.05325716777284843, -0.1602352548724011, -0.04935466995638022, 0.05612096173336967, 0.026984137699484975, -0.10612956516691088, -0.11750776377724531, 0.0692103609916955, 0.0015791090455289233, -0.032536500854836274, 0.05229604180160454, -0.11744202283241911, -0.03794370151973959, -0.009793272949044914, 0.00658684043919127, -0.014735487009565864, -0.1354972465012078, 0.0008975565174666234, -0.07943929327020696, 0.06710354718947563, -0.06053271539859515, -0.009125388746173845, 0.01087316221209517, 0.053001591385012196, 0.01014620828546317, -0.12411614171972991, -0.042118070572932265, 0.16338995345542107, -0.06948112479442173, -0.0018823470801702946, -0.02718210512100856, -0.014381155373070508, -0.018001318562196233, -0.002864800414795859, 0.015694731643783807, -0.06107454524937436, -0.04983794363469602, 0.03420965818819819, -0.06566609365068098, -0.14665884071577218, -0.13581252249112302, 0.03620941385941251, 0.051424235893114234, 0.07040348038325385, -0.00714562121173602, 0.11950352850065236, 0.020111033284635375, 0.07820219484744793, -0.011604668051211317, -0.03753499828495331, -0.041988288788666665, -0.03406145308158891, 0.03272416214226583, 0.01249494541799145, -0.04361897902211027, 0.021416267449421343, -0.009887923680322845, -0.04439142733073846, -0.05776880259694396, 0.027414502120851586, -0.15534584924146871, 0.006687064449294574, -0.06372281338109863, -0.024294488836600025, -0.11061618188683653, 0.012705193180287264, 0.06966510519723158, 0.061485982710719526, 0.06557260313633946, -0.12933509647754854, -0.018458852129863, 0.04065797251997594, -0.01733967914717019, 0.03597142478830845, 0.08103109542415095, 0.04954161999875322, 0.06790806984275008, 0.015170815799172522, -0.03869846217308259, -0.020899997413985637, 0.06162322041732756, 0.04465233164486968, 0.031757775623903445, 0.09340652716449963, -0.06866185052628579, 0.007309932602875656, 0.01442119524086795, 0.022163024108169085, -0.013287468259732208, -0.0370862403284174, -0.009827681926363708, -0.005929807758662797, 0.10799805102911884, -0.0689099534600796, 0.024530919675952733, -0.015008894791074435, 0.04227032578597948, 0.05440795289884961, 0.04814082389912164, -0.08943402818304078, 0.05882907381694011, -0.07452272324710843, -0.010464260240919777, 0.19234315026394405, -0.10029736460947893, 0.03579511684036915, -0.06796648199916441, 0.05042070805380986, -0.0015396832401684863, -0.0007181641168398647, -0.05333773054825608, 0.04395132870752048, -0.11920289036121282, -0.08614766682536053, 0.043348262601723704, -0.11659932044494646, -0.06032057143432177, -0.13248805286396007, 0.01634461998244022, 0.02756338570065461, 0.14984591444552375, 0.10803672464123812, 0.03183633767723971, 0.05800595517930555, -0.04052597073453175, 0.09591840417387827, -0.02664807346029104, -0.0839464016467764, 0.029524451494234816, 0.01272213931523672, 0.07922017256905624, -0.10094528278889306, 0.047826984296118695, -0.019357886739241156, -0.09122364894987844, 0.0011955350722402374, -0.050300915952948194, -0.05287441948220331, 0.06897129162319941, -0.0708624021601062, -0.00019989664080334498, 0.051731589587083515, 0.013169477490626035, 0.034544780598185186, -0.10311989221685279, -0.0826886837487658, 0.06850603222342806, 0.0929811578965555, -0.03682455443803574, 0.06764190347033644, 0.0702144020423434, -0.00910780026115947, 0.11332038108355637, -0.019103825123386937, -0.01586825318659396, 0.17224707988829818, -0.026660180458095872, -0.14335436684817762, -0.05720421718930292, -0.13145893852809779, -0.0017120788391310963, 0.10929071856155588, -0.044081953493234856, -0.10355864195634575, -0.010602157637601224, -0.059905587634024766, 0.0037261687966513614, 0.14529065593200294, -0.07850972486024271, -0.06204316238130795, -0.13102302830704468, 0.06233426360753789, -0.10671872968150078, -0.16710630462280698, -0.03399393371670527, 0.05337717236468841, 0.05428023365046334, 0.040464717144637785, 0.0955362273351002, -0.0582924920005651, -0.21136123738234408, 0.12323469406596822, -0.02932761121781246, -0.018889783612796647, 0.02170206625341781, 0.07048658516027327, -0.11171929366374288, -0.11109623465057879, 0.003936582153458017, -0.0022508427638529434, 0.0302627598879385, 0.10359407610719602, 0.08666148610786596, -0.07205528861460522, 0.01993769556406609, -0.025140993717676318, 0.008430182411906016, -0.10103400940682158, -0.09736112601965738, -0.08466391796698773, 0.058554426954558036, 0.16439754407668128, 0.012998930440720114, 0.014285269451224918, 0.06115698606130318, -0.004144215913482108, 0.054138203876047, 0.048011201356764524, -0.055015568574132694, 0.02560683908834073, 0.20838029002571196, -0.005663940479259222, 0.07153877118774248, 0.05578934870334072, -0.01526668570434971, 0.011858316044301183, -0.05154754837010362, -0.007049239193874473, -0.02841263627260335, -0.00833420582676164, 0.03128575978622873, -0.05401717731726511, -0.18117019617698638, -0.11060571258349591, 0.0497979187917049, -0.04142122423935553, -0.09114217064311524, 0.009983899530934653, -0.0697928082888535, 0.14204040628126818, -0.09175700360232368, 0.05656244498711007, -0.06382002712486576, -0.03682316314011474, -0.02045476423910314, -0.1365510368638118, -0.11264657960232931, 0.0040285870107449814, 0.0007209214688189401, -0.09963279617561484, 0.006191045952363229, -0.06758573191187342, 0.050926628013846446, -0.003041538370009551, 0.04550356576661926, -0.11892951447001535, 0.21519591826415224, -0.08258183813783133, 0.03754115833283299, -0.06861471115426104, 0.05726721940423982, 0.03026205772983852, -0.010101464394904596, -0.10173858781105304, 0.03851020907128126, 0.033528300915011515, 0.05342347767255808, -0.0528353230677862, 0.02914408799217823, 0.02084572012127546, -0.058269477096950094, 0.007544423909671033, 0.01628218243337091, -0.03781298270927031, -0.054863494617444004, -0.09748663952753822, 0.03869286696397531, -0.048367001575465855, 0.03225423418032917, -0.13434379348520029, 0.03185382185386083, -0.14025299465690005, 0.09200696881641199, 0.07236133912754669, -0.11463282964544037, 0.03701689727755334, -0.07344666218194253, 0.021381122879796842, 0.007544498567900464, -0.14483430590554414, -0.0979459692681748, 0.062347837154335166, -0.048237149352686, -0.010670224399692056, 0.024760565873857273, -0.14759224906054172, -0.1206414118675447, 0.008083654369896743, -0.05334154513742963, -0.009466458052463936, -0.00031623664765658, -0.11935254593803155, -0.17420852373190313, -0.09415296768403945, -0.01694060521316381, -0.09441613477584707, -0.07424502525216892, -0.16331049889460264, 0.04789950133985157, 0.12292125550915736, -0.14456997156724405, -0.0859040905819195, -0.05566505175731575, 0.029380031216476258, 0.07733353627601978, 0.0036882214623943413, 0.0992660111987095, -0.1681221811553535, -0.02365160436846403, -0.022493481341213473, -0.04328000289671685, 0.009964176579832899, 0.10782434925010541, -0.12760774360921856, -0.00012570849521128957, -0.05836983991794299, -0.088912746684951, -0.07178681109907209, -0.07205095506534287, -0.07782591619603905, 0.05721163674771863, -0.016707626311948106, -0.03849755898232064, -0.013236578233057182, -0.06633488136597683, 0.05254307362117353, -0.02823053606592985, 0.0210658650615667, -0.1113202357182957, -0.009888356548145903, 0.06599996844658414, -0.01424467854203876, -0.1850682558111528, 0.0350515183129265, 0.08939779403367566, -0.08501396050422293, -0.014793172656604667, -0.012870234288619352, 0.05085324092475882, 0.007883990242142156, -0.06516760670547328, -0.10704296626747097, 0.1157844768060406, -0.04049951067322391, -0.0765137373183603, 0.00760624852291672, 0.210159525932342, 0.10984473723592379, 0.10473305945327464, 0.009291502530427922, -0.045151762289960165, -0.042078544784671955, -0.05076747834522571, -0.034225326550464286, -0.0671630261152875, 0.0807685460526718, -0.10685739020210491, -0.08127230183790628, 0.13369883364164523, -0.012479572053379108, 0.007872786795638548, 0.04183176296389828, -0.041406089417711706, -0.10883263173926781, -0.05870311049816642, -0.010541107347185557, 0.034348436262613344, 0.0936402070121817, -0.03509149851424228, -0.09046421517908104, 0.06668652578942676, -0.05897951015481536, 0.1427004971693208, -0.03544670374837221, 0.0087800419717541, 0.018831670687374655, 0.0049331778231104205, 0.08636197994176761, -0.09727898555262252, 0.04980419469412648, 0.07998066721982663, -0.023177108282735503, -0.024102270147213136, 0.015738234978480776, -0.06226459360560989, 0.03847398264485567, 0.13888736203615665, 0.0882687131319917, -0.026585797906291155, 0.05597070853696476, 0.1643680967982826, 0.06434120715479576, -0.24922200392963043, -0.10733391701822126, -0.13253370993310803, -0.09007242021107605, 0.049451950319460744, -0.0006112302495865114, 0.03174639973178719, 0.0683979530354129, 0.11817369203921117, 0.13809625392262012, -0.10852767075453618, -0.02846749714820898, -0.05873912676534482, -0.08070784393671537, 0.07631501963606944, 0.019109412071911372, 0.06712676333505624, -0.0011149604366701784, -0.0688751723139695, -0.06788188543292709, -0.06333740502446016, -0.06035684305190966, -0.07393904890877737, -0.17329077231128215, 0.051233453379301, -0.004252009855096362, 0.005144089397322666, -0.08785047121090421, -0.028688238662340517, 0.012464308603095836, -0.060733106248115745, 0.03680019945677554, 0.001102964265719388, 0.07955626367587283, 0.04350103481125462, 0.12832067406377232, 0.05205085566216039, 0.04827233355180758, -0.049228266077259504, 0.09330027200391919, 0.010223539886924324, -0.014380913535599272, -0.05529151352479544, -0.025951152942744093, 0.014480132417410233, -0.025048835715769976, -0.08829189566443314, -0.25760092389201317, 0.06862869315311443, 0.06093498840536678, -0.15232573239545275, 0.034258335971016636, 0.0018727451757549944, -0.028014193181529274, -0.08448554945655835, -0.04286325022625546, -0.0027931580166374033, -0.0973802215886384, 0.03090789662459676, 0.0029974319955773164, 0.044013344887525195, 0.044140423876463254, 0.09395451899519584, -0.022369985312803632, 0.0232419384705683, -0.013817658948011765, -0.13617303345249746, -0.059973775299516634, -0.014627430939381084, 0.021088854774574083, 0.14122134003064526, 0.08130927907945806, -0.0221976511554664, -0.013461233239003952, 0.02872922220994426, -0.05277011130260709, -0.03546384084392327, 0.18057423968914504, -0.04182998250935091, -0.063689015570194, -0.14835924916288112, 0.017141288908108725, -0.025663640829149046, 0.10766268154772043, 0.10553975854360935, 0.0011773854583637991, 0.13429345910703203, -0.11378715895923278, -0.0676117164743029, 0.11043189698484444, 0.06956977121492686, -0.058423811750040744, 0.048003468395915704, 0.0594281940744574, -0.020884668917268267, 0.03700708136293553, 0.08022654364865826, 0.02799023993078129, -0.11604801058841194, 0.05786827489946729, -0.05896471000793287, 0.043196240055966534, -0.07196625040391985, -0.0734642265870166, 0.03250516042088788, -0.03638265229074507, 0.06628283435176169, 0.007944479602625438, 0.1335636708921469, 0.03642674130617015, -0.030631470304857213, -0.13523972974299392, -0.06546701803763003, 0.07259343035308152, -0.03209327789899429, -0.09366357699799993, -0.015404952576806661, 0.031169860072719022, 0.02446602061100134, 0.02097460888977597];

In [31]:
center1, center2, members1, members2 = project_by_clustering!(x)

(0.05817174719386851, 0.904941711336926, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000, 10001], [901, 1204, 1306, 3160, 6879])

In [35]:
members2

5-element Array{Int64,1}:
  901
 1204
 1306
 3160
 6879

**Conclusion:** We kept 5 non-zero entries and projected the rest. Seems promising!

# Run full IHT with clustering

In [37]:
@time result = fit_iht(Y, xla, z, tol=1e-8)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse linear regression
Link functin = IdentityLink()
Sparsity parameter (k) = 10
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 100
Converging when tol < 1.0e-8:

number of nonzero entries in full_grad = 5
Iteration 1: loglikelihood = -1636.1269996211058, backtracks = 0, tol = 1.2369352935298255
number of nonzero entries in full_grad = 5
Iteration 2: loglikelihood = -1622.5778553429302, backtracks = 0, tol = 0.07084733043788174
number of nonzero entries in full_grad = 5
Iteration 3: loglikelihood = -1622.5737609326382, backt


IHT estimated 5 nonzero SNP predictors and 0 non-genetic predictors.

Compute time (sec):     0.04921889305114746
Final loglikelihood:    -1622.573743096104
Iterations:             8

Selected genetic predictors:
5×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      901     0.733432
   2 │     1204     0.670333
   3 │     1306    -1.42341
   4 │     3160    -0.856148
   5 │     6879    -1.2274

Selected nongenetic predictors:
0×2 DataFrame

In [38]:
# compare estimated vs true beta values
[result.beta[correct_position] true_b[correct_position]]

10×2 Array{Float64,2}:
  0.0       -0.402269
  0.733432   0.758756
  0.670333   0.729135
 -1.42341   -1.47163
  0.0       -0.172668
 -0.856148  -0.847906
  0.0        0.296183
  0.0       -0.0034339
  0.0        0.125965
 -1.2274    -1.24972